### Analysis code snippets for preprocessed h5 files

Organized as follows:
1. Select runs and filter based on number of hits, including choosing ions, electron and/or photon data
2. Filter based on pulse number
3. Calibrate runs for m/q values
4. Heatmap and electron time of flight plots
5. Fish plots
6. Intensity dependent plots including into time of flight plots, waterfall plots and heatmaps
7. Presentation plots including fish plot, heatmaps, electron and ion data
8. Covariances between ion data and between ion and electron data
9. PNCCD photon data

Need to install reading methods the first time:

pip install --user tables   ### to read dataframe
pip install h5netcdf        ### to read xarrays

# Imports and functions

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter

In [ ]:
TIME_BETWEEN_PULSES = 3.54462e-6
CHANNELS_PER_PULSE = 14080
channel_time = TIME_BETWEEN_PULSES/CHANNELS_PER_PULSE


    
def read(runid):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event, per pulse, and xarrays etof, pnccd in that order'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    
    dfevent = pd.read_hdf(filename, 'dfevent')
    dfpulse = pd.read_hdf(filename, 'dfpulse')
    
    etof = xr.open_dataarray(filename, group="etof")
    pnccd = xr.open_dataarray(filename, group="pnccd")
    
    return dfevent, dfpulse, etof, pnccd



def read_ions_electrons(runid,tof_limit=None):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event, per pulse, and xarrays etof in that order'
    'tof_limit limiting electron time of flight loaded in'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    
    dfevent = pd.read_hdf(filename, 'dfevent')
    dfpulse = pd.read_hdf(filename, 'dfpulse')
    
    etof = xr.open_dataarray(filename, group="etof")

    if type(tof_limit) == int:
        max_coord = int(tof_limit/channel_time)
        etof = etof[:max_coord]
    
    return dfevent, dfpulse, etof



def read_ions_photons(runid):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event, per pulse, and xarrays pnccd in that order'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    
    dfevent = pd.read_hdf(filename, 'dfevent')
    dfpulse = pd.read_hdf(filename, 'dfpulse')
    
    pnccd = xr.open_dataarray(filename, group="pnccd")
    
    return dfevent, dfpulse, pnccd



def read_ion(runid):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event and per pulse'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    
    dfevent = pd.read_hdf(filename, 'dfevent')
    dfpulse = pd.read_hdf(filename, 'dfpulse')
    
    return dfevent, dfpulse



def read_pnccd(runid):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event, per pulse, and xarrays etof, pnccd in that order'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    pnccd = xr.open_dataarray(filename, group="pnccd")
    
    return pnccd



def events_selection(runs,thresholds,num_pulses=None):
    'Reads one or multiple runs from h5 files'
    'Makes a pulse selection based on the number of events per pulse between the defined thresholds'
    'If multiple runs are passed, will merge the runs, once hit selected'
    'Thresholds can be between one and three tuples (lower threshold, upper threshold)'
    'Downsamples by num_pulses'
    
    lower_threshold1, upper_threshold1 = thresholds[0]
    selected_dfevents1 = list()
    selected_dfpulses1 = list()
    selected_etofs1 = list()
    selected_pnccds1 = list()
    
    if len(thresholds) > 1:
        lower_threshold2, upper_threshold2 = thresholds[1]
        selected_dfevents2 = list()
        selected_dfpulses2 = list()
        selected_etofs2 = list()
        selected_pnccds2 = list()
    
    if len(thresholds) > 2:
        lower_threshold3, upper_threshold3 = thresholds[2]  
        selected_dfevents3 = list()
        selected_dfpulses3 = list()
        selected_etofs3 = list()
        selected_pnccds3 = list()
    
    dataframes = dict()
    
    if type(num_pulses) == int:
        num_pulses_run = int(num_pulses/len(runs))
    
    for run in runs:
        
        dfevent, dfpulse, etof, pnccd = read(run)
        
        selections = list()
        
        plt.figure()
        plt.scatter(dfpulse.pulseId,dfpulse.nevents_pulse,c='black',label='All pulses')
        
        if type(num_pulses) == int:
            dfpulse = dfpulse.sample(n=num_pulses_run)
            
        selected_dfpulse1 = dfpulse[lower_threshold1 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold1]
        selected_dfevent1 = dfevent[dfevent.pulseId.isin(selected_dfpulse1.pulseId)]
        selected_etof1 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse1.pulseId))
        selected_pnccd1 = pnccd.sel(trainId=pnccd.coords['trainId'].isin(selected_dfpulse1.trainId))
        selections.append((selected_dfevent1, selected_dfpulse1, selected_etof1, selected_pnccd1))
        plt.scatter(selected_dfpulse1.pulseId,selected_dfpulse1.nevents_pulse,c='r',label=f'Between {lower_threshold1} and {upper_threshold1}')
        
        if len(thresholds) > 1:
            
            selected_dfpulse2 = dfpulse[lower_threshold2 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold2]
            selected_dfevent2 = dfevent[dfevent.pulseId.isin(selected_dfpulse2.pulseId)]
            selected_etof2 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse2.pulseId))
            selected_pnccd2 = pnccd.sel(trainId=pnccd.coords['trainId'].isin(selected_dfpulse2.trainId))
            selections.append((selected_dfevent2, selected_dfpulse2, selected_etof2, selected_pnccd2))
            plt.scatter(selected_dfpulse2.pulseId,selected_dfpulse2.nevents_pulse,c='blue',label=f'Between {lower_threshold2} and {upper_threshold2}')
            
        if len(thresholds) > 2:
            
            selected_dfpulse3 = dfpulse[lower_threshold3 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold3]
            selected_dfevent3 = dfevent[dfevent.pulseId.isin(selected_dfpulse3.pulseId)]
            selected_etof3 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse3.pulseId))
            selected_pnccd3 = pnccd.sel(trainId=pnccd.coords['trainId'].isin(selected_dfpulse3.trainId))
            selections.append((selected_dfevent3, selected_dfpulse3, selected_etof3, selected_pnccd3))
            plt.scatter(selected_dfpulse3.pulseId,selected_dfpulse3.nevents_pulse,c='g',label=f'Between {lower_threshold3} and {upper_threshold3}')  
        
        dataframes[run] = selections
          
        plt.xlabel('Pulse ID')
        plt.ylabel('Number of events per pulse')
        plt.legend()
        plt.title(f'Events per pulse with respect to pulse ID for run {run}')
        plt.show()

        
    for key, values in dataframes.items():
        
        selected_dfevents1.append(values[0][0])
        selected_dfpulses1.append(values[0][1])
        selected_etofs1.append(values[0][2])
        selected_pnccds1.append(values[0][3])
        
        if len(thresholds) > 1:
            selected_dfevents2.append(values[1][0])
            selected_dfpulses2.append(values[1][1])
            selected_etofs2.append(values[1][2])
            selected_pnccds2.append(values[1][3])
            
        if len(thresholds) > 2: 
            selected_dfevents3.append(values[2][0])
            selected_dfpulses3.append(values[2][1])
            selected_etofs3.append(values[2][2])
            selected_pnccds3.append(values[2][3])
        
        
    merged_selection = list()
    
    merged_dfevent1 = pd.concat(selected_dfevents1)
    merged_dfevent1.reset_index(drop=True, inplace=True)
    
    merged_dfpulse1 = pd.concat(selected_dfpulses1)
    merged_dfpulse1.reset_index(drop=True, inplace=True)
    
    merged_etof1 = xr.concat(selected_etofs1, dim='pulseId')
    merged_pnccd1 = xr.concat(selected_pnccds1, dim='pulseId')
    
    merged_selection.append((merged_dfevent1, merged_dfpulse1, merged_etof1, merged_pnccd1))
    
    print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold1} and {upper_threshold1} events: {len(merged_dfpulse1)}")
       
        
    if len(thresholds) > 1:
        merged_dfevent2 = pd.concat(selected_dfevents2)
        merged_dfevent2.reset_index(drop=True, inplace=True)
    
        merged_dfpulse2 = pd.concat(selected_dfpulses2)
        merged_dfpulse2.reset_index(drop=True, inplace=True)
    
        merged_etof2 = xr.concat(selected_etofs2, dim='pulseId')
        merged_pnccd2 = xr.concat(selected_pnccds2, dim='pulseId')
        
        merged_selection.append((merged_dfevent2, merged_dfpulse2, merged_etof2, merged_pnccd2))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold2} and {upper_threshold2} events: {len(merged_dfpulse2)}")
    
    
    if len(thresholds) > 2:
        merged_dfevent3 = pd.concat(selected_dfevents3)
        merged_dfevent3.reset_index(drop=True, inplace=True)
    
        merged_dfpulse3 = pd.concat(selected_dfpulses3)
        merged_dfpulse3.reset_index(drop=True, inplace=True)
    
        merged_etof3 = xr.concat(selected_etofs3, dim='pulseId')
        merged_pnccd3 = xr.concat(selected_pnccds3, dim='pulseId')
        
        merged_selection.append((merged_dfevent3, merged_dfpulse3, merged_etof3, merged_pnccd3))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold3} and {upper_threshold3} events: {len(merged_dfpulse3)}")
        
    
    return merged_selection



def events_selection_ions_electrons(runs,thresholds,num_pulses=None,tof_limit=None):
    'Reads one or multiple runs from h5 files'
    'Makes a pulse selection based on the number of events per pulse between the defined thresholds'
    'If multiple runs are passed, will merge the runs, once hit selected'
    'Thresholds can be between one and three tuples (lower threshold, upper threshold)'
    'Downsamples by num_pulses'
    
    lower_threshold1, upper_threshold1 = thresholds[0]
    selected_dfevents1 = list()
    selected_dfpulses1 = list()
    selected_etofs1 = list()
    
    if len(thresholds) > 1:
        lower_threshold2, upper_threshold2 = thresholds[1]
        selected_dfevents2 = list()
        selected_dfpulses2 = list()
        selected_etofs2 = list()
    
    if len(thresholds) > 2:
        lower_threshold3, upper_threshold3 = thresholds[2]  
        selected_dfevents3 = list()
        selected_dfpulses3 = list()
        selected_etofs3 = list()
    
    dataframes = dict()
    
    if type(num_pulses) == int:
        num_pulses_run = int(num_pulses/len(runs))
    
    for run in runs:
        
        dfevent, dfpulse, etof = read_ions_electrons(run,tof_limit)
        
        selections = list()
        
        if type(num_pulses) == int:
            dfpulse = dfpulse.sample(n=num_pulses_run)
            
        selected_dfpulse1 = dfpulse[lower_threshold1 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold1]
        selected_dfevent1 = dfevent[dfevent.pulseId.isin(selected_dfpulse1.pulseId)]
        selected_etof1 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse1.pulseId))
        selections.append((selected_dfevent1, selected_dfpulse1, selected_etof1))
        plt.scatter(selected_dfpulse1.pulseId,selected_dfpulse1.nevents_pulse,c='r',label=f'Between {lower_threshold1} and {upper_threshold1}')
        
        if len(thresholds) > 1:
            
            selected_dfpulse2 = dfpulse[lower_threshold2 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold2]
            selected_dfevent2 = dfevent[dfevent.pulseId.isin(selected_dfpulse2.pulseId)]
            selected_etof2 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse2.pulseId))
            selections.append((selected_dfevent2, selected_dfpulse2, selected_etof2))
            plt.scatter(selected_dfpulse2.pulseId,selected_dfpulse2.nevents_pulse,c='blue',label=f'Between {lower_threshold2} and {upper_threshold2}')
            
        if len(thresholds) > 2:
            
            selected_dfpulse3 = dfpulse[lower_threshold3 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold3]
            selected_dfevent3 = dfevent[dfevent.pulseId.isin(selected_dfpulse3.pulseId)]
            selected_etof3 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse3.pulseId))
            selections.append((selected_dfevent3, selected_dfpulse3, selected_etof3))
            plt.scatter(selected_dfpulse3.pulseId,selected_dfpulse3.nevents_pulse,c='g',label=f'Between {lower_threshold3} and {upper_threshold3}')  
        
        dataframes[run] = selections
        
    for key, values in dataframes.items():
        
        selected_dfevents1.append(values[0][0])
        selected_dfpulses1.append(values[0][1])
        selected_etofs1.append(values[0][2])
        
        if len(thresholds) > 1:
            selected_dfevents2.append(values[1][0])
            selected_dfpulses2.append(values[1][1])
            selected_etofs2.append(values[1][2])
            
        if len(thresholds) > 2: 
            selected_dfevents3.append(values[2][0])
            selected_dfpulses3.append(values[2][1])
            selected_etofs3.append(values[2][2])
        
    merged_selection = list()
    
    merged_dfevent1 = pd.concat(selected_dfevents1)
    merged_dfevent1.reset_index(drop=True, inplace=True)
    
    merged_dfpulse1 = pd.concat(selected_dfpulses1)
    merged_dfpulse1.reset_index(drop=True, inplace=True)
    
    merged_etof1 = xr.concat(selected_etofs1, dim='pulseId')
    
    merged_selection.append((merged_dfevent1, merged_dfpulse1, merged_etof1))
    
    print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold1} and {upper_threshold1} events: {len(merged_dfpulse1)}")
           
    if len(thresholds) > 1:
        merged_dfevent2 = pd.concat(selected_dfevents2)
        merged_dfevent2.reset_index(drop=True, inplace=True)
    
        merged_dfpulse2 = pd.concat(selected_dfpulses2)
        merged_dfpulse2.reset_index(drop=True, inplace=True)
    
        merged_etof2 = xr.concat(selected_etofs2, dim='pulseId')
        
        merged_selection.append((merged_dfevent2, merged_dfpulse2, merged_etof2))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold2} and {upper_threshold2} events: {len(merged_dfpulse2)}") 
    
    if len(thresholds) > 2:
        merged_dfevent3 = pd.concat(selected_dfevents3)
        merged_dfevent3.reset_index(drop=True, inplace=True)
    
        merged_dfpulse3 = pd.concat(selected_dfpulses3)
        merged_dfpulse3.reset_index(drop=True, inplace=True)
    
        merged_etof3 = xr.concat(selected_etofs3, dim='pulseId')
        
        merged_selection.append((merged_dfevent3, merged_dfpulse3, merged_etof3))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold3} and {upper_threshold3} events: {len(merged_dfpulse3)}")
        
    return merged_selection



def events_selection_ions_photons(runs,thresholds,num_pulses=None):
    'Reads one or multiple runs from h5 files'
    'Makes a pulse selection based on the number of events per pulse between the defined thresholds'
    'If multiple runs are passed, will merge the runs, once hit selected'
    'Thresholds can be between one and three tuples (lower threshold, upper threshold)'
    'Downsamples by num_pulses'
    
    lower_threshold1, upper_threshold1 = thresholds[0]
    selected_dfevents1 = list()
    selected_dfpulses1 = list()
    selected_pnccds1 = list()
    
    if len(thresholds) > 1:
        lower_threshold2, upper_threshold2 = thresholds[1]
        selected_dfevents2 = list()
        selected_dfpulses2 = list()
        selected_pnccds2 = list()
    
    if len(thresholds) > 2:
        lower_threshold3, upper_threshold3 = thresholds[2]  
        selected_dfevents3 = list()
        selected_dfpulses3 = list()
        selected_pnccds3 = list()
    
    dataframes = dict()
    
    if type(num_pulses) == int:
        num_pulses_run = int(num_pulses/len(runs))
    
    for run in runs:
        
        print('Processing run number',run)
        
        dfevent, dfpulse, pnccd = read_ions_photons(run)
        
        selections = list()
        
        if type(num_pulses) == int:
            dfpulse = dfpulse.sample(n=num_pulses_run)
            
        selected_dfpulse1 = dfpulse[lower_threshold1 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold1]
        selected_dfevent1 = dfevent[dfevent.pulseId.isin(selected_dfpulse1.pulseId)]
        selected_pnccd1 = pnccd.sel(trainId=pnccd.coords['trainId'].isin(selected_dfpulse1.trainId))
        selections.append((selected_dfevent1, selected_dfpulse1, selected_pnccd1))
        
        if len(thresholds) > 1:
            
            selected_dfpulse2 = dfpulse[lower_threshold2 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold2]
            selected_dfevent2 = dfevent[dfevent.pulseId.isin(selected_dfpulse2.pulseId)]
            selected_pnccd2 = pnccd.sel(trainId=pnccd.coords['trainId'].isin(selected_dfpulse2.trainId))
            selections.append((selected_dfevent2, selected_dfpulse2, selected_pnccd2))
            
        if len(thresholds) > 2:
            
            selected_dfpulse3 = dfpulse[lower_threshold3 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold3]
            selected_dfevent3 = dfevent[dfevent.pulseId.isin(selected_dfpulse3.pulseId)]
            selected_pnccd3 = pnccd.sel(trainId=pnccd.coords['trainId'].isin(selected_dfpulse3.trainId))
            selections.append((selected_dfevent3, selected_dfpulse3, selected_pnccd3))
            
        dataframes[run] = selections
        
    for key, values in dataframes.items():
        
        selected_dfevents1.append(values[0][0])
        selected_dfpulses1.append(values[0][1])
        selected_pnccds1.append(values[0][2])
        
        if len(thresholds) > 1:
            selected_dfevents2.append(values[1][0])
            selected_dfpulses2.append(values[1][1])
            selected_pnccds2.append(values[1][2])
            
        if len(thresholds) > 2: 
            selected_dfevents3.append(values[2][0])
            selected_dfpulses3.append(values[2][1])
            selected_pnccds3.append(values[2][2])
        
        
    merged_selection = list()
    
    merged_dfevent1 = pd.concat(selected_dfevents1)
    merged_dfevent1.reset_index(drop=True, inplace=True)
    
    merged_dfpulse1 = pd.concat(selected_dfpulses1)
    merged_dfpulse1.reset_index(drop=True, inplace=True)
    
    merged_pnccd1 = xr.concat(selected_pnccds1, dim='trainId')
    
    merged_selection.append((merged_dfevent1, merged_dfpulse1, merged_pnccd1))
    
    print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold1} and {upper_threshold1} events: {len(merged_dfpulse1)}")
       
        
    if len(thresholds) > 1:
        merged_dfevent2 = pd.concat(selected_dfevents2)
        merged_dfevent2.reset_index(drop=True, inplace=True)
    
        merged_dfpulse2 = pd.concat(selected_dfpulses2)
        merged_dfpulse2.reset_index(drop=True, inplace=True)
    
        merged_pnccd2 = xr.concat(selected_pnccds2, dim='trainId')
        
        merged_selection.append((merged_dfevent2, merged_dfpulse2, merged_pnccd2))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold2} and {upper_threshold2} events: {len(merged_dfpulse2)}")
    
    
    if len(thresholds) > 2:
        merged_dfevent3 = pd.concat(selected_dfevents3)
        merged_dfevent3.reset_index(drop=True, inplace=True)
    
        merged_dfpulse3 = pd.concat(selected_dfpulses3)
        merged_dfpulse3.reset_index(drop=True, inplace=True)
    
        merged_pnccd3 = xr.concat(selected_pnccds3, dim='trainId')
        
        merged_selection.append((merged_dfevent3, merged_dfpulse3, merged_pnccd3))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold3} and {upper_threshold3} events: {len(merged_dfpulse3)}")

    
    return merged_selection



def ion_selection(runs,thresholds,num_pulses=None):
    'Only handles ion data'
    'Reads one or multiple runs from h5 files'
    'Makes a pulse selection based on the number of events per pulse between the defined thresholds'
    'If multiple runs are passed, will merge the runs, once hit selected'
    'Thresholds can be between one and three tuples (lower threshold, upper threshold)'
    'Downsamples by num_pulses'
    
    lower_threshold1, upper_threshold1 = thresholds[0]
    selected_dfevents1 = list()
    selected_dfpulses1 = list()
    
    if len(thresholds) > 1:
        lower_threshold2, upper_threshold2 = thresholds[1]
        selected_dfevents2 = list()
        selected_dfpulses2 = list()
    
    if len(thresholds) > 2:
        lower_threshold3, upper_threshold3 = thresholds[2]  
        selected_dfevents3 = list()
        selected_dfpulses3 = list()
    
    dataframes = dict()
    
    if type(num_pulses) == int:
        num_pulses_run = int(num_pulses/len(runs))
    
    for run in runs:
        
        print('Handling run', run)
        dfevent, dfpulse = read_ion(run)
        
        selections = list()
        # plt.figure()
        # plt.scatter(dfpulse.pulseId,dfpulse.nevents_pulse,c='black',label='All pulses')
        
        if type(num_pulses) == int:
            dfpulse = dfpulse.sample(n=num_pulses_run)
            
        selected_dfpulse1 = dfpulse[lower_threshold1 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold1]
        selected_dfevent1 = dfevent[dfevent.pulseId.isin(selected_dfpulse1.pulseId)]
        selections.append((selected_dfevent1, selected_dfpulse1))
        
        if len(thresholds) > 1:
            
            selected_dfpulse2 = dfpulse[lower_threshold2 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold2]
            selected_dfevent2 = dfevent[dfevent.pulseId.isin(selected_dfpulse2.pulseId)]
            selections.append((selected_dfevent2, selected_dfpulse2))
            
        if len(thresholds) > 2:
            
            selected_dfpulse3 = dfpulse[lower_threshold3 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold3]
            selected_dfevent3 = dfevent[dfevent.pulseId.isin(selected_dfpulse3.pulseId)]
            selections.append((selected_dfevent3, selected_dfpulse3))
        
        dataframes[run] = selections
          
        # plt.xlabel('Pulse ID')
        # plt.ylabel('Number of events per pulse')
        # plt.legend()
        # plt.title(f'Events per pulse with respect to pulse ID for run {run}')
        # plt.show()

        
    for key, values in dataframes.items():
        
        selected_dfevents1.append(values[0][0])
        selected_dfpulses1.append(values[0][1])
        
        if len(thresholds) > 1:
            selected_dfevents2.append(values[1][0])
            selected_dfpulses2.append(values[1][1])
            
        if len(thresholds) > 2: 
            selected_dfevents3.append(values[2][0])
            selected_dfpulses3.append(values[2][1])
        
        
    merged_selection = list()
    
    merged_dfevent1 = pd.concat(selected_dfevents1)
    merged_dfevent1.reset_index(drop=True, inplace=True)
    
    merged_dfpulse1 = pd.concat(selected_dfpulses1)
    merged_dfpulse1.reset_index(drop=True, inplace=True)
    
    merged_selection.append((merged_dfevent1, merged_dfpulse1))
    
    print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold1} and {upper_threshold1} events: {len(merged_dfpulse1)}")
       
        
    if len(thresholds) > 1:
        merged_dfevent2 = pd.concat(selected_dfevents2)
        merged_dfevent2.reset_index(drop=True, inplace=True)
    
        merged_dfpulse2 = pd.concat(selected_dfpulses2)
        merged_dfpulse2.reset_index(drop=True, inplace=True)
        
        merged_selection.append((merged_dfevent2, merged_dfpulse2))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold2} and {upper_threshold2} events: {len(merged_dfpulse2)}")
    
    
    if len(thresholds) > 2:
        merged_dfevent3 = pd.concat(selected_dfevents3)
        merged_dfevent3.reset_index(drop=True, inplace=True)
    
        merged_dfpulse3 = pd.concat(selected_dfpulses3)
        merged_dfpulse3.reset_index(drop=True, inplace=True)
        
        merged_selection.append((merged_dfevent3, merged_dfpulse3))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold3} and {upper_threshold3} events: {len(merged_dfpulse3)}")
        
    
    return merged_selection



def heatmap(dfevent):
    'Creates heatmap of the ions hits, based on a dfevent dataframe'
    
    counts_df = dfevent.groupby(['x', 'y']).size().reset_index(name='count')
    heatmap_data = counts_df.pivot(index='y', columns='x', values='count')
    
    plt.figure()
    ax = sns.heatmap(heatmap_data, cmap='viridis',cbar_kws={'label': 'Number of events'})
    plt.title('Ion heatmap')
    plt.show()
    

    
def ion_tof(dfevent,xlimits=(0,TIME_BETWEEN_PULSES),nbins=1000):
    'Plots ion time of flight data using dfevent dataframe'
    
    bounded_dfevent = dfevent[dfevent.tof > xlimits[0]][dfevent.tof < xlimits[1]]
    hist, bin_edges = np.histogram(bounded_dfevent.tof, bins=nbins)
    
    plt.figure()
    plt.plot(bin_edges[:-1], hist)
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of hits per bin')
    plt.title('Ions time of flight')
    # plt.xlim(xlimits[0],xlimits[1])
    plt.show()   
    
    
    
def e_tof(etof):
    'Plots electron time of flight data using etof xarray data'
    
    channel_time = TIME_BETWEEN_PULSES/CHANNELS_PER_PULSE
    
    xaxis = np.arange(14080)*channel_time
    avg_selected_etof = -np.mean(etof, axis=0)
    
    plt.figure()
    plt.plot(xaxis,avg_selected_etof/max(avg_selected_etof))
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Normalized signal')
    plt.title('Electrons time of flight')
    plt.show()



def events_selection_plots(runs,thresholds,downsampling=None):
    'Runs functions events_selection, heatmap, e_tof, ion_tof'
    'Downsamples by downsampling integer value if one is given'
    
    selections = events_selection(runs,thresholds,downsampling)
    
    print(f'\n Plots for selection between {thresholds[0][0]} and {thresholds[0][1]} events:')
    selected_dfevent1, selected_dfpulse1, selected_etof1, selected_pnccd1 = selections[0]
    heatmap(selected_dfevent1)
    ion_tof(selected_dfevent1)
    e_tof(selected_etof1)
    
    if len(selections) > 1:
        print(f'Plots for selection between {thresholds[1][0]} and {thresholds[1][1]} events:')
        selected_dfevent2, selected_dfpulse2, selected_etof2, selected_pnccd2 = selections[1]
        heatmap(selected_dfevent2)
        ion_tof(selected_dfevent2)
        e_tof(selected_etof2)
    
    elif len(selections) > 2:
        print(f'Plots for selection between {thresholds[2][0]} and {thresholds[2][1]} events:')
        selected_dfevent3, selected_dfpulse3, selected_etof3, selected_pnccd3 = selections[2]
        heatmap(selected_dfevent3)
        ion_tof(selected_dfevent3)
        e_tof(selected_etof3)
    
    return selections



def pulse_filtered_ion_tof(dfevent,pulse_number=None,xlim=TIME_BETWEEN_PULSES):
    'Plots ion time of flight data using dfevent dataframe'
    
    if pulse_number != None:
        pulse_filtered_dfevent = dfevent[dfevent.pulseId.astype(str).str.endswith(pulse_number)]
    else:
        pulse_filtered_dfevent = dfevent
    
    bounded_dfevent = pulse_filtered_dfevent[pulse_filtered_dfevent.tof < TIME_BETWEEN_PULSES]
    hist, bin_edges = np.histogram(bounded_dfevent.tof, bins=1000)
    
    plt.figure()
    plt.plot(bin_edges[:-1], hist/max(hist))
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Normalized signal')
    plt.title('Ions time of flight')
    plt.xlim(0,xlim)
    plt.yscale('log')
    plt.show()
    
    plt.figure()
    plt.plot(bin_edges[:-1], hist/max(hist))
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Normalized signal')
    plt.title('Ions time of flight')
    plt.xlim(0,TIME_BETWEEN_PULSES)
    plt.yscale('log')
    plt.show()  
    
    
    
def heatmap_with_zones(dfevent,zones):
    'Creates heatmap of the ions hits, based on a dfevent dataframe'
    'Draws a rectangle around zones defined by a list of tuples where each tuple represents a tilted zone (xstart, ystart, width, height, angle in degrees)'
    
    counts_df = dfevent.groupby(['x', 'y']).size().reset_index(name='count')
    heatmap_data = counts_df.pivot(index='y', columns='x', values='count')
    
    plt.figure()
    ax = sns.heatmap(heatmap_data, cmap='viridis', cbar_kws={'label': 'Number of events'})
    
    xlim = int(ax.get_xticklabels()[0].get_text())
    ylim = int(ax.get_yticklabels()[0].get_text())

    for zone in zones:
        x, y, width, height, angle = zone
        x_adjusted = x - xlim
        y_adjusted = y - ylim
        
        rect = plt.Rectangle((x_adjusted, y_adjusted), width, height, fill=False, edgecolor='red', lw=1, angle=angle)
        ax.add_patch(rect)
    
    plt.title('Ion heatmap')
    plt.show()
    
    
        
def spatial_selection(dfevent,dfpulse,etof,zones):
    'Square selection from the heatmap using spatial coordinates'
    'Zone is a tuple representing a zone that is not tilted (xstart, ystart, width, height)'
    'Returns spatially selected dfevent,dfpulse,etof'
    
    heatmap_with_zones(dfevent,zones)
    
    selected_dfevents = []
    
    for zone in zones:
        
        xstart,ystart,width,height,angle = zone
    
        spatial_selected_dfevent = dfevent[dfevent.x > xstart][dfevent.x < xstart+width][dfevent.y > ystart][dfevent.y < ystart+height]
        selected_dfevents.append(spatial_selected_dfevent)
        
    merged_dfevent = pd.concat(selected_dfevents)
    merged_dfevent.reset_index(drop=True, inplace=True)
    
    spatial_selected_dfpulse = dfpulse[dfpulse.pulseId.isin(merged_dfevent.pulseId)]
    spatial_selected_etof = etof.sel(pulseId=etof.coords['pulseId'].isin(merged_dfevent.pulseId))
    
    return merged_dfevent,spatial_selected_dfpulse,spatial_selected_etof



def spatial_ion_selection(dfevent,dfpulse,zones):
    'Square ion selection from the heatmap using spatial coordinates'
    'Zone is a tuple representing a zone that is not tilted (xstart, ystart, width, height)'
    'Returns spatially selected dfevent,dfpulse'
    
    heatmap_with_zones(dfevent,zones)
    
    selected_dfevents = []
    
    for zone in zones:
        
        xstart,ystart,width,height,angle = zone
    
        spatial_selected_dfevent = dfevent[dfevent.x > xstart][dfevent.x < xstart+width][dfevent.y > ystart][dfevent.y < ystart+height]
        selected_dfevents.append(spatial_selected_dfevent)
        
    merged_dfevent = pd.concat(selected_dfevents)
    merged_dfevent.reset_index(drop=True, inplace=True)
    
    spatial_selected_dfpulse = dfpulse[dfpulse.pulseId.isin(merged_dfevent.pulseId)]
    
    return merged_dfevent,spatial_selected_dfpulse



def big_ion_tof(dfevent):
    'Plots widget of big ion time of flight data using dfevent dataframe'
    
    bounded_dfevent = dfevent[dfevent.tof < TIME_BETWEEN_PULSES]
    hist, bin_edges = np.histogram(bounded_dfevent.tof, bins=1000)
    
    plt.figure(figsize=(18, 8))
    plt.plot(bin_edges[:-1], hist, c='g')
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of hits per bin')
    plt.title('Ions time of flight')
    plt.show()
    
 
    
def autoscale_y(ax,margin=0.1):
    """This function rescales the y-axis based on the data that is visible given the current xlim of the axis.
    ax -- a matplotlib axes object
    margin -- the fraction of the total height of the y-data to pad the upper and lower ylims"""

    import numpy as np

    def get_bottom_top(line):
        xd = line.get_xdata()
        yd = line.get_ydata()
        lo,hi = ax.get_xlim()
        y_displayed = yd[((xd>lo) & (xd<hi))]
        h = np.max(y_displayed) - np.min(y_displayed)
        bot = np.min(y_displayed)-margin*h
        top = np.max(y_displayed)+margin*h
        return bot,top

    lines = ax.get_lines()
    bot,top = np.inf, -np.inf

    for line in lines:
        new_bot, new_top = get_bottom_top(line)
        if new_bot < bot: bot = new_bot
        if new_top > top: top = new_top

    ax.set_ylim(bot,top)
    
    
    
def zoomed_ion_tof(dfevent,anchor):
    'Plots zoom around anchor point of ion time of flight data using dfevent dataframe'
    
    bounded_dfevent = dfevent[dfevent.tof < TIME_BETWEEN_PULSES]
    hist, bin_edges = np.histogram(bounded_dfevent.tof, bins=1000)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(bin_edges[:-1], hist, c='g')
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of hits per bin')
    plt.title('Ions time of flight')
    plt.xlim(anchor-1e-7,anchor+1e-7)
    autoscale_y(ax)
    ax.axvline(x=anchor, color='black', linestyle='--')
    plt.show()
    
    
    
def power_law(x, a, b):
    'Calibration fit power law'
    return a * x**b



def compute_calibration(calibration_lines):

    # Corresponding m/q argon values
    mq_lines = [40,20,40/3,40/4,40/5]
    
    # Initial guesses for parameters a and b
    initial_guess = [1.6e13, 2]#[1.9e20, 3]
    
    # Perform the curve fitting
    params, covariance = curve_fit(power_law, calibration_lines, mq_lines, p0=initial_guess, maxfev=10000)

    # Extract the fitted values for a and b
    a_fit, b_fit = params

    print(f"The fit looks as follows: m/q = {a_fit:.2e} * tof^{b_fit:.2f}")
    
    return a_fit, b_fit



def calibrate(backgrd_dfevent):
    'Computes calibration by least mean squares using backgrd_dfevent'
    'Uses user input to compute fit based on displayed plots'
    
    # Show a large widget ion tof
    big_ion_tof(backgrd_dfevent)

    done = False
    while not done:

        # Ask for five numbers input
        anchors = []
        for i in range(5):
            value = input(f"Enter value Ar{i + 1}: ")
            try:
                anchors.append(float(value))
            except ValueError:
                print("Invalid input. Please enter a number.")

        # Show five additional plots based on inputs
        %matplotlib inline
        for anchor in anchors:
            zoomed_ion_tof(backgrd_dfevent,anchor)

        # Ask if the user is done
        done_response = input("Are you done? (y/n): ").strip().lower()
        if done_response == 'y':
            done = True
    
    # Compute calibration fit
    a_fit, b_fit = compute_calibration(anchors)
    
    return a_fit, b_fit



def apply_calibration(dfevents,a_fit,b_fit):
    'Applies calibration to each dfevent of the list of dfevents and outputs calibrated_dfevents list of dataframes with m/q column'
        
    calibrated_dfevents = list()
        
    for dfevent in dfevents:
        dfevent['mq'] = a_fit * dfevent.tof ** b_fit
        calibrated_dfevents.append(dfevent)
        
    return calibrated_dfevents



def mq_selection(calibrated_dfevent,dfpulse,etof,lower_mq,upper_mq):
    'Selects based on m/q values. Need to input calibrated_dfevent! Returns m/q selected dfevent,dfpulse,etof.'
    
    mqselected_dfevent = calibrated_dfevent[lower_mq < calibrated_dfevent.mq][calibrated_dfevent.mq < upper_mq]
    mqselected_dfpulse = dfpulse[dfpulse.pulseId.isin(mqselected_dfevent.pulseId)]
    mqselected_etof = etof.sel(pulseId=etof.coords['pulseId'].isin(mqselected_dfevent.pulseId))
    
    return mqselected_dfevent,mqselected_dfpulse,mqselected_etof



def find_rectangle_corners(zone):
    
    x, y, width, height, angle_degrees = zone
    
    angle_radians = np.deg2rad(angle_degrees)
    c, s = np.cos(angle_radians), np.sin(angle_radians)
    
    # Calculate the coordinates of the other three corners
    corners = np.array([
        [x, y],
        [x + width * c, y + width * s],
        [x + width * c - height * s, y + width * s + height * c],
        [x - height * s, y + height * c]
    ])
    
    return corners



def find_integer_coordinates(corners, zone):
    
    min_x, min_y = np.floor(np.min(corners, axis=0))
    max_x, max_y = np.ceil(np.max(corners, axis=0))

    integer_coordinates = []
    for x in range(int(min_x), int(max_x) + 1):
        for y in range(int(min_y), int(max_y) + 1):
            if is_inside(x, y, corners, zone):
                integer_coordinates.append((x, y))
    
    return integer_coordinates



def is_inside(x, y, corners, zone):
    
    ox, oy, width, height, angle_degrees = zone
    
    angle_radians = np.deg2rad(angle_degrees)
    c, s = np.cos(angle_radians), np.sin(angle_radians)
    
    rotated_x = (x-ox)*c + (y-oy)*s + ox
    rotated_y = -(x-ox)*s + (y-oy)*c + oy
    
    return ox <= rotated_x <= ox + width and oy <= rotated_y <= oy + height



def tilted_spatial_ion_selection(dfevent,dfpulse,etof,zones):
    'Selection from the heatmap using spatial coordinates'
    'Zones is a list of tuple representing zones - tilted or not - (xstart, ystart, width, height, angle in degrees)'
    'Returns spatially selected dfevent,dfpulse,etof'
    
    heatmap_with_zones(dfevent,zones)
    
    integer_coordinates = []
    for zone in zones:
        corners = find_rectangle_corners(zone)
        integer_coordinates.extend(find_integer_coordinates(corners, zone))
    
    x_coords, y_coords = zip(*integer_coordinates)
    
    spatial_selected_dfevent = dfevent[dfevent.x.isin(x_coords)][dfevent.y.isin(y_coords)]
    spatial_selected_dfpulse = dfpulse[dfpulse.pulseId.isin(spatial_selected_dfevent.pulseId)]
    spatial_selected_etof = etof.sel(pulseId=etof.coords['pulseId'].isin(spatial_selected_dfevent.pulseId))
    
    return spatial_selected_dfevent,spatial_selected_dfpulse,spatial_selected_etof



def fish_plot_ion_selection(dfevent,zone):
    'Selection from the heatmap using spatial coordinates'
    'Zone is a tuple representing a zone - tilted or not - (xstart, ystart, width, height, angle in degrees)'
    'Returns spatially selected dfevent'
    
    heatmap_with_zones(dfevent,[zone])
    
    corners = find_rectangle_corners(zone)
    integer_coordinates = find_integer_coordinates(corners, zone)
    
    x_coords, y_coords = zip(*integer_coordinates)
    
    spatial_selected_dfevent = dfevent[dfevent.x.isin(x_coords)][dfevent.y.isin(y_coords)]
    
    return spatial_selected_dfevent
    


def fish_plot_x(dfevent,zone,tof_bins=1000,mq_bins=500):
    'Produces fish plots along x with respect to time of flight and m/q from dfevent dataframe and a zone defined as (startx, starty, width, height, angle in degrees)'
    
    fish_dfevent = fish_plot_ion_selection(dfevent,zone)
    
    tof_bin_edges = np.linspace(0,TIME_BETWEEN_PULSES,tof_bins+1)
    fish_dfevent['tof_binned'] = pd.cut(fish_dfevent['tof'], bins=tof_bin_edges, labels=tof_bin_edges[:-1].astype('str'))
    tof_pivot_table = fish_dfevent.pivot_table(index='x', columns='tof_binned', aggfunc='size', fill_value=0)
    max_tof_value = tof_pivot_table.values.max()
    normalized_tof = tof_pivot_table.values / max_tof_value
    
    mq_bin_edges = np.linspace(0,200,mq_bins+1)
    fish_dfevent['mq_binned'] = pd.cut(fish_dfevent['mq'], bins=mq_bin_edges, labels=mq_bin_edges[:-1].astype('str'))
    mq_pivot_table = fish_dfevent.pivot_table(index='x', columns='mq_binned', aggfunc='size', fill_value=0)
    max_mq_value = mq_pivot_table.values.max()
    normalized_mq = mq_pivot_table.values / max_mq_value
    
    
    fig, axes = plt.subplots(2, 1, figsize=(30, 12))
    
    cax_tof = axes[0].imshow(normalized_tof, cmap='viridis', aspect='auto', norm=LogNorm(), extent=[tof_bin_edges.min(), tof_bin_edges.max(), 0, 1])
    axes[0].set_xlabel('Time of flight (s)')
    axes[0].set_xlim(0,5e-7)
    axes[0].set_ylabel('x')
    axes[0].set_yticklabels(np.linspace(256,0,6,dtype=int))
    axes[0].set_title('Fish plot along x with respect to time of flight')
    cbar = plt.colorbar(cax_tof, ax=axes[0], label='Normalized number of events', norm=LogNorm())
    
    cax_mq = axes[1].imshow(normalized_mq, cmap='viridis', aspect='auto', norm=LogNorm(), extent=[mq_bin_edges.min(), mq_bin_edges.max(), 0, 1])
    axes[1].set_xlabel('m/q')
    axes[1].set_ylabel('x')
    axes[1].set_yticklabels(np.linspace(256,0,6,dtype=int))
    axes[1].set_title('Fish plot along x with respect to m/q')
    cbar = plt.colorbar(cax_mq, ax=axes[1], label='Normalized number of events', norm=LogNorm())
    
    plt.show()

    
    
def fish_plot_y(dfevent,zone,tof_bins=1000,mq_bins=500):
    'Produces a fish plot along y from dfevent dataframe and a zone defined as (startx, starty, width, height, angle in degrees)'
    
    fish_dfevent = fish_plot_ion_selection(dfevent,zone)
    
    tof_bin_edges = np.linspace(0,TIME_BETWEEN_PULSES,tof_bins+1)
    fish_dfevent['tof_binned'] = pd.cut(fish_dfevent['tof'], bins=tof_bin_edges, labels=tof_bin_edges[:-1].astype('str'))
    tof_pivot_table = fish_dfevent.pivot_table(index='y', columns='tof_binned', aggfunc='size', fill_value=0)
    max_tof_value = tof_pivot_table.values.max()
    normalized_tof = tof_pivot_table.values / max_tof_value
    
    mq_bin_edges = np.linspace(0,200,mq_bins+1)
    fish_dfevent['mq_binned'] = pd.cut(fish_dfevent['mq'], bins=mq_bin_edges, labels=mq_bin_edges[:-1].astype('str'))
    mq_pivot_table = fish_dfevent.pivot_table(index='y', columns='mq_binned', aggfunc='size', fill_value=0)
    max_mq_value = mq_pivot_table.values.max()
    normalized_mq = mq_pivot_table.values / max_mq_value
    
    fig, axes = plt.subplots(2, 1, figsize=(30, 12))
    
    cax_tof = axes[0].imshow(normalized_tof, cmap='viridis', aspect='auto', norm=LogNorm(), extent=[tof_bin_edges.min(), tof_bin_edges.max(), 0, 1])
    axes[0].set_xlabel('Time of flight (s)')
    axes[0].set_ylabel('y')
    axes[0].set_yticklabels(np.linspace(256,0,6,dtype=int))
    axes[0].set_title('Fish plot along y with respect to time of flight')
    cbar = plt.colorbar(cax_tof, ax=axes[0], label='Normalized number of events', norm=LogNorm())
    
    cax_mq = axes[1].imshow(normalized_mq, cmap='viridis', aspect='auto', norm=LogNorm(), extent=[mq_bin_edges.min(), mq_bin_edges.max(), 0, 1])
    axes[1].set_xlabel('m/q')
    axes[1].set_ylabel('y')
    axes[1].set_yticklabels(np.linspace(256,0,6,dtype=int))
    axes[1].set_title('Fish plot along y with respect to m/q')
    cbar = plt.colorbar(cax_mq, ax=axes[1], label='Normalized number of events', norm=LogNorm())

    plt.show()
    
    
def big_mq_plot(dfevent,nbins_mq=1500,xlimits=(0,200)):
    'Plots big intensity vs m/q plot using dfevent'
    
    x_lower, x_upper = xlimits
    
    plt.figure(figsize=(18,8))
        
    hist, bin_edges = np.histogram(dfevent.mq, bins=np.linspace(0,200,nbins_mq+1))
    plt.plot(bin_edges[:-1], hist/max(hist))
    
    plt.xlabel('m/q')
    plt.ylabel('Relatively normalized number of hits per bin')
    plt.title('Normalized ions time of flight')
    plt.xlim(x_lower, x_upper)
    plt.show()

In [ ]:
def nevents_binning(dfevent,dfpulse,etof,nbins_events,nbins_mq):
    'Binning dfevent dataframe into a number of bins nbins using number of events per pulse dfpulse.nevents_pulse'
    'Outputs nbins sized list of tuples (filtered_dfevent, filtered_dfpulse) and nbins sized array of histograms'
    'hists is divided by number of pulses in a certain number of events bin'
    
    nevents_min = min(dfpulse.nevents_pulse)
    nevents_max = max(dfpulse.nevents_pulse)
    
    bins = np.linspace(nevents_min,nevents_max,nbins_events+1).astype(int)
    
    filtered_dfevents = []
    filtered_dfpulses = []
    filtered_etofs = []
    hists = []
    
    for i in range(len(bins) - 1):

        start_edge = bins[i]
        end_edge = bins[i + 1]

        filtered_dfpulse = dfpulse[(dfpulse.nevents_pulse >= start_edge) & (dfpulse.nevents_pulse < end_edge)]
        filtered_dfevent = dfevent[dfevent.pulseId.isin(filtered_dfpulse.pulseId)]
        filtered_etof = etof.sel(pulseId=etof.coords['pulseId'].isin(filtered_dfpulse.pulseId))

        filtered_dfevents.append(filtered_dfevent)
        filtered_dfpulses.append(filtered_dfpulse)
        filtered_etofs.append(filtered_etof)
        
        hist, bin_edges = np.histogram(filtered_dfevent.mq, bins=np.linspace(0,200,nbins_mq+1),range=(0,200))
        hists.append(hist/len(filtered_dfpulse))
        
    return filtered_dfevents, filtered_dfpulses, filtered_etofs, np.array(hists), bins



def nevents_binning_tof(dfevent,dfpulse,etof,nbins_events,nbins_tof):
    'Bins into time of flight bins'
    'Binning dfevent dataframe into a number of bins nbins using number of events per pulse dfpulse.nevents_pulse'
    'Outputs nbins sized list of tuples (filtered_dfevent, filtered_dfpulse) and nbins sized array of histograms'
    'hists is divided by number of pulses in a certain number of events bin'
    
    nevents_min = min(dfpulse.nevents_pulse)
    nevents_max = max(dfpulse.nevents_pulse)
    
    bins = np.linspace(nevents_min,nevents_max,nbins_events+1).astype(int)
    
    filtered_dfevents = []
    filtered_dfpulses = []
    filtered_etofs = []
    hists = []
    
    for i in range(len(bins) - 1):

        start_edge = bins[i]
        end_edge = bins[i + 1]

        filtered_dfpulse = dfpulse[(dfpulse.nevents_pulse >= start_edge) & (dfpulse.nevents_pulse < end_edge)]
        filtered_dfevent = dfevent[dfevent.pulseId.isin(filtered_dfpulse.pulseId)]
        filtered_etof = etof.sel(pulseId=etof.coords['pulseId'].isin(filtered_dfpulse.pulseId))

        filtered_dfevents.append(filtered_dfevent)
        filtered_dfpulses.append(filtered_dfpulse)
        filtered_etofs.append(filtered_etof)
        
        hist, bin_edges = np.histogram(filtered_dfevent.tof, bins=np.linspace(0,TIME_BETWEEN_PULSES,nbins_tof+1),range=(0,TIME_BETWEEN_PULSES))
        hists.append(hist/len(filtered_dfpulse))
        
    return filtered_dfevents, filtered_dfpulses, filtered_etofs, np.array(hists), bins



def nions_binning(dfevent,dfpulse,nbins_events,nbins_mq):
    'Only handles ions'
    'Binning dfevent dataframe into a number of bins nbins using number of events per pulse dfpulse.nevents_pulse'
    'Outputs nbins sized list of tuples (filtered_dfevent, filtered_dfpulse) and nbins sized array of histograms'
    'hists is divided by number of pulses in a certain number of events bin'
    
    nevents_min = min(dfpulse.nevents_pulse)
    nevents_max = max(dfpulse.nevents_pulse)
    
    bins = np.linspace(nevents_min,nevents_max,nbins_events+1).astype(int)
    
    filtered_dfevents = []
    filtered_dfpulses = []
    hists = []
    
    for i in range(len(bins) - 1):

        start_edge = bins[i]
        end_edge = bins[i + 1]

        filtered_dfpulse = dfpulse[(dfpulse.nevents_pulse >= start_edge) & (dfpulse.nevents_pulse < end_edge)]
        filtered_dfevent = dfevent[dfevent.pulseId.isin(filtered_dfpulse.pulseId)]

        filtered_dfevents.append(filtered_dfevent)
        filtered_dfpulses.append(filtered_dfpulse)
        
        hist, bin_edges = np.histogram(filtered_dfevent.mq, bins=np.linspace(0,200,nbins_mq+1),range=(0,200))
        hists.append(hist/len(filtered_dfpulse))
        
    return filtered_dfevents, filtered_dfpulses, np.array(hists), bins



def nions_binning_tof(dfevent,dfpulse,nbins_events,nbins_tof):
    'Bins into time of flight bins'
    'Only handles ions'
    'Binning dfevent dataframe into a number of bins nbins using number of events per pulse dfpulse.nevents_pulse'
    'Outputs nbins sized list of tuples (filtered_dfevent, filtered_dfpulse) and nbins sized array of histograms'
    'hists is divided by number of pulses in a certain number of events bin'
    
    nevents_min = min(dfpulse.nevents_pulse)
    nevents_max = max(dfpulse.nevents_pulse)
    
    bins = np.linspace(nevents_min,nevents_max,nbins_events+1).astype(int)
    
    filtered_dfevents = []
    filtered_dfpulses = []
    hists = []
    
    for i in range(len(bins) - 1):

        start_edge = bins[i]
        end_edge = bins[i + 1]

        filtered_dfpulse = dfpulse[(dfpulse.nevents_pulse >= start_edge) & (dfpulse.nevents_pulse < end_edge)]
        filtered_dfevent = dfevent[dfevent.pulseId.isin(filtered_dfpulse.pulseId)]

        filtered_dfevents.append(filtered_dfevent)
        filtered_dfpulses.append(filtered_dfpulse)
        
        hist, bin_edges = np.histogram(filtered_dfevent.tof, bins=np.linspace(0,TIME_BETWEEN_PULSES,nbins_tof+1),range=(0,TIME_BETWEEN_PULSES))
        hists.append(hist/len(filtered_dfpulse))
        
    return filtered_dfevents, filtered_dfpulses, np.array(hists), bins



def nevents_binning_plot(dfevent,dfpulse,nbins_events,nbins_mq,xlimits=(0,200)):
    'Bins by m/q'
    'Binning dfevent dataframe into a number of bins nbins using number of events per pulse dfpulse.nevents_pulse'
    'Outputs nbins sized list of tuples (filtered_dfevent, filtered_dfpulse)'
    
    x_lower, x_upper = xlimits
    
    nevents_min = min(dfpulse.nevents_pulse)
    nevents_max = max(dfpulse.nevents_pulse)
    bins = np.linspace(nevents_min,nevents_max,nbins_events+1).astype(int)
    
    filtered_dfs = []
    plt.figure(figsize=(20,8))
    
    for i in range(len(bins) - 1):

        start_edge = bins[i]
        end_edge = bins[i + 1]

        filtered_dfpulse = dfpulse[(dfpulse.nevents_pulse >= start_edge) & (dfpulse.nevents_pulse < end_edge)]
        filtered_dfevent = dfevent[dfevent.pulseId.isin(filtered_dfpulse.pulseId)]

        filtered_dfs.append((filtered_dfevent,filtered_dfpulse))
        
        hist, bin_edges = np.histogram(filtered_dfevent.mq, bins=np.linspace(0,200,nbins_mq),range=(0,200))
        
        plt.plot(bin_edges[:-1], hist/max(hist), label=f'{start_edge}-{end_edge}')
        
    plt.xlabel('m/q')
    plt.ylabel('Relatively normalized number of hits per bin')
    plt.title('Normalized ions time of flight')
    plt.legend()
    plt.xlim(x_lower, x_upper)
    plt.show()   
        
    return filtered_dfs

    

def nevents_binning_plot_tof(dfevent,dfpulse,nbins_events,nbins_tof,xlimits=(0,TIME_BETWEEN_PULSES)):
    'Bins by time of flight'
    'Binning dfevent dataframe into a number of bins nbins using number of events per pulse dfpulse.nevents_pulse'
    'Outputs nbins sized list of tuples (filtered_dfevent, filtered_dfpulse)'
    
    x_lower, x_upper = xlimits
    
    nevents_min = min(dfpulse.nevents_pulse)
    nevents_max = max(dfpulse.nevents_pulse)
    bins = np.linspace(nevents_min,nevents_max,nbins_events+1).astype(int)
    
    filtered_dfs = []
    plt.figure(figsize=(20,8))
    
    for i in range(len(bins) - 1):

        start_edge = bins[i]
        end_edge = bins[i + 1]

        filtered_dfpulse = dfpulse[(dfpulse.nevents_pulse >= start_edge) & (dfpulse.nevents_pulse < end_edge)]
        filtered_dfevent = dfevent[dfevent.pulseId.isin(filtered_dfpulse.pulseId)]

        filtered_dfs.append((filtered_dfevent,filtered_dfpulse))
        
        hist, bin_edges = np.histogram(filtered_dfevent.tof, bins=np.linspace(0,TIME_BETWEEN_PULSES,nbins_tof),range=(0,TIME_BETWEEN_PULSES))
        
        plt.plot(bin_edges[:-1], hist/max(hist), label=f'{start_edge}-{end_edge}')
        
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Relatively normalized number of hits per bin')
    plt.title('Normalized ions time of flight')
    plt.legend()
    plt.xlim(x_lower, x_upper)
    plt.show()   
        
    return filtered_dfs



def nevents_binning_cov(dfevent,dfpulse,etof,nbins_events,nbins_ion_tof,nbins_e_tof,max_ion_limit=TIME_BETWEEN_PULSES,max_e_limit=TIME_BETWEEN_PULSES):
    'Binning dfevent and etof by numbers of events, and number of bins along time of flight'
    'Can select maximal time limit for electrons and ions'
    
    channel_time = TIME_BETWEEN_PULSES/CHANNELS_PER_PULSE
    
    nevents_min = min(dfpulse.nevents_pulse)
    nevents_max = max(dfpulse.nevents_pulse)
    
    bins = np.linspace(nevents_min,nevents_max,nbins_events+1).astype(int)
    
    max_int_ion_limit = int(max_ion_limit/channel_time)
    new_int_ion_limit = max_int_ion_limit - max_int_ion_limit % nbins_ion_tof
    new_ion_limit = new_int_ion_limit*channel_time
    
    max_int_e_limit = int(max_e_limit/channel_time)
    new_int_e_limit = max_int_e_limit - max_int_e_limit % nbins_e_tof
    e_group_size = int(new_int_e_limit/nbins_e_tof)
    
    shortened_dfevent = dfevent[dfevent.tof < new_ion_limit]
    shortened_etof = etof[:,:new_int_e_limit]
    
    hists = []
    hists_etof = []
    
    for i in range(len(bins) - 1):

        start_edge = bins[i]
        end_edge = bins[i + 1]

        filtered_dfpulse = dfpulse[(dfpulse.nevents_pulse >= start_edge) & (dfpulse.nevents_pulse < end_edge)]
        filtered_dfevent = shortened_dfevent[shortened_dfevent.pulseId.isin(filtered_dfpulse.pulseId)]
        filtered_etof = shortened_etof.sel(pulseId=shortened_etof.coords['pulseId'].isin(filtered_dfpulse.pulseId))
        
        hist, bin_edges = np.histogram(filtered_dfevent.tof, bins=nbins_ion_tof)
        hists.append(hist)
        
        numpy_etof = filtered_etof.to_numpy()
        reshaped_etof = numpy_etof.reshape((numpy_etof.shape[0], -1, e_group_size))
        summed_etof = np.sum(reshaped_etof, axis=-1)
        avg_etof = -np.mean(summed_etof, axis=0)
        hists_etof.append(avg_etof)
        
    hists = np.array(hists)
    hists_etof = np.array(hists_etof)
        
    return hists, hists_etof, bins



def stacked_ion_tof_max(hists,nbins_tof,bins,xlimits=(0,TIME_BETWEEN_PULSES),backgrd_dfevent=None):
    "Plots ion tof for each number of events defined on top of each other"
    "Can determine number of bins along time of flight and limits along x"
    
    precision = TIME_BETWEEN_PULSES/nbins_tof
    x_lower, x_upper = xlimits
    hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
    
    x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
    plt.figure(figsize=(20, 8))
    
    for i in range(len(hists)):
        
        shortened_hist = hists[i][hist_lower:hist_upper]
        plt.plot(x_edges, shortened_hist/max(shortened_hist), label=f'{bins[i]}-{bins[i+1]}')
        
    if isinstance(backgrd_dfevent, pd.DataFrame):
        hist, bin_edges = np.histogram(backgrd_dfevent.tof, bins=np.linspace(0,TIME_BETWEEN_PULSES,nbins_tof+1),range=(0,TIME_BETWEEN_PULSES))
        shortened_hist = hist[hist_lower:hist_upper]
        plt.plot(x_edges, shortened_hist/max(shortened_hist), label='Background')

    plt.xlabel('Time of flight (s)')
    plt.ylabel('Individually normalized signal')
    plt.title('Normalized ions time of flight')
    plt.legend()
    plt.xlim(x_lower, x_upper)
    plt.show()


def stacked_ion_tof_sum(hists,nbins_tof,bins,xlimits=(0,TIME_BETWEEN_PULSES),backgrd_dfevent=None):
    "Plots ion tof for each number of events defined on top of each other"
    "Can determine number of bins along time of flight and limits along x"
    
    precision = TIME_BETWEEN_PULSES/nbins_tof
    x_lower, x_upper = xlimits
    hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
    
    x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
    plt.figure(figsize=(20, 8))
    
    for i in range(len(hists)):
        
        shortened_hist = hists[i][hist_lower:hist_upper]
        plt.plot(x_edges, shortened_hist/sum(shortened_hist), label=f'{bins[i]}-{bins[i+1]}')
        
    if isinstance(backgrd_dfevent, pd.DataFrame):
        hist, bin_edges = np.histogram(backgrd_dfevent.tof, bins=np.linspace(0,TIME_BETWEEN_PULSES,nbins_tof+1),range=(0,TIME_BETWEEN_PULSES))
        shortened_hist = hist[hist_lower:hist_upper]
        plt.plot(x_edges, shortened_hist/sum(shortened_hist), label='Background')

    plt.xlabel('Time of flight (s)')
    plt.ylabel('Individually normalized signal')
    plt.title('Normalized ions time of flight')
    plt.legend()
    plt.xlim(x_lower, x_upper)
    plt.show() 
    


def waterfall_rel(hists,nbins_mq,xlimits=(0,200)):
    "Waterfall plot of relative normalization with respect to m/q using hists, which is a list of histograms"

    x_lower, x_upper = xlimits
    nbins_events = len(hists)
    bin_edges = np.linspace(0, 200, nbins_mq+1)

    plt.figure(figsize=(20, 8))
    colormap = plt.cm.inferno
    color_index = np.linspace(0.2, 0.8, nbins_events)

    for i in range(nbins_events):
        
        hist_norm = hists[i] / max(hists[i]) + i

        line_color = colormap(color_index[i])
        plt.plot(bin_edges[:-1], hist_norm, color=line_color)

    plt.title('Relative waterfall plot')
    plt.xlabel('m/q')
    plt.ylabel('Relative normalized counts')
    plt.xlim(x_lower, x_upper)
    plt.show()

    
    
def nevents_heatmap_rel(hists,nbins_mq,bins,xlimits=(0,200)):
    "Heatmap of relative normalized counts with number of events slices on the y axis, with respect to m/q on the x axis using hists, which is a list of histograms"
    
    precision = 200/nbins_mq
    x_lower, x_upper = xlimits
    hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
    nbins_events = len(hists)
    hists_norm = []

    for i in range(nbins_events):

        shortened_hist = hists[i][hist_lower:hist_upper]
        hist_norm = shortened_hist / max(shortened_hist)
        hists_norm.append(hist_norm)

    x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
    y_edges = bins[:-1]
    X, Y = np.meshgrid(x_edges, y_edges)

    plt.figure(figsize=(20, 8))
    c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
    plt.colorbar(c, label='Relative normalized counts', extend='max')
    plt.xlabel('m/q')
    plt.ylabel('Number of events slice')
    plt.title('Relative heatmap for number of events slices with respect to m/q')
    plt.xlim(x_lower, x_upper)
    plt.show()
    
    
    
def nevents_heatmap_rel_tof(hists,nbins_tof,bins,xlimits=(0,TIME_BETWEEN_PULSES)):
    "Heatmap of relative normalized counts with number of events slices on the y axis, with respect to tof on the x axis using hists, which is a list of histograms"
    
    precision = TIME_BETWEEN_PULSES/nbins_tof
    x_lower, x_upper = xlimits
    hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
    nbins_events = len(hists)
    hists_norm = []

    for i in range(nbins_events):

        shortened_hist = hists[i][hist_lower:hist_upper]
        hist_norm = shortened_hist / max(shortened_hist)
        hists_norm.append(hist_norm)

    x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
    X, Y = np.meshgrid(x_edges, bins[:-1])

    plt.figure(figsize=(20, 8))
    c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
    plt.colorbar(c, label='Relative normalized counts', extend='max')
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of events slice')
    plt.title('Relative heatmap for number of events slices with respect to tof')
    plt.xlim(x_lower, x_upper)
    # plt.yticks(bins[:-1])
    plt.show()
   
    
    
def waterfall_abs(hists,nbins_mq,xlimits=(0,200)):
    "Waterfall plot of absolute normalization with respect to m/q using hists, which is a list of histograms"

    x_lower, x_upper = xlimits
    nbins_events = len(hists)
    bin_edges = np.linspace(0, 200, nbins_mq+1)
    hists_norm = hists/np.max(hists)

    plt.figure(figsize=(20, 8))
    colormap = plt.cm.inferno
    color_index = np.linspace(0.2, 0.8, nbins_events)

    for i in range(nbins_events):

        line_color = colormap(color_index[i])
        plt.plot(bin_edges[:-1], hists_norm[i] + i, color=line_color)

    plt.title('Absolute waterfall plot')
    plt.xlabel('m/q')
    plt.ylabel('Relative normalized counts')
    plt.xlim(x_lower, x_upper)
    plt.show()

    
    
def nevents_heatmap_abs(hists,nbins_mq,bins,xlimits=(0,200)):
    "Heatmap of absolute normalized counts with number of events slices on the y axis, with respect to m/q on the x axis using hists, which is a list of histograms"
    
    precision = 200/nbins_mq
    x_lower, x_upper = xlimits
    hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
    hists_shortened = hists[:,hist_lower:hist_upper]
    nbins_events = len(hists_shortened)
    hists_norm = hists_shortened/np.max(hists_shortened)

    x_edges = np.linspace(0, 200, nbins_mq)
    y_edges = bins[:-1]
    X, Y = np.meshgrid(x_edges, y_edges)
    
    plt.figure(figsize=(20, 8))
    c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
    plt.colorbar(c, label='Relative normalized counts', extend='max')
    plt.xlabel('m/q')
    plt.ylabel('Number of events slice')
    plt.title('Absolute heatmap for number of events slices with respect to m/q')
    plt.xlim(x_lower, x_upper)
    plt.show()
    
    
    
def nevents_heatmap_abs_tof(hists,nbins_tof,bins,xlimits=(0,TIME_BETWEEN_PULSES)):
    "Heatmap of absolute normalized counts with number of events slices on the y axis, with respect to tof on the x axis using hists, which is a list of histograms"
    
    precision = TIME_BETWEEN_PULSES/nbins_tof
    x_lower, x_upper = xlimits
    hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
    hists_shortened = hists[:,hist_lower:hist_upper]
    nbins_events = len(hists_shortened)
    hists_norm = hists_shortened/np.max(hists_shortened)

    x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
    X, Y = np.meshgrid(x_edges, bins[:-1])

    plt.figure(figsize=(20, 8))
    c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
    plt.colorbar(c, label='Relative normalized counts', extend='max')
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of events slice')
    plt.title('Absolute heatmap for number of events slices with respect to tof')
    plt.xlim(x_lower, x_upper)
    # plt.yticks(bins[:-1])
    plt.show()
    
    
    
def waterfall_etof(filtered_etofs,xlimits=(0,200)):
    'Waterfall plot of etof data using list of etofs filtered_etofs'

    x_lower, x_upper = xlimits
    nbins = len(filtered_etofs)
    channel_time = TIME_BETWEEN_PULSES/CHANNELS_PER_PULSE
    xaxis = np.arange(14080)*channel_time

    plt.figure(figsize=(20, 8))
    colormap = plt.cm.inferno
    color_index = np.linspace(0.2, 0.8, nbins)

    for i in range(nbins):
        
        summed_etof = -np.sum(filtered_etofs[i],axis=0)
        line_color = colormap(color_index[i])
        plt.plot(xaxis, summed_etof/np.max(summed_etof) + i, color=line_color)

    plt.title('Relative electron waterfall plot')
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Normalized signal')
    plt.show()
    


def mq_np_covariance(dfevent,mq_bins=200,log=True,vmin=None,vmax=None):
    'Produces a positive and a negative covariance map of m/q vs m/q employing the numpy cov function'
    'Uses dfevent as input, can select number of mq bins, can produce plot as log, standard, or between defined ranges'
    
    mq_bin_edges = np.linspace(0,200,mq_bins+1)

    dfevent['mq_bin'] = pd.cut(dfevent['mq'], bins=mq_bin_edges)

    result_matrix = pd.crosstab(dfevent['pulseId'], dfevent['mq_bin'])
    result_numpy_matrix = result_matrix.values
    
    cov_matrix = np.cov(result_numpy_matrix, rowvar=False)
    

    plt.figure(figsize=(10, 8))
    if log == True:
        ax = sns.heatmap(cov_matrix, cmap='viridis', fmt='.2f', norm=LogNorm())
    elif vmax == None:
        ax = sns.heatmap(cov_matrix, cmap='viridis', fmt='.2f')
    else:
        ax = sns.heatmap(cov_matrix, cmap='viridis', fmt='.2f', vmin=0, vmax=vmax)

    tick_positions = np.linspace(0, mq_bins, 11)
    tick_labels = np.linspace(0, 200, 11).astype(int)

    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels)
    ax.set_yticks(tick_positions)
    ax.set_yticklabels(tick_labels)
    ax.invert_yaxis()

    plt.title('Positive Numpy Covariance Heatmap  m/q vs m/q')
    plt.xlabel('m/q')
    plt.ylabel('m/q')
    plt.show()
    

    plt.figure(figsize=(10, 8))
    if log == True:
        ax = sns.heatmap(-cov_matrix, cmap='viridis', fmt='.2f', norm=LogNorm())
    elif vmin == None:
        ax = sns.heatmap(-cov_matrix, cmap='viridis', fmt='.2f')
    else:
        ax = sns.heatmap(-cov_matrix, cmap='viridis', fmt='.2f', vmin=0, vmax=-vmin)

    tick_positions = np.linspace(0, mq_bins, 11)
    tick_labels = np.linspace(0, 200, 11).astype(int)

    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels)
    ax.set_yticks(tick_positions)
    ax.set_yticklabels(tick_labels)
    ax.invert_yaxis()

    plt.title('Negative Numpy Covariance Heatmap  m/q vs m/q')
    plt.xlabel('m/q')
    plt.ylabel('m/q')
    plt.show()



def mq_covariance_1d(dfevent,mq_bin_range,mq_bins=200):
    'Produces 1d covariance of an m/q bin range vs m/q employing the numpy cov function'
    'Uses dfevent as input, can select number of mq bins'
    
    mq_bin_edges = np.linspace(0,200,mq_bins+1)

    dfevent['mq_bin'] = pd.cut(dfevent['mq'], bins=mq_bin_edges)

    result_matrix = pd.crosstab(dfevent['pulseId'], dfevent['mq_bin'])
    result_numpy_matrix = result_matrix.values

    array_hyd = result_matrix.values[:,mq_bin_range[0]:mq_bin_range[1]].sum(axis=1)
    shape = result_numpy_matrix.shape[1]

    # Calculate the covariance between each row of result_numpy_matrix and array_hyd
    covariances = np.array([np.cov(np.column_stack((result_numpy_matrix[:, j], array_hyd)), rowvar=False)[0, 1] for j in range(shape)])

    xaxis = np.linspace(0,200,shape)
    fig, ax = plt.subplots()
    ax.plot(xaxis,covariances)
    plt.xlabel('m/q')
    plt.ylabel('Covariance with m/q')
    plt.title(f'Covariance Plot between m/q bin range {mq_bin_range} and m/q')
    ax.set_yscale('symlog', linthresh=10)
    plt.show()
    
    
    
def fix_missing_row(dfevent,dfpulse,mq_bins=200):
    'Fixes the missing row in dfevent dataframe when computing the cross-tabulation of pulseId and mq_bin'
    
    mq_bin_edges = np.linspace(0,200,mq_bins+1)

    dfevent['mq_bin'] = pd.cut(dfevent['mq'], bins=mq_bin_edges)

    result_matrix = pd.crosstab(dfevent['pulseId'], dfevent['mq_bin'])
    result_numpy_matrix = result_matrix.values
    
    resultlist = result_matrix.index.to_list()
    resultlist.append(0)
    selectedlist = dfpulse[dfpulse.pulseId.isin(dfevent.pulseId)].pulseId.to_list()
    truefalse = np.equal(resultlist,selectedlist)
    first_instance = np.argmax(~truefalse)
    missing_pulse = int(dfpulse.iloc[first_instance].pulseId)

    new_dfevent = dfevent[dfevent.pulseId != missing_pulse]
    
    return new_dfevent



def calc_corrs(array1, array2, pcovparams, alpha=1):
    print('calculating covariance')

    assert len(pcovparams)==len(array1)==len(array2)
    numshots=len(array1)
    
    # heavy stuff
    syx=np.einsum('ij,ik->jk', array1, array2)
    print('calculated syx')
    syi=np.einsum('ij,i->j', array1, pcovparams)
    print('calculated syi')
    six=np.einsum('ij,i->j', array2, pcovparams)
    print('calculated six')

    # lighter stuff
    sy=array1.sum(axis=0)
    sx=array2.sum(axis=0)
    si=pcovparams.sum(axis=0)
    
    syy=(array1**2).sum(axis=0)
    sxx=(array2**2).sum(axis=0)
    sii=(pcovparams**2).sum()

    sysx=np.outer(sy, sx)
    sisx=si*sx
    sysi=sy*si
    
    # calculate covariances
    covyx=(syx-sysx/numshots)/(numshots-1)
    covyi=(syi-sysi/numshots)/(numshots-1)
    covix=(six-sisx/numshots)/(numshots-1)

    covyy=(syy-sy**2/numshots)/(numshots-1)
    covxx=(sxx-sx**2/numshots)/(numshots-1)
    covii=(sii-si**2/numshots)/(numshots-1) # renamed from varii

    # calculate partial covariances
    pcovyx=(numshots-1)/(numshots-2) * (covyx - alpha * np.outer(covyi, covix)/covii)
    pcovyy=(numshots-1)/(numshots-2) * (covyy - (covyi**2)/covii)
    pcovxx=(numshots-1)/(numshots-2) * (covxx - (covix**2)/covii)
    
    # calculate correlation
    corryx = covyx / np.sqrt(np.outer(covyy, covxx))
    # calculate partial correlation
    pcorryx = pcovyx / np.sqrt(np.outer(pcovyy, pcovxx))
    
    return covyx, pcovyx, corryx, pcorryx



def mq_covariance(dfevent,dfpulse,mq_bins=200,log=True,vmin=None,vmax=None):
    'Produces covariance maps of m/q vs m/q employing the calc_corrs function'
    'Uses dfevent and dfpulse as inputs, can select number of mq bins, can produce plot as log, standard, or between defined ranges'
    
    mq_bin_edges = np.linspace(0,200,mq_bins+1)

    dfevent['mq_bin'] = pd.cut(dfevent['mq'], bins=mq_bin_edges)

    result_matrix = pd.crosstab(dfevent['pulseId'], dfevent['mq_bin'])
    result_numpy_matrix = result_matrix.values
    
    nevents_pulse = dfpulse[dfpulse.pulseId.isin(dfevent.pulseId)].nevents_pulse
    
    covyx, pcovyx, corryx, pcorryx = calc_corrs(result_numpy_matrix, result_numpy_matrix, nevents_pulse)
    
    
    tick_positions = np.linspace(0, mq_bins, 11)
    tick_labels = np.linspace(0, 200, 11).astype(int)
    
    plt.figure(figsize=(10, 8))
    if log == True:
        ax = sns.heatmap(covyx, cmap='viridis', fmt='.2f', norm=LogNorm())

        ax.set_xticks(tick_positions)
        ax.set_xticklabels(tick_labels)
        ax.set_yticks(tick_positions)
        ax.set_yticklabels(tick_labels)
        ax.invert_yaxis()

        plt.title('Positive Covariance Heatmap m/q vs m/q')
        plt.xlabel('m/q')
        plt.ylabel('m/q')
        plt.show()

        plt.figure(figsize=(10, 8))
        ax = sns.heatmap(-covyx, cmap='viridis', fmt='.2f', norm=LogNorm())

        ax.set_xticks(tick_positions)
        ax.set_xticklabels(tick_labels)
        ax.set_yticks(tick_positions)
        ax.set_yticklabels(tick_labels)
        ax.invert_yaxis()

        plt.title('Negative Covariance Heatmap m/q vs m/q')
        plt.xlabel('m/q')
        plt.ylabel('m/q')
        plt.show()
    
    elif vmax == None:
        ax = sns.heatmap(covyx, cmap='seismic', fmt='.2f')
        
        ax.set_xticks(tick_positions)
        ax.set_xticklabels(tick_labels)
        ax.set_yticks(tick_positions)
        ax.set_yticklabels(tick_labels)
        ax.invert_yaxis()

        plt.title('Covariance Heatmap m/q vs m/q')
        plt.xlabel('m/q')
        plt.ylabel('m/q')
        plt.show()
        
    else:
        ax = sns.heatmap(covyx, cmap='seismic', fmt='.2f', vmin=vmin, vmax=vmax)
        
        ax.set_xticks(tick_positions)
        ax.set_xticklabels(tick_labels)
        ax.set_yticks(tick_positions)
        ax.set_yticklabels(tick_labels)
        ax.invert_yaxis()

        plt.title('Covariance Heatmap m/q vs m/q')
        plt.xlabel('m/q')
        plt.ylabel('m/q')
        plt.show()
        
        

def mq_partial_covariance(dfevent,dfpulse,mq_bins=200,alpha=1,log=True,vmin=None,vmax=None):
    'Produces partial covariance maps of m/q vs m/q employing the calc_corrs function'
    'Uses dfevent and dfpulse as inputs, can select number of mq bins and factor alpha, can produce plot as log, standard, or between defined ranges'
    
    mq_bin_edges = np.linspace(0,200,mq_bins+1)

    dfevent['mq_bin'] = pd.cut(dfevent['mq'], bins=mq_bin_edges)

    result_matrix = pd.crosstab(dfevent['pulseId'], dfevent['mq_bin'])
    result_numpy_matrix = result_matrix.values
    
    nevents_pulse = dfpulse[dfpulse.pulseId.isin(dfevent.pulseId)].nevents_pulse
    
    covyx, pcovyx, corryx, pcorryx = calc_corrs(result_numpy_matrix, -result_numpy_matrix, nevents_pulse, alpha)
    
    
    tick_positions = np.linspace(0, mq_bins, 11)
    tick_labels = np.linspace(0, 200, 11).astype(int)
    
    plt.figure(figsize=(10, 8))
    if log == True:
        ax = sns.heatmap(pcovyx, cmap='viridis', fmt='.2f', norm=LogNorm())

        ax.set_xticks(tick_positions)
        ax.set_xticklabels(tick_labels)
        ax.set_yticks(tick_positions)
        ax.set_yticklabels(tick_labels)
        ax.invert_yaxis()

        plt.title('Positive Partial Covariance Heatmap m/q vs m/q')
        plt.xlabel('m/q')
        plt.ylabel('m/q')
        plt.show()

        plt.figure(figsize=(10, 8))
        ax = sns.heatmap(-pcovyx, cmap='viridis', fmt='.2f', norm=LogNorm())

        ax.set_xticks(tick_positions)
        ax.set_xticklabels(tick_labels)
        ax.set_yticks(tick_positions)
        ax.set_yticklabels(tick_labels)
        ax.invert_yaxis()

        plt.title('Negative Partial Covariance Heatmap m/q vs m/q')
        plt.xlabel('m/q')
        plt.ylabel('m/q')
        plt.show()
    
    elif vmax == None:
        ax = sns.heatmap(pcovyx, cmap='seismic', fmt='.2f')
        
        ax.set_xticks(tick_positions)
        ax.set_xticklabels(tick_labels)
        ax.set_yticks(tick_positions)
        ax.set_yticklabels(tick_labels)
        ax.invert_yaxis()

        plt.title('Partial Covariance Heatmap m/q vs m/q')
        plt.xlabel('m/q')
        plt.ylabel('m/q')
        plt.show()
        
    else:
        ax = sns.heatmap(pcovyx, cmap='seismic', fmt='.2f', vmin=vmin, vmax=vmax)
        
        ax.set_xticks(tick_positions)
        ax.set_xticklabels(tick_labels)
        ax.set_yticks(tick_positions)
        ax.set_yticklabels(tick_labels)
        ax.invert_yaxis()

        plt.title('Partial Covariance Heatmap m/q vs m/q')
        plt.xlabel('m/q')
        plt.ylabel('m/q')
        plt.show()


def ion_ion_covariance(dfevent, dfpulse, nbins_ion_tof, max_ion_limit=TIME_BETWEEN_PULSES, alpha=1, log=True, vmin=None, vmax=None):
    'Produces numpy covariance maps of ion tof vs ion tof employing the calc_corrs function'
    
    # Create bins for ion TOF
    ion_tof_bin_edges = np.linspace(0, max_ion_limit, nbins_ion_tof+1)
    dfevent['tof_bin'] = pd.cut(dfevent['tof'], bins=ion_tof_bin_edges)

    # Create ion matrix
    ion_matrix = pd.crosstab(dfevent['pulseId'], dfevent['tof_bin'])
    
    # Get number of events per pulse
    nevents_pulse = dfpulse[dfpulse.pulseId.isin(ion_matrix.index.to_numpy())].nevents_pulse

    # Calculate correlations
    covyx, pcovyx, corryx, pcorryx = calc_corrs(ion_matrix.values, ion_matrix.values, nevents_pulse, alpha)

    # Setup tick positions and labels
    ion_tick_positions = np.linspace(0, nbins_ion_tof, 5).astype(int)
    formatted_ion_tick_labels = np.linspace(0, max_ion_limit, 5)
    
    # Plotting functions
    def plot_heatmap(data, title, is_negative=False):
        plt.figure(figsize=(10, 8))
        if log:
            data_to_plot = -data if is_negative else data
            ax = sns.heatmap(data_to_plot, cmap='viridis', fmt='.2f', norm=LogNorm())
        elif vmax is None:
            ax = sns.heatmap(data, cmap='seismic', fmt='.2f')
        else:
            ax = sns.heatmap(data, cmap='seismic', fmt='.2f', vmin=vmin, vmax=vmax)
            
        ax.set_yticks(ion_tick_positions)
        ax.set_yticklabels(formatted_ion_tick_labels)
        ax.set_xticks(ion_tick_positions)
        ax.set_xticklabels(formatted_ion_tick_labels)
        ax.invert_yaxis()
        plt.title(title)
        plt.ylabel('Ion tof')
        plt.xlabel('Ion tof')
        plt.show()
    
    # Plot covariance maps
    if log:
        plot_heatmap(covyx, 'Positive Covariance Heatmap ion tof vs ion tof')
        plot_heatmap(covyx, 'Negative Covariance Heatmap ion tof vs ion tof', is_negative=True)
    else:
        plot_heatmap(covyx, 'Covariance Heatmap ion tof vs ion tof')
    
    # Plot partial covariance maps
    if log:
        plot_heatmap(pcovyx, 'Positive Partial Covariance Heatmap ion tof vs ion tof')
        plot_heatmap(pcovyx, 'Negative Partial Covariance Heatmap ion tof vs ion tof', is_negative=True)
    else:
        plot_heatmap(pcovyx, 'Partial Covariance Heatmap ion tof vs ion tof')

        
        
        
def etof_ion_covariance(dfevent,dfpulse,etof,nbins_ion_tof,nbins_e_tof,max_ion_limit=TIME_BETWEEN_PULSES,max_e_limit=TIME_BETWEEN_PULSES,alpha=1,log=True,vmin=None,vmax=None):
    'Produces numpy covariance maps of etof vs ion employing the calc_corrs function'
    'Uses dfevent and etof as inputs, can select number of bins along ion tof and electron tof, can produce plot as log, standard, or between defined ranges, and set alpha'
    'Can select maximal time limit for electrons and ions'
    
    ion_tof_bin_edges = np.linspace(0,max_ion_limit,nbins_ion_tof+1)
    dfevent['tof_bin'] = pd.cut(dfevent['tof'], bins=ion_tof_bin_edges)

    ion_matrix = pd.crosstab(dfevent['pulseId'], dfevent['tof_bin'])


    coords_etof = etof.assign_coords(data=np.arange(0,TIME_BETWEEN_PULSES,channel_time))

    e_tof_bin_edges = np.linspace(0,max_e_limit,nbins_e_tof+1)
    binned_etof = coords_etof.groupby_bins("data", e_tof_bin_edges).sum()
    e_matrix = binned_etof.to_pandas()
    
    ion_list = ion_matrix.index.to_list()
    # ion_list.append(0)
    e_list = e_matrix.index.to_list()
    truefalse = np.equal(ion_list,e_list)
    first_instance = np.argmax(~truefalse)

    new_e_matrix = e_matrix#.drop(e_list[first_instance])


    nevents_pulse = dfpulse[dfpulse.pulseId.isin(ion_matrix.index.to_numpy())].nevents_pulse

    covyx, pcovyx, corryx, pcorryx = calc_corrs(ion_matrix.values, -new_e_matrix.values, nevents_pulse, alpha)


    ion_tick_positions = np.linspace(0, nbins_ion_tof, 5).astype(int)
    formatted_ion_tick_labels = np.linspace(0, max_ion_limit, 5)

    e_tick_positions = np.linspace(0, nbins_e_tof, 5).astype(int)
    formatted_e_tick_labels = np.linspace(0, max_e_limit, 5)
    
    
    plt.figure(figsize=(10, 8))
    if log == True:
        ax = sns.heatmap(covyx, cmap='viridis', fmt='.2f', norm=LogNorm())
        ax.set_yticks(ion_tick_positions)
        ax.set_yticklabels(formatted_ion_tick_labels)
        ax.set_xticks(e_tick_positions)
        ax.set_xticklabels(formatted_e_tick_labels)
        ax.invert_yaxis()
        plt.title('Positive Covariance Heatmap etof vs ion tof')
        plt.ylabel('Ion tof')
        plt.xlabel('Electron tof')
        plt.show()

        plt.figure(figsize=(10, 8))
        ax = sns.heatmap(-covyx, cmap='viridis', fmt='.2f', norm=LogNorm())
        ax.set_yticks(ion_tick_positions)
        ax.set_yticklabels(formatted_ion_tick_labels)
        ax.set_xticks(e_tick_positions)
        ax.set_xticklabels(formatted_e_tick_labels)
        ax.invert_yaxis()
        plt.title('Negative Covariance Heatmap etof vs ion tof')
        plt.ylabel('Ion tof')
        plt.xlabel('Electron tof')
        plt.show()
    
    elif vmax == None:
        ax = sns.heatmap(covyx, cmap='seismic', fmt='.2f')
        ax.set_yticks(ion_tick_positions)
        ax.set_yticklabels(formatted_ion_tick_labels)
        ax.set_xticks(e_tick_positions)
        ax.set_xticklabels(formatted_e_tick_labels)
        ax.invert_yaxis()
        plt.title('Covariance Heatmap etof vs ion tof')
        plt.ylabel('Ion tof')
        plt.xlabel('Electron tof')
        plt.show()
        
    else:
        ax = sns.heatmap(covyx, cmap='seismic', fmt='.2f', vmin=vmin, vmax=vmax)
        ax.set_yticks(ion_tick_positions)
        ax.set_yticklabels(formatted_ion_tick_labels)
        ax.set_xticks(e_tick_positions)
        ax.set_xticklabels(formatted_e_tick_labels)
        ax.invert_yaxis()
        plt.title('Covariance Heatmap etof vs ion tof')
        plt.ylabel('Ion tof')
        plt.xlabel('Electron tof')
        plt.show()
    
    
    plt.figure(figsize=(10, 8))
    if log == True:
        ax = sns.heatmap(pcovyx, cmap='viridis', fmt='.2f', norm=LogNorm())
        ax.set_yticks(ion_tick_positions)
        ax.set_yticklabels(formatted_ion_tick_labels)
        ax.set_xticks(e_tick_positions)
        ax.set_xticklabels(formatted_e_tick_labels)
        ax.invert_yaxis()
        plt.title('Positive Partial Covariance Heatmap etof vs ion tof')
        plt.ylabel('Ion tof')
        plt.xlabel('Electron tof')
        plt.show()

        plt.figure(figsize=(10, 8))
        ax = sns.heatmap(-pcovyx, cmap='viridis', fmt='.2f', norm=LogNorm())
        ax.set_yticks(ion_tick_positions)
        ax.set_yticklabels(formatted_ion_tick_labels)
        ax.set_xticks(e_tick_positions)
        ax.set_xticklabels(formatted_e_tick_labels)
        ax.invert_yaxis()
        plt.title('Negative Partial Covariance Heatmap etof vs ion tof')
        plt.ylabel('Ion tof')
        plt.xlabel('Electron tof')
        plt.show()
    
    elif vmax == None:
        ax = sns.heatmap(pcovyx, cmap='seismic', fmt='.2f')
        ax.set_yticks(ion_tick_positions)
        ax.set_yticklabels(formatted_ion_tick_labels)
        ax.set_xticks(e_tick_positions)
        ax.set_xticklabels(formatted_e_tick_labels)
        ax.invert_yaxis()
        plt.title('Partial Covariance Heatmap etof vs ion tof')
        plt.ylabel('Ion tof')
        plt.xlabel('Electron tof')
        plt.show()
        
    else:
        ax = sns.heatmap(pcovyx, cmap='seismic', fmt='.2f', vmin=vmin, vmax=vmax)
        ax.set_yticks(ion_tick_positions)
        ax.set_yticklabels(formatted_ion_tick_labels)
        ax.set_xticks(e_tick_positions)
        ax.set_xticklabels(formatted_e_tick_labels)
        ax.invert_yaxis()
        plt.title('Partial Covariance Heatmap etof vs ion tof')
        plt.ylabel('Ion tof')
        plt.xlabel('Electron tof')
        plt.show()
        
        
        
def getBadPixelMask(avg_dark, dark_thr, std_dark, std_thr):
    mask = np.ones_like(avg_dark)

    mask[avg_dark>dark_thr]=0
    mask[std_dark>std_thr]=0
    mask[std_dark==0]=0

    return mask



def correct_pnccd(pnccd, dark_pnccd):
    'Correct pnccd using dark run pnccd input'
    
    avg_dark= dark_pnccd.mean(axis=0)
    std_dark= dark_pnccd.std(axis=0)
    
    mask = getBadPixelMask(avg_dark, 60000, std_dark, 600)
    
    corr_pnccd = (pnccd - avg_dark)*mask
    
    pretty_pnccd = corr_pnccd.isel(dim_0=slice(412, 512), dim_1=slice(412, 612))
    
    return pretty_pnccd
    
    
    
def pnccd_image(corr_pnccd, trainId):
    'Show pnccd image for given train id'
    
    image_data = corr_pnccd.sel(trainId=trainId).squeeze()
    
    plt.figure(figsize=(10,10))
    plt.imshow(image_data, vmin=10)
    plt.title(f'PNCCD image of train {trainId}')
    plt.show()
    
    
    
def integrate_pnccd(corr_pnccd, trainId, prnt=True):
    'Integrate upper half of pnncd for given train id'
    
    image_data = corr_pnccd.sel(trainId=trainId).squeeze()
    number = np.round(image_data.where(image_data > 10).sum().item())
    
    if prnt:
        print(f'Integrating yields {number}')
    else:
        return number

# Analysis

## Selections

### with pnccd

In [ ]:
RUNID = [232,233]

In [ ]:
LOWER_BOUND1 = 100
UPPER_BOUND1 = 500

LOWER_BOUND2 = 500
UPPER_BOUND2 = 1500

LOWER_BOUND3 = 2000
UPPER_BOUND3 = 5000

THRESHOLDS = [(LOWER_BOUND1, UPPER_BOUND1), (LOWER_BOUND2, UPPER_BOUND2), (LOWER_BOUND3, UPPER_BOUND3)]

selections = events_selection_plots(RUNID,THRESHOLDS)

selected_dfevent1, selected_dfpulse1, selected_etof1, selected_pnccd1 = selections[0]
selected_dfevent2, selected_dfpulse2, selected_etof2, selected_pnccd2 = selections[1]
selected_dfevent3, selected_dfpulse3, selected_etof3, selected_pnccd3 = selections[2]

In [ ]:
LOWER_BOUND1 = 5000
UPPER_BOUND1 = 8000

LOWER_BOUND2 = 8000
UPPER_BOUND2 = 12000

LOWER_BOUND3 = 12000
UPPER_BOUND3 = 20000

THRESHOLDS = [(LOWER_BOUND1, UPPER_BOUND1), (LOWER_BOUND2, UPPER_BOUND2), (LOWER_BOUND3, UPPER_BOUND3)]

selections = events_selection_plots(RUNID,THRESHOLDS)

selected_dfevent4, selected_dfpulse4, selected_etof4, selected_pnccd4 = selections[0]
selected_dfevent5, selected_dfpulse5, selected_etof5, selected_pnccd5 = selections[1]
selected_dfevent6, selected_dfpulse6, selected_etof6, selected_pnccd6 = selections[2]

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 5000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]

selections = events_selection_plots(RUNID,THRESHOLD)
selected_dfevent, selected_dfpulse, selected_etof, selected_pnccd = selections[0]

In [ ]:
LOWER_BACKGRD_BOUND = 20
UPPER_BACKGRD_BOUND = 40
BKGRD_THRESHOLD = [(LOWER_BACKGRD_BOUND,UPPER_BACKGRD_BOUND)]
DOWNSAMPLING = 200000

backgrd_dfevent, backgrd_dfpulse, backgrd_etof, backgrd_pnccd = events_selection(RUNID,BKGRD_THRESHOLD,DOWNSAMPLING)[0]

### only ions

In [ ]:
dfevent_90, dfpulse_90 = read_ion(405)

In [ ]:
dfevent_33, dfpulse_33 = read_ion(411)

In [ ]:
dfevent, dfpulse = read_ion(413)

In [ ]:
lower_threshold, upper_threshold = 500, 8000

selected_dfpulse = dfpulse[lower_threshold < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold]
selected_dfevent = dfevent[dfevent.pulseId.isin(selected_dfpulse.pulseId)]

selected_dfpulse_90 = dfpulse_90[lower_threshold < dfpulse_90.nevents_pulse][dfpulse_90.nevents_pulse < upper_threshold]
selected_dfevent_90 = dfevent_90[dfevent_90.pulseId.isin(selected_dfpulse_90.pulseId)]

selected_dfpulse_33 = dfpulse_33[lower_threshold < dfpulse_33.nevents_pulse][dfpulse_33.nevents_pulse < upper_threshold]
selected_dfevent_33 = dfevent_33[dfevent_33.pulseId.isin(selected_dfpulse_33.pulseId)]

In [ ]:
bounded_dfevent = selected_dfevent[selected_dfevent.tof < TIME_BETWEEN_PULSES]
hist, bin_edges = np.histogram(bounded_dfevent.tof, bins=1000)

bounded_dfevent_90 = selected_dfevent_90[selected_dfevent_90.tof < TIME_BETWEEN_PULSES]
hist_90, bin_edges = np.histogram(bounded_dfevent_90.tof, bins=1000)

bounded_dfevent_33 = selected_dfevent_33[selected_dfevent_33.tof < TIME_BETWEEN_PULSES]
hist_33, bin_edges = np.histogram(bounded_dfevent_33.tof, bins=1000)   
 
plt.figure()
plt.plot(bin_edges[:-1], hist/max(hist), label='EtOH background')
plt.plot(bin_edges[:-1], hist_90/max(hist_90), label='90% EtOH')
plt.plot(bin_edges[:-1], hist_33/max(hist_33), label='33% EtOH')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of hits per bin')
plt.title('Ions time of flight')
# plt.xlim(0,TIME_BETWEEN_PULSES)
plt.legend()
plt.show()  

In [ ]:
# 7594V 500nm
RUNIDstretch = [376,380,382,383,384,386,387,388,389,390,391,393,398,399,400,402,403,404]
# 5400V
# 500nm
RUNID500 = [232,233,313,315,316,317,318,319,320,321,322,325,326,327,328,329,331,332,333,336,337,338,339,340,341,342,343,345,347,348,349]
# 300nm
RUNID300 = [195,196,197,198,199,200,201,202,203,205,206,207,208,213,214,215,216,217,218,219,220,221,222,223,224,225,226]
# 100nm
RUNID100 = [293,294,296,297,298,299,300,301,302,303,304,306,307,308,309]
# core shell
RUNIDcoreshell = [227,228,229,273,274,275,276,277,278,279,280,282,283,284,285,286,287]
# water
RUNIDwater = [211]
# alcohol
RUNIDstrongalcohol = [405,406,407]
RUNIDweakalcohol = [409,411,412,413]
# photon energy
RUNIDphotonenergy = [440,441,441,442,443,444,445,446,447,448,449,450,451]

In [ ]:
LOWER_BOUND1 = 100
UPPER_BOUND1 = 500

LOWER_BOUND2 = 500
UPPER_BOUND2 = 1500

LOWER_BOUND3 = 2000
UPPER_BOUND3 = 5000

THRESHOLDS = [(LOWER_BOUND1, UPPER_BOUND1), (LOWER_BOUND2, UPPER_BOUND2), (LOWER_BOUND3, UPPER_BOUND3)]

ion_selections = ion_selection(RUNID500,THRESHOLDS)

ion_dfevent1, ion_dfpulse1 = ion_selections[0]
ion_dfevent2, ion_dfpulse2 = ion_selections[1]
ion_dfevent3, ion_dfpulse3 = ion_selections[2]

In [ ]:
LOWER_BOUND1 = 5000
UPPER_BOUND1 = 8000

LOWER_BOUND2 = 8000
UPPER_BOUND2 = 12000

LOWER_BOUND3 = 12000
UPPER_BOUND3 = 20000

THRESHOLDS = [(LOWER_BOUND1, UPPER_BOUND1), (LOWER_BOUND2, UPPER_BOUND2), (LOWER_BOUND3, UPPER_BOUND3)]

ion_selections = ion_selection(RUNID,THRESHOLDS)

ion_dfevent4, ion_dfpulse4 = ion_selections[0]
ion_dfevent5, ion_dfpulse5 = ion_selections[1]
ion_dfevent6, ion_dfpulse6 = ion_selections[2]

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 10000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]

ion_dfevent500, ion_dfpulse500 = ion_selection(RUNID500,THRESHOLD)[0]

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 10000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]

ion_dfevent500, ion_dfpulse500 = ion_selection(RUNID500,THRESHOLD)[0]

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 10000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]

ion_dfevent300, ion_dfpulse300 = ion_selection(RUNID300,THRESHOLD)[0]

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 5000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]

ion_dfevent100, ion_dfpulse100 = ion_selection(RUNID100,THRESHOLD)[0]

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 5000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]

ion_dfevent_strongalcohol, ion_dfpulse_strongalcohol = ion_selection(RUNIDstrongalcohol,THRESHOLD)[0]

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 5000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]

ion_dfevent_weakalcohol, ion_dfpulse_weakalcohol = ion_selection(RUNIDweakalcohol,THRESHOLD)[0]

In [ ]:
LOWER_BOUND = 2500
UPPER_BOUND = 3000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]

newion_dfevent300, newion_dfpulse300 = ion_selection(RUNID300,THRESHOLD)[0]

In [ ]:
LOWER_BOUND1 = 1000
UPPER_BOUND1 = 1500

LOWER_BOUND2 = 1500
UPPER_BOUND2 = 2000

LOWER_BOUND3 = 2000
UPPER_BOUND3 = 2500

THRESHOLDS = [(LOWER_BOUND1, UPPER_BOUND1), (LOWER_BOUND2, UPPER_BOUND2), (LOWER_BOUND3, UPPER_BOUND3)]

ion_selections = ion_selection(RUNID300,THRESHOLDS)

ion_dfevent1, ion_dfpulse1 = ion_selections[0]
ion_dfevent2, ion_dfpulse2 = ion_selections[1]
ion_dfevent3, ion_dfpulse3 = ion_selections[2]

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 5000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]

ion_dfevent, ion_dfpulse = ion_selection(RUNIDphotonenergy,THRESHOLD)[0]

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 5000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]

coreshell_ion_dfevent, coreshell_ion_dfpulse = ion_selection(RUNIDcoreshell,THRESHOLD)[0]

In [ ]:
LOWER_BOUND = 50
UPPER_BOUND = 17500
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]

ion_dfeventcoreshell, ion_dfpulsecoreshell = ion_selection(RUNIDcoreshell,THRESHOLD)[0]

In [ ]:
LOWER_BOUND = 50
UPPER_BOUND = 17500
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]

ion_dfeventwater, ion_dfpulsewater = ion_selection(RUNIDwater,THRESHOLD)[0]

In [ ]:
LOWER_BACKGRD_BOUND = 20
UPPER_BACKGRD_BOUND = 40
BKGRD_THRESHOLD = [(LOWER_BACKGRD_BOUND,UPPER_BACKGRD_BOUND)]
DOWNSAMPLING = 200000

backgrd_ion_dfevent500, backgrd_ion_dfpulse500 = ion_selection(RUNID500,BKGRD_THRESHOLD,DOWNSAMPLING)[0]

In [ ]:
LOWER_BACKGRD_BOUND = 20
UPPER_BACKGRD_BOUND = 40
BKGRD_THRESHOLD = [(LOWER_BACKGRD_BOUND,UPPER_BACKGRD_BOUND)]
DOWNSAMPLING = 100000

backgrd_ion_dfevent300, backgrd_ion_dfpulse300 = ion_selection(RUNID300,BKGRD_THRESHOLD,DOWNSAMPLING)[0]

In [ ]:
LOWER_BACKGRD_BOUND = 20
UPPER_BACKGRD_BOUND = 40
BKGRD_THRESHOLD = [(LOWER_BACKGRD_BOUND,UPPER_BACKGRD_BOUND)]
DOWNSAMPLING = 100000

backgrd_ion_dfevent100, backgrd_ion_dfpulse100 = ion_selection(RUNID100,BKGRD_THRESHOLD,DOWNSAMPLING)[0]

In [ ]:
num_bins = 100
nevents_binning = np.linspace(500, 17000, num_bins)

bin_counts = pd.cut(ion_dfpulse500['nevents_pulse'], bins=nevents_binning).value_counts().sort_index()

plt.figure()
plt.plot(nevents_binning[:-1], bin_counts)
plt.xlabel('Number of events per pulse')
plt.ylabel('Number of pulses')
plt.title('Number of pulses per number of events bin')
plt.yscale('log')
plt.axvline(1300)
plt.axvline(3400)
plt.axvline(6000)
plt.axvline(10000) 
plt.axvline(14000)
plt.axvline(16500)
plt.show()

In [ ]:
bins = [200, 1300, 3400, 6000, 10000]  # Define the bin ranges
labels = ['regime1', 'regime2', 'regime3', 'regime4']  # Assign labels to the bins

# Create a new column 'bin' with the bin labels
ion_dfpulse500['nevents_pulse_bin'] = pd.cut(ion_dfpulse500['nevents_pulse'], bins=bins, labels=labels)

# Group the DataFrame by the 'bin' column and create separate DataFrames
dataframes = {group: df for group, df in ion_dfpulse500.groupby('nevents_pulse_bin')}

# Access the individual DataFrames
regime1_ion_dfpulse = dataframes['regime1']
regime2_ion_dfpulse = dataframes['regime2']
regime3_ion_dfpulse = dataframes['regime3']
regime4_ion_dfpulse = dataframes['regime4']

ion_dfevent500 = ion_dfevent500[ion_dfevent500.tof < TIME_BETWEEN_PULSES]

regime1_ion_dfevent = ion_dfevent500[ion_dfevent500.pulseId.isin(regime1_ion_dfpulse.pulseId)]
regime2_ion_dfevent = ion_dfevent500[ion_dfevent500.pulseId.isin(regime2_ion_dfpulse.pulseId)]
regime3_ion_dfevent = ion_dfevent500[ion_dfevent500.pulseId.isin(regime3_ion_dfpulse.pulseId)]
regime4_ion_dfevent = ion_dfevent500[ion_dfevent500.pulseId.isin(regime4_ion_dfpulse.pulseId)]

In [ ]:
bins = [50, 1300, 3400, 6000, 10000]  # Define the bin ranges
labels = ['regime1', 'regime2', 'regime3', 'regime4']  # Assign labels to the bins

# Create a new column 'bin' with the bin labels
ion_dfpulse300['nevents_pulse_bin'] = pd.cut(ion_dfpulse300['nevents_pulse'], bins=bins, labels=labels)

# Group the DataFrame by the 'bin' column and create separate DataFrames
dataframes = {group: df for group, df in ion_dfpulse300.groupby('nevents_pulse_bin')}

# Access the individual DataFrames
regime1_ion_dfpulse = dataframes['regime1']
regime2_ion_dfpulse = dataframes['regime2']
regime3_ion_dfpulse = dataframes['regime3']
regime4_ion_dfpulse = dataframes['regime4']

ion_dfevent300 = ion_dfevent300[ion_dfevent300.tof < TIME_BETWEEN_PULSES]

regime1_ion_dfevent = ion_dfevent300[ion_dfevent300.pulseId.isin(regime1_ion_dfpulse.pulseId)]
regime2_ion_dfevent = ion_dfevent300[ion_dfevent300.pulseId.isin(regime2_ion_dfpulse.pulseId)]
regime3_ion_dfevent = ion_dfevent300[ion_dfevent300.pulseId.isin(regime3_ion_dfpulse.pulseId)]
regime4_ion_dfevent = ion_dfevent300[ion_dfevent300.pulseId.isin(regime4_ion_dfpulse.pulseId)]

In [ ]:
NBINS_TOF = 2000
hist1, bin_edges = np.histogram(regime1_ion_dfevent.tof, bins=NBINS_TOF)
hist2, bin_edges = np.histogram(regime2_ion_dfevent.tof, bins=NBINS_TOF)
hist3, bin_edges = np.histogram(regime3_ion_dfevent.tof, bins=NBINS_TOF)
hist4, bin_edges = np.histogram(regime4_ion_dfevent.tof, bins=NBINS_TOF)
regimehists = [hist1,hist2,hist3,hist4]

In [ ]:
stacked_ion_tof_max(regimehists,NBINS_TOF,bins,(3.5e-7,7e-7),backgrd_ion_dfevent500)

In [ ]:
stacked_ion_tof_max(regimehists,NBINS_TOF,bins,(1e-7,5e-7),backgrd_ion_dfevent500)

In [ ]:
stacked_ion_tof_max(regimehists,NBINS_TOF,bins,(1.05e-6,1.4e-6),backgrd_ion_dfevent)

In [ ]:
stacked_ion_tof_max(regimehists,NBINS_TOF,bins,(1.4e-6,1.8e-6),backgrd_ion_dfevent)

In [ ]:
stacked_ion_tof_max(regimehists,NBINS_TOF,bins,(1.8e-6,2.6e-6),backgrd_ion_dfevent)

In [ ]:
ion_dfpulse_max = ion_dfpulse500[ion_dfpulse500.nevents_pulse < 4000]
ion_dfevent_max = ion_dfevent500[ion_dfevent500.pulseId.isin(ion_dfpulse_max.pulseId)]

In [ ]:
ion_dfpulse_lower_max = ion_dfpulse500[ion_dfpulse500.nevents_pulse < 1000]
ion_dfevent_lower_max = ion_dfevent500[ion_dfevent500.pulseId.isin(ion_dfpulse_lower_max.pulseId)]

### transmission

In [ ]:
# Transmission percentages
RUNID1 = [376,380,382,383,384,398]
RUNID05 = [386,387,388,389,390,391,393,399]
RUNID025 = [400,402,403]
RUNID012 = [404]

In [ ]:
LOWER_BOUND = 50
UPPER_BOUND = 30000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]
    
ion_dfevent10, ion_dfpulse10 = ion_selection(RUNID1,THRESHOLD)[0]
ion_dfevent05, ion_dfpulse05 = ion_selection(RUNID05,THRESHOLD)[0]
ion_dfevent025, ion_dfpulse025 = ion_selection(RUNID025,THRESHOLD)[0]
ion_dfevent012, ion_dfpulse012 = ion_selection(RUNID012,THRESHOLD)[0]

In [ ]:
LOWER_BOUND1 = 100
UPPER_BOUND1 = 1000

LOWER_BOUND2 = 1000
UPPER_BOUND2 = 5000

LOWER_BOUND3 = 5000
UPPER_BOUND3 = 10000

THRESHOLDS = [(LOWER_BOUND1, UPPER_BOUND1), (LOWER_BOUND2, UPPER_BOUND2), (LOWER_BOUND3, UPPER_BOUND3)]

ion_selections = ion_selection(RUNID1,THRESHOLDS)

ion_dfevent10_1, ion_dfpulse10_1 = ion_selections[0]
ion_dfevent10_2, ion_dfpulse10_2 = ion_selections[1]
ion_dfevent10_3, ion_dfpulse10_3 = ion_selections[2]

ion_selections = ion_selection(RUNID05,THRESHOLDS)

ion_dfevent05_1, ion_dfpulse05_1 = ion_selections[0]
ion_dfevent05_2, ion_dfpulse05_2 = ion_selections[1]
ion_dfevent05_3, ion_dfpulse05_3 = ion_selections[2]

ion_selections = ion_selection(RUNID025,THRESHOLDS)

ion_dfevent025_1, ion_dfpulse025_1 = ion_selections[0]
ion_dfevent025_2, ion_dfpulse025_2 = ion_selections[1]
ion_dfevent025_3, ion_dfpulse025_3 = ion_selections[2]

ion_selections = ion_selection(RUNID012,THRESHOLDS)

ion_dfevent012_1, ion_dfpulse012_1 = ion_selections[0]
ion_dfevent012_2, ion_dfpulse012_2 = ion_selections[1]
ion_dfevent012_3, ion_dfpulse012_3 = ion_selections[2]

In [ ]:
num_bins = 100
nevents_binning = np.linspace(LOWER_BOUND, UPPER_BOUND, num_bins)

bin_counts = pd.cut(ion_dfpulse['nevents_pulse'], bins=nevents_binning).value_counts().sort_index()
bin_counts10 = pd.cut(ion_dfpulse10['nevents_pulse'], bins=nevents_binning).value_counts().sort_index()
bin_counts05 = pd.cut(ion_dfpulse05['nevents_pulse'], bins=nevents_binning).value_counts().sort_index()
bin_counts025 = pd.cut(ion_dfpulse025['nevents_pulse'], bins=nevents_binning).value_counts().sort_index()
bin_counts012 = pd.cut(ion_dfpulse012['nevents_pulse'], bins=nevents_binning).value_counts().sort_index()

plt.figure(figsize=(10,6))
plt.plot(nevents_binning[:-1], bin_counts/max(bin_counts), label='total')
plt.plot(nevents_binning[:-1], bin_counts10/max(bin_counts10), label='1% transmission')
plt.plot(nevents_binning[:-1], bin_counts05/max(bin_counts05), label='.5% transmisison')
plt.plot(nevents_binning[:-1], bin_counts025/max(bin_counts025), label='.25% transmisison')
plt.plot(nevents_binning[:-1], bin_counts012/max(bin_counts012), label='.12% transmisison')
plt.xlabel('Number of events per pulse')
plt.ylabel('Number of pulses')
plt.title('Number of pulses per number of events bin')
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
LOWER_BACKGRD_BOUND = 20
UPPER_BACKGRD_BOUND = 40
BKGRD_THRESHOLD = [(LOWER_BACKGRD_BOUND,UPPER_BACKGRD_BOUND)]
DOWNSAMPLING = 200000

backgrd_ion_dfevent, backgrd_ion_dfpulse = ion_selection(RUNID,BKGRD_THRESHOLD,DOWNSAMPLING)[0]

### Ions and photons

In [ ]:
# 5400V
# 500nm
# remove no pnccd runs
RUNID500 = [232,233,315,316,317,318,319,320,322,325,326,327,328,329,331,332,333,336,337,338,339,340,341,342,343,345,347,348,349]
# 300nm
RUNID300 = [195,196,197,198,199,200,201,202,203,205,206,207,208,213,214,215,216,217,218,219,220,221,222,223,224,225,226]
# 100nm
RUNID100 = [293,294,296,297,298,299,300,301,302,303,304,306,307,308,309]

In [ ]:
LOWER_BOUND1 = 1000
UPPER_BOUND1 = 5000

LOWER_BOUND2 = 5000
UPPER_BOUND2 = 7000

LOWER_BOUND3 = 8000
UPPER_BOUND3 = 10000

THRESHOLDS = [(LOWER_BOUND1, UPPER_BOUND1)]#, (LOWER_BOUND2, UPPER_BOUND2), (LOWER_BOUND3, UPPER_BOUND3)]

selections = events_selection_ions_photons(RUNID100,THRESHOLDS)

selected_dfevent1, selected_dfpulse1, selected_pnccd1 = selections[0]
#selected_dfevent2, selected_dfpulse2, selected_pnccd2 = selections[1]
#selected_dfevent3, selected_dfpulse3, selected_pnccd3 = selections[2]

### Ions and electrons

In [ ]:
# 5400V
# 500nm
# remove no pnccd runs
RUNID500 = [315,316,317,318,319,320,322,325,326,327,328,329,331,332,333]#,336,337,338,339,340,341,342,343,345,347,348,349]#[232,233,
RUNID300 = [195,196,197,198,199,200,201,202,203,205,206,207,208,213,214,215,216,217,218,219,220,221,222,223,224,225,226]
RUNID100 = [293,294,296,297,298,299,300,301,302,303,304,306,307,308,309]

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 10000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]
TOF_LIMIT = 6e-7

selected_dfevent, selected_dfpulse, selected_etof = events_selection_ions_electrons(RUNID500,THRESHOLD,tof_limit=TOF_LIMIT)[0]

In [ ]:
selected_etof.to_netcdf("selected_etof.nc")

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 10000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]
TOF_LIMIT = 6e-7

selected_dfevent300, selected_dfpulse300, selected_etof300 = events_selection_ions_electrons(RUNID300,THRESHOLD,tof_limit=TOF_LIMIT)[0]

In [ ]:
selected_etof300.to_netcdf("selected_etof300.nc")

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 6000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]
TOF_LIMIT = 6e-7

selected_dfevent100, selected_dfpulse100, selected_etof100 = events_selection_ions_electrons(RUNID100,THRESHOLD,tof_limit=TOF_LIMIT)[0]

In [ ]:
selected_etof100.to_netcdf("selected_etof100.nc")

In [ ]:
LOWER_BOUND = 500
UPPER_BOUND = 6000
THRESHOLD = [(LOWER_BOUND,UPPER_BOUND)]
TOF_LIMIT = 6e-7

selected_dfevent_coreshell, selected_dfpulse_coreshell, selected_etof_coreshell = events_selection_ions_electrons([227,228,229,273,274,275,276,277,278],THRESHOLD,tof_limit=TOF_LIMIT)[0]

In [ ]:
selected_etof_coreshell.to_netcdf("selected_etof_coreshell.nc")

In [ ]:
NBINS_EVENTS = 50
NBINS_TOF = 1000

filtered_dfevents, filtered_dfpulses, filtered_etofs, hists, bins = nevents_binning_tof(selected_dfevent,selected_dfpulse,selected_etof,NBINS_EVENTS,NBINS_TOF)

In [ ]:
NBINS_EVENTS = 50
NBINS_TOF = 1000

filtered_dfevents300, filtered_dfpulses300, filtered_etofs300, hists300, bins300 = nevents_binning_tof(selected_dfevent300,selected_dfpulse300,selected_etof300,NBINS_EVENTS,NBINS_TOF)

In [ ]:
NBINS_EVENTS = 30
NBINS_TOF = 1000

filtered_dfevents100, filtered_dfpulses100, filtered_etofs100, hists100, bins100 = nevents_binning_tof(selected_dfevent100,selected_dfpulse100,selected_etof100,NBINS_EVENTS,NBINS_TOF)

In [ ]:
NBINS_EVENTS = 30
NBINS_TOF = 1000

filtered_dfevents_coreshell, filtered_dfpulses_coreshell, filtered_etofs_coreshell, hists_coreshell, bins_coreshell = nevents_binning_tof(selected_dfevent_coreshell,selected_dfpulse_coreshell,selected_etof_coreshell,NBINS_EVENTS,NBINS_TOF)

In [ ]:
selected_etof.to_netcdf("selected_etof.nc")
selected_etof300.to_netcdf("selected_etof300.nc")
selected_etof100.to_netcdf("selected_etof100.nc")
selected_etof_coreshell.to_netcdf("selected_etof_coreshell.nc")

In [ ]:
selected_etof = xr.open_dataset("selected_etof.nc")
selected_etof300 = xr.open_dataset("selected_etof300.nc")
selected_etof100 = xr.open_dataset("selected_etof100.nc")
selected_etof_coreshell = xr.open_dataset("selected_etof_coreshell.nc")

In [ ]:
x_lower, x_upper = (1e-7,2e-7)
tof_lower, tof_upper = int(x_lower/channel_time), int(x_upper/channel_time)


avg_etofs_500 = []
for etof in filtered_etofs:
    short_etof = etof.isel(data=slice(tof_lower, tof_upper))
    avg_etof = -np.mean(short_etof,axis=0)
    avg_etofs_500.append(sum(avg_etof))     

avg_etofs_300 = []
for etof in filtered_etofs300:
    short_etof = etof.isel(data=slice(tof_lower, tof_upper))
    avg_etof = -np.mean(short_etof,axis=0)
    avg_etofs_300.append(sum(avg_etof))

avg_etofs_100 = []
for etof in filtered_etofs100:
    short_etof = etof.isel(data=slice(tof_lower, tof_upper))
    avg_etof = -np.mean(short_etof,axis=0)
    avg_etofs_100.append(sum(avg_etof))

avg_etofs_coreshell = []
for etof in filtered_etofs_coreshell:
    short_etof = etof.isel(data=slice(tof_lower, tof_upper))
    avg_etof = -np.mean(short_etof,axis=0)
    avg_etofs_coreshell.append(sum(avg_etof)) 

In [ ]:
plt.figure(figsize=(12,6))

plt.plot(bins[:-1], avg_etofs_500, label='500nm SiO$_2$', linewidth=2.5)
plt.plot(bins300[:-1], avg_etofs_300, label='300nm SiO$_2$', linewidth=2.5)
plt.plot(bins100[:-1], avg_etofs_100, label='100nm SiO$_2$', linewidth=2.5)
plt.plot(bins_coreshell[:-1], avg_etofs_coreshell, label='140nm SiO$_2$@Au', linewidth=2.5)

plt.xlabel('Normalized ion counts (arb. unit)', fontsize=18)
plt.ylabel('Number of electrons detected', fontsize=18)

# plt.xlim(120,490)
# plt.ylim(0,2.9)
plt.yscale('log')
plt.tick_params(axis='both', labelsize=14)

plt.legend(fontsize=16)
plt.show()

In [ ]:
def some_sum(etofs,nbins_tof,bins,xlimits=(0,TIME_BETWEEN_PULSES)):
    "Etof sum values"

    x_lower, x_upper = xlimits
    tof_lower, tof_upper = int(x_lower/channel_time), int(x_upper/channel_time)

    avg_etofs = []
    for etof in etofs:
        short_etof = etof.isel(data=slice(tof_lower, tof_upper))
        avg_etof = -np.mean(short_etof,axis=0)
        avg_etofs.append(sum(avg_etof))
        
    plt.figure(figsize=(20, 8))
    plt.plot(bins[:-1],avg_etofs)
    plt.show()
    

In [ ]:
nevents_heatmap_abs_tof(hists,NBINS_TOF,bins,(0,TOF_LIMIT))

In [ ]:
nevents_heatmap_rel_tof(hists,NBINS_TOF,bins,(0,TOF_LIMIT))

In [ ]:
nevents_heatmap_abs_tof(hists,NBINS_TOF,bins,(1.1e-6,1.4e-6))

In [ ]:
nevents_heatmap_rel_tof(hists,NBINS_TOF,bins,(1.1e-6,1.4e-6))

In [ ]:
etof_nevents_heatmap_abs_tof(filtered_etofs,NBINS_TOF,bins,(1e-8,TOF_LIMIT))

In [ ]:
etof_nevents_heatmap_rel_tof(filtered_etofs,NBINS_TOF,bins,(1e-8,TOF_LIMIT))

In [ ]:
etof_nevents_heatmap_rel_tof(filtered_etofs300,NBINS_TOF,bins300,(1e-8,TOF_LIMIT))

In [ ]:
etof_nevents_heatmap_rel_tof(filtered_etofs100,NBINS_TOF,bins100,(1e-8,TOF_LIMIT))

In [ ]:
some(filtered_etofs,NBINS_TOF,bins,(1e-7,2e-7))

In [ ]:
some(filtered_etofs300,NBINS_TOF,bins300,(1e-7,2e-7))

In [ ]:
some(filtered_etofs100,NBINS_TOF,bins100,(1e-7,2e-7))

In [ ]:
some_sum(filtered_etofs,NBINS_TOF,bins,(1e-7,2e-7))

In [ ]:
some_sum(filtered_etofs,NBINS_TOF,bins,(1e-7,2e-7))

In [ ]:
some_sum(filtered_etofs300,NBINS_TOF,bins300,(1e-7,2e-7))

In [ ]:
some_sum(filtered_etofs100,NBINS_TOF,bins100,(1e-7,2e-7))

In [ ]:
new_selected_dfpulse = selected_dfpulse[selected_dfpulse.nevents_pulse < 5000]
new_selected_dfevent = selected_dfevent[selected_dfevent.pulseId.isin(new_selected_dfpulse.pulseId)]
new_selected_etof = selected_etof.sel(pulseId=selected_etof.coords['pulseId'].isin(new_selected_dfpulse.pulseId))

In [ ]:
new_selected_dfpulse300 = selected_dfpulse300[selected_dfpulse300.nevents_pulse < 5000]
new_selected_dfevent300 = selected_dfevent300[selected_dfevent300.pulseId.isin(new_selected_dfpulse300.pulseId)]
new_selected_etof300 = selected_etof300.sel(pulseId=selected_etof300.coords['pulseId'].isin(new_selected_dfpulse300.pulseId))

In [ ]:
new_selected_dfpulse100 = selected_dfpulse100[selected_dfpulse100.nevents_pulse < 5000]
new_selected_dfevent100 = selected_dfevent100[selected_dfevent100.pulseId.isin(new_selected_dfpulse100.pulseId)]
new_selected_etof100 = selected_etof100.sel(pulseId=selected_etof100.coords['pulseId'].isin(new_selected_dfpulse100.pulseId))

In [ ]:
NBINS_EVENTS = 40
NBINS_TOF = 4000

new_filtered_dfevents, new_filtered_dfpulses, new_filtered_etofs, new_hists, new_bins = nevents_binning_tof(new_selected_dfevent,new_selected_dfpulse,new_selected_etof,NBINS_EVENTS,NBINS_TOF)

In [ ]:
NBINS_EVENTS = 20
NBINS_TOF = 1000

new_filtered_dfevents300, new_filtered_dfpulses300, new_filtered_etofs300, new_hists300, new_bins300 = nevents_binning_tof(new_selected_dfevent300,new_selected_dfpulse300,new_selected_etof300,NBINS_EVENTS,NBINS_TOF)

In [ ]:
NBINS_EVENTS = 20
NBINS_TOF = 1000

fnew_iltered_dfevents100, new_filtered_dfpulses100, new_filtered_etofs100, new_hists100, new_bins100 = nevents_binning_tof(new_selected_dfevent100,new_selected_dfpulse100,new_selected_etof100,NBINS_EVENTS,NBINS_TOF)

In [ ]:
nbins_tof = NBINS_TOF
xlimits = (1e-7,5e-7)

hists=new_hists
bins = new_bins

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins[:-1])

X_ns = X * 1e9

In [ ]:
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X_ns, Y, hists_norm, cmap='magma', shading='auto')
colorbar = plt.colorbar(c, label='Normalized counts (arb. units)', extend='max')
plt.xlabel('Time of flight (ns)', fontsize=20)
plt.ylabel('Hit intensity (arb. unit)', fontsize=20)
plt.xlim(110, 500)
plt.ylim(550,4900)

ax1 = plt.gca()
ax1.tick_params(axis='x', labelsize=16)
ax1.tick_params(axis='y', labelsize=16)
colorbar.ax.set_ylabel('Normalized counts (arb. unit)', fontsize=20)
colorbar.ax.tick_params(labelsize=16)

ax2 = ax1.twiny()
top_tick_positions = [210,227]
top_tick_labels = ['125', '0']
ax2.set_xticks(top_tick_positions)
ax2.set_xticklabels(top_tick_labels)

ax2.set_xlim(ax1.get_xlim())
ax2.set_xlabel('Kinetic Energy (eV)', fontsize=20)

ax2.spines['top'].set_color('r')
ax2.xaxis.label.set_color('r')
ax2.tick_params(axis='x', colors='r', labelsize=16)

plt.axvline(227,c='r')
plt.axvline(210,c='r')

plt.show()

In [ ]:
some(new_filtered_etofs,NBINS_TOF,new_bins,(1e-7,2e-7))

In [ ]:
some(new_filtered_etofs300,NBINS_TOF,new_bins300,(1e-7,2e-7))

In [ ]:

    x_lower, x_upper = xlimits
    tof_lower, tof_upper = int(x_lower/channel_time), int(x_upper/channel_time)

    avg_etofs = []
    for etof in etofs:
        short_etof = etof.isel(data=slice(tof_lower, tof_upper))
        avg_etof = -np.mean(short_etof,axis=0)
        avg_etofs.append(max(avg_etof))
        
    plt.figure(figsize=(20, 8))
    plt.plot(bins[:-1],avg_etofs)
    plt.show()

In [ ]:
x_lower, x_upper = (1e-7,2e-7)
tof_lower, tof_upper = int(x_lower/channel_time), int(x_upper/channel_time)

short_etof = filtered_etofs[0].isel(data=slice(tof_lower, tof_upper))

In [ ]:
def some(etofs,nbins_tof,bins,xlimits=(0,TIME_BETWEEN_PULSES)):
    "Etof max values"

    x_lower, x_upper = xlimits
    tof_lower, tof_upper = int(x_lower/channel_time), int(x_upper/channel_time)

    avg_etofs = []
    for etof in etofs:
        short_etof = etof.isel(data=slice(tof_lower, tof_upper))
        avg_etof = -np.mean(short_etof,axis=0)
        avg_etofs.append(max(avg_etof))
        
    plt.figure(figsize=(20, 8))
    plt.plot(bins[:-1],avg_etofs)
    plt.show()
    
    

def some_sum(etofs,nbins_tof,bins,xlimits=(0,TIME_BETWEEN_PULSES)):
    "Etof sum values"

    x_lower, x_upper = xlimits
    tof_lower, tof_upper = int(x_lower/channel_time), int(x_upper/channel_time)

    avg_etofs = []
    for etof in etofs:
        short_etof = etof.isel(data=slice(tof_lower, tof_upper))
        avg_etof = -np.mean(short_etof,axis=0)
        avg_etofs.append(sum(avg_etof))
        
    plt.figure(figsize=(20, 8))
    plt.plot(bins[:-1],avg_etofs)
    plt.show()
    
    

def etof_nevents_heatmap_abs_tof(etofs,nbins_tof,bins,xlimits=(0,TIME_BETWEEN_PULSES)):
    "Etof heatmap of relative normalized counts with number of events slices on the y axis, with respect to tof on the x axis using hists, which is a list of histograms"

    x_lower, x_upper = xlimits
    tof_lower, tof_upper = int(x_lower/channel_time), int(x_upper/channel_time)

    avg_etofs = []
    for etof in etofs:
        short_etof = etof.isel(data=slice(tof_lower, tof_upper))
        avg_etof = -np.mean(short_etof,axis=0)
        avg_etofs.append(avg_etof)

    x_edges = np.linspace(x_lower, x_upper, tof_upper-tof_lower)
    X, Y = np.meshgrid(x_edges, bins[:-1])

    plt.figure(figsize=(20, 8))
    c = plt.pcolormesh(X, Y, avg_etofs, shading='auto')
    plt.colorbar(c, label='Signal', extend='max')
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of events slice')
    plt.title('Electrons heatmap for number of events slices with respect to tof')
    plt.xlim(x_lower, x_upper)
    plt.show()



def etof_nevents_heatmap_rel_tof(etofs,nbins_tof,bins,xlimits=(0,TIME_BETWEEN_PULSES)):
    "Etof heatmap of relative normalized counts with number of events slices on the y axis, with respect to tof on the x axis using hists, which is a list of histograms"

    x_lower, x_upper = xlimits
    tof_lower, tof_upper = int(x_lower/channel_time), int(x_upper/channel_time)

    norm_avg_etofs = []
    for etof in etofs:
        short_etof = etof.isel(data=slice(tof_lower, tof_upper))
        avg_etof = -np.mean(short_etof,axis=0)
        norm_avg_etofs.append(avg_etof/max(avg_etof))

    x_edges = np.linspace(x_lower, x_upper, tof_upper-tof_lower)
    X, Y = np.meshgrid(x_edges, bins[:-1])

    plt.figure(figsize=(20, 8))
    c = plt.pcolormesh(X, Y, norm_avg_etofs, shading='auto')
    plt.colorbar(c, label='Signal normalized per events slice', extend='max')
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of events slice')
    plt.title('Electrons heatmap for number of events slices with respect to tof')
    plt.xlim(x_lower, x_upper)
    plt.show()

In [ ]:
def nevents_heatmap_rel_tof(hists,nbins_tof,bins,xlimits=(0,TIME_BETWEEN_PULSES)):
    "Heatmap of relative normalized counts with number of events slices on the y axis, with respect to tof on the x axis using hists, which is a list of histograms"
    
    precision = TIME_BETWEEN_PULSES/nbins_tof
    x_lower, x_upper = xlimits
    hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
    nbins_events = len(hists)
    hists_norm = []

    for i in range(nbins_events):

        shortened_hist = hists[i][hist_lower:hist_upper]
        hist_norm = shortened_hist / max(shortened_hist)
        hists_norm.append(hist_norm)

    x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
    X, Y = np.meshgrid(x_edges, bins[:-1])

    plt.figure(figsize=(20, 8))
    c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
    plt.colorbar(c, label='Relative normalized counts', extend='max')
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of events slice')
    plt.title('Relative heatmap for number of events slices with respect to tof')
    plt.xlim(x_lower, x_upper)
    # plt.yticks(bins[:-1])
    plt.show()

def nevents_heatmap_abs_tof(hists,nbins_tof,bins,xlimits=(0,TIME_BETWEEN_PULSES)):
    "Heatmap of absolute normalized counts with number of events slices on the y axis, with respect to tof on the x axis using hists, which is a list of histograms"
    
    precision = TIME_BETWEEN_PULSES/nbins_tof
    x_lower, x_upper = xlimits
    hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
    hists_shortened = hists[:,hist_lower:hist_upper]
    nbins_events = len(hists_shortened)
    hists_norm = hists_shortened/np.max(hists_shortened)

    x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
    X, Y = np.meshgrid(x_edges, bins[:-1])

    plt.figure(figsize=(20, 8))
    c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
    plt.colorbar(c, label='Relative normalized counts', extend='max')
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of events slice')
    plt.title('Absolute heatmap for number of events slices with respect to tof')
    plt.xlim(x_lower, x_upper)
    # plt.yticks(bins[:-1])
    plt.show()

In [ ]:
LOWER_BOUND1 = 1000
UPPER_BOUND1 = 3000

LOWER_BOUND2 = 5000
UPPER_BOUND2 = 8000

LOWER_BOUND3 = 10000
UPPER_BOUND3 = 15000

THRESHOLDS = [(LOWER_BOUND1, UPPER_BOUND1), (LOWER_BOUND2, UPPER_BOUND2), (LOWER_BOUND3, UPPER_BOUND3)]

ion_electron_selections = events_selection_ions_electrons(RUNID500,THRESHOLDS)

dfevent1, dfpulse1, etof1 = ion_electron_selections[0]
dfevent2, dfpulse2, etof2 = ion_electron_selections[1]
dfevent3, dfpulse3, etof3 = ion_electron_selections[2]

In [ ]:
LOWER_BACKGRD_BOUND = 20
UPPER_BACKGRD_BOUND = 40
BKGRD_THRESHOLD = [(LOWER_BACKGRD_BOUND,UPPER_BACKGRD_BOUND)]
DOWNSAMPLING = 20000

backgrd_dfevent, backgrd_dfpulse, backgrd_etof = events_selection_ions_electrons(RUNID500,BKGRD_THRESHOLD,DOWNSAMPLING)[0]

In [ ]:
xaxis = np.arange(14080)*channel_time

avg_etof1 = -np.mean(etof1, axis=0)
max_etof1 = max(avg_etof1)
avg_etof2 = -np.mean(etof2, axis=0)
max_etof2 = max(avg_etof2)
avg_etof3 = -np.mean(etof3, axis=0)
max_etof3 = max(avg_etof3)
avg_backgrd_etof = -np.mean(backgrd_etof, axis=0)
max_backgrd_etof = max(avg_backgrd_etof)

plt.figure()
plt.plot(xaxis, avg_etof1/max_etof1, label='1000-3000 events')
plt.plot(xaxis, avg_etof2/max_etof2, label='5000-8000 events')
plt.plot(xaxis, avg_etof3/max_etof3, label='10000-15000 events')
plt.plot(xaxis, avg_backgrd_etof/max_backgrd_etof, label='Background')
plt.xlabel('Time of flight (s)')
plt.ylabel('Normalized signal')
plt.title('Electrons time of flight')
plt.legend()
plt.show()

In [ ]:
bgrd_subs1 = np.abs(avg_etof1-avg_backgrd_etof)/np.abs(avg_backgrd_etof)
bgrd_subs2 = np.abs(avg_etof2-avg_backgrd_etof)/np.abs(avg_backgrd_etof)
bgrd_subs3 = np.abs(avg_etof3-avg_backgrd_etof)/np.abs(avg_backgrd_etof)

plt.figure()
plt.plot(xaxis, bgrd_subs1/max(bgrd_subs1), label='1000-3000 events')
plt.plot(xaxis, bgrd_subs2/max(bgrd_subs2), label='5000-8000 events')
plt.plot(xaxis, bgrd_subs3/max(bgrd_subs3), label='10000-15000 events')
plt.xlabel('Time of flight (s)')
plt.ylabel('Signal')
plt.title('Background substracted electrons time of flight')
plt.legend()
plt.show()

In [ ]:
max_time = 6e-7
max_coord = int(max_time/channel_time)

avg_etof1 = avg_etof1[:max_coord]
max_etof1 = max(avg_etof1)
avg_etof2 = avg_etof2[:max_coord]
max_etof2 = max(avg_etof2)
avg_etof3 = avg_etof3[:max_coord]
max_etof3 = max(avg_etof3)
avg_backgrd_etof = avg_backgrd_etof[:max_coord]
max_backgrd_etof = max(avg_backgrd_etof)

xaxis = np.arange(max_coord)*channel_time

plt.figure()
plt.plot(xaxis, avg_etof1/max_etof1, label='1000-3000 events')
plt.plot(xaxis, avg_etof2/max_etof2, label='5000-8000 events')
plt.plot(xaxis, avg_etof3/max_etof3, label='10000-15000 events')
plt.plot(xaxis, avg_backgrd_etof/max_backgrd_etof, label='Background')
plt.xlabel('Time of flight (s)')
plt.ylabel('Normalized signal')
plt.title('Electrons time of flight')
plt.legend()
plt.show()

In [ ]:
xaxis = np.arange(14080)*channel_time

avg_etof1 = -np.mean(etof1, axis=0)
max_etof1 = max(avg_etof1)
avg_etof2 = -np.mean(etof2, axis=0)
max_etof2 = max(avg_etof2)
avg_etof3 = -np.mean(etof3, axis=0)
max_etof3 = max(avg_etof3)
avg_backgrd_etof = -np.mean(backgrd_etof, axis=0)
max_backgrd_etof = max(avg_backgrd_etof)

plt.figure()
plt.plot(xaxis, avg_etof1, label='1000-3000 events')
plt.plot(xaxis, avg_etof2, label='5000-8000 events')
plt.plot(xaxis, avg_etof3, label='10000-15000 events')
plt.plot(xaxis, avg_backgrd_etof/max_backgrd_etof, label='Background')
plt.xlabel('Time of flight (s)')
plt.ylabel('Normalized signal')
plt.title('Electrons time of flight')
plt.legend()
plt.show()

In [ ]:
max_time = 6e-7
max_coord = int(max_time/channel_time)

avg_etof1 = avg_etof1[:max_coord]
max_etof1 = max(avg_etof1)
avg_etof2 = avg_etof2[:max_coord]
max_etof2 = max(avg_etof2)
avg_etof3 = avg_etof3[:max_coord]
max_etof3 = max(avg_etof3)
avg_backgrd_etof = avg_backgrd_etof[:max_coord]
max_backgrd_etof = max(avg_backgrd_etof)

xaxis = np.arange(max_coord)*channel_time

plt.figure()
plt.plot(xaxis, avg_etof1, label='1000-3000 events')
plt.plot(xaxis, avg_etof2, label='5000-8000 events')
plt.plot(xaxis, avg_etof3, label='10000-15000 events')
plt.plot(xaxis, avg_backgrd_etof/max_backgrd_etof, label='Background')
plt.xlabel('Time of flight (s)')
plt.ylabel('Signal')
plt.title('Electrons time of flight')
plt.legend()
plt.show()

In [ ]:
bgrd_subs1 = np.abs(avg_etof1-avg_backgrd_etof)/np.abs(avg_backgrd_etof)
bgrd_subs2 = np.abs(avg_etof2-avg_backgrd_etof)/np.abs(avg_backgrd_etof)
bgrd_subs3 = np.abs(avg_etof3-avg_backgrd_etof)/np.abs(avg_backgrd_etof)

plt.figure()
plt.plot(xaxis, bgrd_subs1, label='1000-3000 events')
plt.plot(xaxis, bgrd_subs2, label='5000-8000 events')
plt.plot(xaxis, bgrd_subs3, label='10000-15000 events')
plt.xlabel('Time of flight (s)')
plt.ylabel('Signal')
plt.title('Background substracted electrons time of flight')
plt.legend()
plt.ylim(0,5e5)
plt.show()

In [ ]:
xlimit = 1.5e-7

bounded_dfevent1 = dfevent1[dfevent1.tof < xlimit]
hist1, bin_edges = np.histogram(bounded_dfevent1.tof, bins=1000)

bounded_dfevent2 = dfevent2[dfevent2.tof < xlimit]
hist2, bin_edges = np.histogram(bounded_dfevent2.tof, bins=1000)

bounded_dfevent3 = dfevent3[dfevent3.tof < xlimit]
hist3, bin_edges = np.histogram(bounded_dfevent3.tof, bins=1000)

backgrd_bounded_dfevent = backgrd_dfevent[backgrd_dfevent.tof < xlimit]
backgrd_hist, bin_edges = np.histogram(backgrd_bounded_dfevent.tof, bins=1000)
    
plt.figure()
plt.plot(bin_edges[:-1], hist1, label='1000-3000 events')
plt.plot(bin_edges[:-1], hist2, label='5000-8000 events')
plt.plot(bin_edges[:-1], hist3, label='10000-15000 events')
plt.plot(bin_edges[:-1], backgrd_hist, label='Background')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of hits per bin')
plt.title('Ions time of flight')
plt.legend()
plt.show()   

In [ ]:
max_time = 2e-7

max_coord = int(max_time/channel_time)
etof_sums = selected_etof.isel(data=slice(None, max_coord)).sum(dim='data')

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(selected_dfpulse.nevents_pulse,-etof_sums)
plt.xlabel('Number of ions')
plt.ylabel('Number of electrons')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(selected_dfpulse.nevents_pulse,-etof_sums, s=1)
plt.xlabel('Number of ions')
plt.ylabel('Number of electrons')
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(selected_dfpulse.nevents_pulse,-etof_sums)
plt.xlabel('Number of ions')
plt.ylabel('Number of electrons')
plt.xscale('log')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(selected_dfpulse.nevents_pulse,-etof_sums, s=1)
plt.xlabel('Number of ions')
plt.ylabel('Number of electrons')
plt.yscale('log')
plt.show()

## Pulse filtering

In [ ]:
ion_dfpulse['pulseId'] = ion_dfpulse['pulseId'].astype(str)
ion_dfpulse['last_two_digits'] = ion_dfpulse['pulseId'].str[-2:]
result_df = ion_dfpulse.groupby('last_two_digits')['nevents_pulse'].mean().reset_index()

plt.figure()
plt.plot(result_df['last_two_digits'], result_df['nevents_pulse'], marker='o')
plt.xlabel('Last Two Digits of pulseId')
plt.ylabel('Average nevents_pulse')
plt.title('Average nevents_pulse for each Last Two Digits of pulseId')
plt.show()

In [ ]:
bounded_dfevent = ion_dfevent1[ion_dfevent1.tof < TIME_BETWEEN_PULSES]
hist, bin_edges = np.histogram(bounded_dfevent.tof, bins=1000)
    
plt.figure()
plt.plot(bin_edges[:-1], hist/max(hist))
plt.xlabel('Time of flight (s)')
plt.ylabel('Normalized signal')
plt.title('Ions time of flight')
plt.xlim(0,1e-6)
plt.show()

In [ ]:
plt.figure()
plt.plot(bin_edges[:-1], hist/max(hist))
plt.xlabel('Time of flight (s)')
plt.ylabel('Normalized signal')
plt.title('Ions time of flight')
plt.xlim(0,2e-6)
plt.show()

In [ ]:
event_numbers=['750-1500','3000-8000','10000-14000']
k=0
for i_df in [ion_dfevent1,ion_dfevent2,ion_dfevent3]:
    print('event_numbers',event_numbers[k])
    k+=1
    for pulse_number in [None,'01','46','92']:
        print('pulse number',pulse_number)
        pulse_filtered_ion_tof(i_df,pulse_number,5e-7)
print('background 20-40')
pulse_filtered_ion_tof(backgrd_ion_dfevent,None,5e-7)
print('argon')
pulse_filtered_ion_tof(argon_dfevent,None,5e-7)

## Calibration

In [ ]:
X = 127
Y = 113
WIDTH = 13
HEIGHT = 10
ZONE = [X,Y,WIDTH,HEIGHT,0]

In [ ]:
### To test where your square selection is, use:
heatmap_with_zones(ion_dfevent500,[ZONE])

In [ ]:
spatial_bkgrd_dfevent,spatial_bkgrd_dfpulse = spatial_ion_selection(backgrd_ion_dfevent500,backgrd_ion_dfpulse500,[ZONE])

In [ ]:
%matplotlib widget
big_ion_tof(spatial_bkgrd_dfevent)

In [ ]:
#CALIBRATION_LINES 500nm 7594V run390 = [1.312e-6, 9.26e-7, 7.56e-7, 6.65e-7, 5.97e-7]
# CALIBRATION_LINES 500/300nm 5400V = [1.556e-6, 1.103e-6, 9e-7, 7.83e-7, 7e-7]

In [ ]:
%matplotlib inline
calibrate(spatial_bkgrd_dfevent)

In [ ]:
# a_fit, b_fit = (29497236325759.32, 2.0175071848701878) #500nm 7594V run390, alcohol
a_fit, b_fit = (19788855827929.47, 2.013491982616985) #500nm 5400V all runs

In [ ]:
calibrated_selected_dfevent, calibrated_backgrd_dfevent = apply_calibration([selected_dfevent,backgrd_dfevent],a_fit,b_fit)

In [ ]:
calibrated_selected_dfevent1, calibrated_backgrd_dfevent = apply_calibration([selected_dfevent1,backgrd_dfevent],a_fit,b_fit)
calibrated_selected_dfevent2, calibrated_backgrd_dfevent = apply_calibration([selected_dfevent2,backgrd_dfevent],a_fit,b_fit)
calibrated_selected_dfevent3, calibrated_backgrd_dfevent = apply_calibration([selected_dfevent3,backgrd_dfevent],a_fit,b_fit)
calibrated_selected_dfevent4, calibrated_backgrd_dfevent = apply_calibration([selected_dfevent4,backgrd_dfevent],a_fit,b_fit)
calibrated_selected_dfevent5, calibrated_backgrd_dfevent = apply_calibration([selected_dfevent5,backgrd_dfevent],a_fit,b_fit)
calibrated_selected_dfevent6, calibrated_backgrd_dfevent = apply_calibration([selected_dfevent6,backgrd_dfevent],a_fit,b_fit)

In [ ]:
ion_dfevent500, backgrd_ion_dfevent500 = apply_calibration([ion_dfevent500,backgrd_ion_dfevent500],a_fit,b_fit)
ion_dfevent300, backgrd_ion_dfevent300 = apply_calibration([ion_dfevent300,backgrd_ion_dfevent300],a_fit,b_fit)
# ion_dfevent100, backgrd_ion_dfevent = apply_calibration([ion_dfevent100,backgrd_ion_dfevent],a_fit,b_fit)

In [ ]:
lowcalibrated_ion_dfevent, lowcalibrated_backgrd_ion_dfevent = apply_calibration([lowion_dfevent300,backgrd_ion_dfevent300],a_fit,b_fit)

In [ ]:
calibrated_ion_dfevent1, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent1,backgrd_ion_dfevent300],a_fit,b_fit)
calibrated_ion_dfevent2, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent2,backgrd_ion_dfevent300],a_fit,b_fit)
calibrated_ion_dfevent3, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent3,backgrd_ion_dfevent300],a_fit,b_fit)

In [ ]:
calibrated_ion_dfevent1, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent1,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent2, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent2,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent3, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent3,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent4, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent4,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent5, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent5,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent6, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent6,backgrd_ion_dfevent],a_fit,b_fit)

In [ ]:
calibrated_ion_dfevent10, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent10,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent05, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent05,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent025, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent025,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent012, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent012,backgrd_ion_dfevent],a_fit,b_fit)

In [ ]:
calibrated_ion_dfevent10_1, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent10_1,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent10_2, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent10_2,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent10_3, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent10_3,backgrd_ion_dfevent],a_fit,b_fit)

calibrated_ion_dfevent05_1, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent05_1,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent05_2, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent05_2,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent05_3, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent05_3,backgrd_ion_dfevent],a_fit,b_fit)

calibrated_ion_dfevent025_1, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent025_1,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent025_2, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent025_2,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent025_3, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent025_3,backgrd_ion_dfevent],a_fit,b_fit)

calibrated_ion_dfevent012_1, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent012_1,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent012_2, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent012_2,backgrd_ion_dfevent],a_fit,b_fit)
calibrated_ion_dfevent012_3, calibrated_backgrd_dfevent = apply_calibration([ion_dfevent012_3,backgrd_ion_dfevent],a_fit,b_fit)

In [ ]:
calibrated_selected_dfevent = ion_dfevent500
calibrated_backgrd_dfevent = backgrd_ion_dfevent500

In [ ]:
%matplotlib widget
MQ_LINES = [40,20,40/3,40/4,40/5]

plt.figure(figsize=(20, 10))
histselected, bin_edgesselected = np.histogram(calibrated_selected_dfevent.mq, bins=np.linspace(0,200,1000),range=(0,200))
histgrd, bin_edgesgrd = np.histogram(calibrated_backgrd_dfevent.mq, bins=np.linspace(0,200,1000),range=(0,200))
plt.plot(bin_edgesselected[:-1], histselected/max(histselected), linewidth = 1, c='b')
plt.plot(bin_edgesgrd[:-1], histgrd/max(histgrd), linewidth = 1, c='g')
plt.vlines(MQ_LINES,0,1,colors='black')
plt.show()

In [ ]:
%matplotlib widget
big_mq_plot(calibrated_selected_dfevent,2500,(0,150))

### Heatmaps / Etofs

In [ ]:
%matplotlib inline
LOWER_MQ = .25
UPPER_MQ = 3

mqselected_dfevent,mqselected_dfpulse,mqselected_etof = mq_selection(calibrated_selected_dfevent,selected_dfpulse,selected_etof,LOWER_MQ,UPPER_MQ)
mqselected_backgrd_dfevent,mqselected_backgrd_dfpulse,mqselected_backgrd_etof = mq_selection(calibrated_backgrd_dfevent,backgrd_dfpulse,backgrd_etof,LOWER_MQ,UPPER_MQ)

heatmap(mqselected_dfevent)
heatmap(mqselected_backgrd_dfevent)

TIME_BETWEEN_PULSES = 3.54462e-6
CHANNELS_PER_PULSE = 14080
channel_time = TIME_BETWEEN_PULSES/CHANNELS_PER_PULSE
    
xaxis = np.arange(14080)*channel_time
avg_mqselected_etof = -np.mean(mqselected_etof, axis=0)
avg_mqbackgrd_etof = -np.mean(mqselected_backgrd_etof, axis=0)
max_mqselected_etof = max(avg_mqselected_etof)
max_mqbackgrd_etof = max(avg_mqbackgrd_etof)

plt.figure()
plt.plot(xaxis,avg_mqselected_etof/max_mqselected_etof,c='r')
plt.plot(xaxis,avg_mqbackgrd_etof/max_mqbackgrd_etof,c='g')
plt.xlabel('Time of flight (s)')
plt.ylabel('Normalized signal')
plt.title('Electrons time of flight')
plt.show()

In [ ]:
LOWER_MQ = 20.5
UPPER_MQ = 21.5

mqselected_dfevent,mqselected_dfpulse,mqselected_etof = mq_selection(calibrated_selected_dfevent,selected_dfpulse,selected_etof,LOWER_MQ,UPPER_MQ)
heatmap(mqselected_dfevent)
e_tof(mqselected_etof)

In [ ]:
mqselected_dfevent1,mqselected_dfpulse1,mqselected_etof1 = mq_selection(calibrated_selected_dfevent1,selected_dfpulse1,selected_etof1,LOWER_MQ,UPPER_MQ)

avg_mqselected_etof = -np.mean(mqselected_etof, axis=0)
avg_mqselected_etof1 = -np.mean(mqselected_etof1, axis=0)
max_mqselected_etof = max(avg_mqselected_etof)
max_mqselected_etof1 = max(avg_mqselected_etof1)

xaxis = np.arange(14080)*channel_time
plt.figure()
plt.plot(xaxis,avg_mqselected_etof/max_mqselected_etof,c='r')
plt.plot(xaxis,avg_mqselected_etof1/max_mqselected_etof1,c='b')
plt.xlabel('Time of flight (s)')
plt.ylabel('Normalized signal')
plt.title('Electrons time of flight')
plt.show()

## Fish plots

In [ ]:
# Define a list of tuples where each tuple represents a tilted zone (x, y, width, height, angle)
ZONES = [(0, 98, 256, 10, 7), (145, 0, 10, 256, 7)]  # Add the angle of rotation in degrees
spatial_selected_dfevent,spatial_selected_dfpulse,spatial_selected_etof = tilted_spatial_ion_selection(selected_dfevent, selected_dfpulse, selected_etof, ZONES)

In [ ]:
slow_calibrated_ion_dfevent1 = calibrated_ion_dfevent1[calibrated_ion_dfevent1.tof < 5e-7]
slow_calibrated_ion_dfevent2 = calibrated_ion_dfevent2[calibrated_ion_dfevent2.tof < 5e-7]
slow_calibrated_ion_dfevent3 = calibrated_ion_dfevent3[calibrated_ion_dfevent3.tof < 5e-7]
slow_calibrated_ion_dfevent = calibrated_ion_dfevent[calibrated_ion_dfevent.tof < 5e-7]
slow_lowcalibrated_ion_dfevent = lowcalibrated_ion_dfevent[lowcalibrated_ion_dfevent.tof < 5e-7]

In [ ]:
X_ZONE = (0, 98, 256, 10, 7)
Y_ZONE = (145, 0, 10, 256, 7)
TOF_BINS = 1000
MQ_BINS = 500

fish_plot_x(ion_dfevent,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent,Y_ZONE)

In [ ]:
X_ZONE = (0, 98, 256, 10, 7)
Y_ZONE = (145, 0, 10, 256, 7)
TOF_BINS = 1000
MQ_BINS = 500

fish_plot_x(slow_calibrated_ion_dfevent1,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(slow_calibrated_ion_dfevent1,Y_ZONE)

In [ ]:
X_ZONE = (0, 98, 256, 10, 7)
Y_ZONE = (145, 0, 10, 256, 7)
TOF_BINS = 1000
MQ_BINS = 500

fish_plot_x(slow_calibrated_ion_dfevent2,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(slow_calibrated_ion_dfevent2,Y_ZONE)

In [ ]:
X_ZONE = (0, 98, 256, 10, 7)
Y_ZONE = (145, 0, 10, 256, 7)
TOF_BINS = 1000
MQ_BINS = 500

fish_plot_x(calibrated_ion_dfevent2,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(calibrated_ion_dfevent2,Y_ZONE)

In [ ]:
X_ZONE = (0, 98, 256, 10, 7)
Y_ZONE = (145, 0, 10, 256, 7)
TOF_BINS = 1000
MQ_BINS = 500

fish_plot_x(calibrated_ion_dfevent3,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(calibrated_ion_dfevent3,Y_ZONE)

In [ ]:
X_ZONE = (0, 98, 256, 10, 7)
Y_ZONE = (145, 0, 10, 256, 7)
TOF_BINS = 1000
MQ_BINS = 500

fish_plot_x(calibrated_ion_dfevent,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(calibrated_ion_dfevent,Y_ZONE)

In [ ]:
X_ZONE = (0, 98, 256, 10, 7)
Y_ZONE = (145, 0, 10, 256, 7)
TOF_BINS = 1000
MQ_BINS = 500

fish_plot_x(ion_dfevent10_1,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent10_1,Y_ZONE,TOF_BINS,MQ_BINS)
fish_plot_x(ion_dfevent10_2,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent10_2,Y_ZONE,TOF_BINS,MQ_BINS)
fish_plot_x(ion_dfevent10_3,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent10_3,Y_ZONE,TOF_BINS,MQ_BINS)

fish_plot_x(ion_dfevent05_1,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent05_1,Y_ZONE,TOF_BINS,MQ_BINS)
fish_plot_x(ion_dfevent05_2,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent05_2,Y_ZONE,TOF_BINS,MQ_BINS)
fish_plot_x(ion_dfevent05_3,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent05_3,Y_ZONE,TOF_BINS,MQ_BINS)

fish_plot_x(ion_dfevent025_1,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent025_1,Y_ZONE,TOF_BINS,MQ_BINS)
fish_plot_x(ion_dfevent025_2,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent025_2,Y_ZONE,TOF_BINS,MQ_BINS)
fish_plot_x(ion_dfevent025_3,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent025_3,Y_ZONE,TOF_BINS,MQ_BINS)

fish_plot_x(ion_dfevent012_1,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent012_1,Y_ZONE,TOF_BINS,MQ_BINS)
fish_plot_x(ion_dfevent012_2,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent012_2,Y_ZONE,TOF_BINS,MQ_BINS)
fish_plot_x(ion_dfevent012_3,X_ZONE,TOF_BINS,MQ_BINS)
fish_plot_y(ion_dfevent012_3,Y_ZONE,TOF_BINS,MQ_BINS)

## Intensity dependent

### tofs

In [ ]:
NBINS_EVENTS = 6
NBINS_MQ = 1500

filtered_dfs = nevents_binning_plot(calibrated_selected_dfevent,selected_dfpulse,NBINS_EVENTS,NBINS_MQ)

In [ ]:
NBINS_EVENTS = 6
NBINS_TOF = 1500

filtered_dfs = nevents_binning_plot_tof(ion_dfevent500,ion_dfpulse500,NBINS_EVENTS,NBINS_TOF)

In [ ]:
NBINS_TOF = 1500
NBINS_EVENTS = 6

short_filtered_dfevents, short_filtered_dfpulses, short_hists, short_bins = nions_binning_tof(ion_dfevent,ion_dfpulse,NBINS_EVENTS,NBINS_TOF)

In [ ]:
#Normalized individually to maximum in specified region
stacked_ion_tof_max(short_hists,NBINS_TOF,short_bins,(0,4e-7),backgrd_ion_dfevent500)

In [ ]:
stacked_ion_tof_max(short_hists,NBINS_TOF,short_bins,(.9e-6,1.3e-6),backgrd_ion_dfevent500)

In [ ]:
#Normalized individually to number of hits in specified region
stacked_ion_tof_sum(short_hists,NBINS_TOF,short_bins,(0,4e-7),backgrd_ion_dfevent500)

In [ ]:
stacked_ion_tof_sum(short_hists,NBINS_TOF,short_bins,(.9e-6,1.3e-6),backgrd_ion_dfevent500)

### waterfalls

In [ ]:
NBINS_MQ = 1500
NBINS_EVENTS = 20

filtered_dfevents, filtered_dfpulses, filtered_etofs, hists, bins = nevents_binning(calibrated_selected_dfevent,selected_dfpulse,selected_etof,NBINS_EVENTS,NBINS_MQ)
#filtered_dfevents, filtered_dfpulses, hists, bins = nions_binning(calibrated_ion_dfevent,ion_dfpulse,NBINS_EVENTS,NBINS_MQ)

In [ ]:
waterfall_rel(hists,NBINS_MQ,(0.5,8))

In [ ]:
waterfall_abs(hists,NBINS_MQ,(0,150))

In [ ]:
waterfall_etof(filtered_etofs)

### heatmaps

#### general

In [ ]:
NBINS_TOF = 1500
NBINS_EVENTS = 50

filtered_dfevents1, filtered_dfpulses1, hists1, bins = nions_binning_tof(ion_dfevent1,ion_dfpulse1,NBINS_EVENTS,NBINS_TOF)
filtered_dfevents05, filtered_dfpulses05, hists05, bins = nions_binning_tof(ion_dfevent05,ion_dfpulse05,NBINS_EVENTS,NBINS_TOF)
filtered_dfevents025, filtered_dfpulses025, hists025, bins = nions_binning_tof(ion_dfevent025,ion_dfpulse025,NBINS_EVENTS,NBINS_TOF)
filtered_dfevents012, filtered_dfpulses012, hists012, bins = nions_binning_tof(ion_dfevent012,ion_dfpulse012,NBINS_EVENTS,NBINS_TOF)

In [ ]:
tof = np.linspace(0,TIME_BETWEEN_PULSES,NBINS_TOF)
np.savetxt('heatmap_tof.csv', tof, delimiter=',')
np.savetxt('heatmap_nevents.csv', bins, delimiter=',')

np.savetxt('1heatmap_matrix.csv', hists1, delimiter=',')
np.savetxt('05heatmap_matrix.csv', hists05, delimiter=',')
np.savetxt('025heatmap_matrix.csv', hists025, delimiter=',')
np.savetxt('012heatmap_matrix.csv', hists012, delimiter=',')

In [ ]:
NBINS_TOF = 4000
NBINS_EVENTS = 80

filtered_dfevents, filtered_dfpulses, hists, bins = nions_binning_tof(ion_dfevent,ion_dfpulse,NBINS_EVENTS,NBINS_TOF)

In [ ]:
NBINS_TOF = 3000
NBINS_EVENTS = 30

filtered_maxdfevents500, filtered_maxdfpulses500, maxhists500, bins = nions_binning_tof(ion_dfevent_max,ion_dfpulse_max,NBINS_EVENTS,NBINS_TOF)

In [ ]:
NBINS_TOF = 3000
NBINS_EVENTS = 30

filtered_lowermaxdfevents500, filtered_lowermaxdfpulses500, lowermaxhists500, bins = nions_binning_tof(ion_dfevent_lower_max,ion_dfpulse_lower_max,NBINS_EVENTS,NBINS_TOF)

In [ ]:
NBINS_TOF = 3000
NBINS_EVENTS = 40

filtered_dfevents500, filtered_dfpulses500, hists500, bins = nions_binning_tof(ion_dfevent500,ion_dfpulse500,NBINS_EVENTS,NBINS_TOF)

In [ ]:
NBINS_TOF = 4000
NBINS_EVENTS = 70

filtered_dfevents500, filtered_dfpulses500, hists500detail, binsdetail = nions_binning_tof(ion_dfevent500,ion_dfpulse500,NBINS_EVENTS,NBINS_TOF)

In [ ]:
NBINS_TOF = 3000
NBINS_EVENTS = 30

filtered_dfevents100, filtered_dfpulses100, hists100, bins100 = nions_binning_tof(ion_dfevent100,ion_dfpulse100,NBINS_EVENTS,NBINS_TOF)

In [ ]:
NBINS_MQ = 3000
NBINS_EVENTS = 40

mq_dfevents500, mq_dfpulses500, mq_hists500, mq_bins = nions_binning(ion_dfevent500,ion_dfpulse500,NBINS_EVENTS,NBINS_MQ)

In [ ]:
nevents_heatmap_rel(mq_hists500,NBINS_MQ,mq_bins,(0,5))

In [ ]:
nevents_heatmap_rel_tof(hists500,NBINS_TOF,bins,(1e-7,5e-7))

In [ ]:
hists = hists500
nbins_tof = 3000
xlimits = (1e-7,5e-7)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins[:-1])

In [ ]:
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', vmax=.3)
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', cmap='viridis', norm=LogNorm())
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of events slice')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)
# plt.axvline(2.37e-7,c='r')
# plt.axvline(3.34e-7,c='r')
plt.axvline(3.7e-7,c='r')
plt.axvline(4.1e-7,c='r')
plt.axvline(4.5e-7,c='r')
# plt.axhline(y=2000, c='red')
plt.show()

In [ ]:
hists = hists500detail
nbins_tof = NBINS_TOF
xlimits = (1e-7,5e-7)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, binsdetail[:-1])

In [ ]:
plt.figure(figsize=(20, 8))
# c = plt.pcolormesh(X, Y, hists_norm, cmap='plasma', shading='auto')
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', vmax=0.7)
c = plt.pcolormesh(X, Y, hists_norm, shading='auto', cmap='plasma', norm=LogNorm(vmin=.1, vmax=1))
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of events slice')
# plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)
# plt.axvline(2.37e-7,c='r')
# plt.axvline(3.34e-7,c='r')
plt.axvline(3.7e-7,c='r')
plt.axvline(4.1e-7,c='r')
plt.axvline(4.5e-7,c='r')
# plt.axhline(y=2000, c='red')
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto', cmap='plasma')
colorbar = plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (ns)', fontsize=24)
plt.ylabel('Hit intensity', fontsize=24)
# plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(1.1e-7, 3.4e-7)
plt.ylim(0,6800)

ax1 = plt.gca()
ax1.tick_params(axis='x', labelsize=22)
ax1.tick_params(axis='y', labelsize=22)
colorbar.ax.set_ylabel('Relative normalized counts', fontsize=24)
colorbar.ax.tick_params(labelsize=22)

bottom_tick_positions = ax1.get_xticks()[1:-1]
bottom_tick_labels = [f'{tick * 1e9:.0f}' for tick in bottom_tick_positions]

ax1.set_xticks(bottom_tick_positions)
ax1.set_xticklabels(bottom_tick_labels, fontsize=22)  # Adjust fontsize


plt.axvline(2.2e-7,c='b',linewidth=4)
plt.axvline(1.965e-7,c='b',linewidth=4)

plt.show()

In [ ]:
X_ns = X * 1e9

In [ ]:
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X_ns, Y, hists_norm, cmap='magma', shading='auto')
colorbar = plt.colorbar(c, label='Normalized counts (arb. units)', extend='max')
plt.xlabel('Time of flight (ns)', fontsize=20)
plt.ylabel('Hit intensity (arb. unit)', fontsize=20)
plt.xlim(120, 490)
plt.ylim(501,4900)
ax1 = plt.gca()
ax1.tick_params(axis='x', labelsize=16)
ax1.tick_params(axis='y', labelsize=16)
colorbar.ax.set_ylabel('Normalized counts (arb. units)', fontsize=20)
colorbar.ax.tick_params(labelsize=16)

plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, cmap='plasma', shading='auto')

from matplotlib.colors import Normalize

vmin = np.percentile(hists_norm, 20)
vmax = np.percentile(hists_norm, 99)
norm = Normalize(vmin=vmin, vmax=vmax)
c = plt.pcolormesh(X, Y, hists_norm, shading='auto', norm=norm, cmap='plasma')

# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', vmax=0.7)
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', cmap='plasma', norm=LogNorm(vmin=.1, vmax=1))
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of events slice')
# plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)
# plt.axvline(2.37e-7,c='r')
# plt.axvline(3.34e-7,c='r')
plt.axvline(3.7e-7,c='r')
plt.axvline(4.1e-7,c='r')
plt.axvline(4.5e-7,c='r')
# plt.axhline(y=2000, c='red')
plt.show()

In [ ]:
intensities = [500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500]

# Create a new figure for the 1D projections
plt.figure(figsize=(12, 8))

# Plot 1D projections for each intensity
for intensity in intensities:
    # Find the index closest to the desired intensity
    idx = np.argmin(np.abs(bins[:-1] - intensity))
    
    # Extract the 1D projection for this intensity
    projection = hists_norm[idx]
    
    # Plot the projection
    plt.plot(x_edges, projection, label=f'Intensity {intensity}')

# Customize the plot
plt.xlabel('Time of flight (s)')
plt.ylabel('Relative normalized counts')
plt.title('1D Projections for Different Intensities')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

# Add vertical lines as in the original plot
plt.axvline(3.7e-7, c='r', linestyle='--', label='Vertical lines')
plt.axvline(4.1e-7, c='r', linestyle='--')
plt.axvline(4.5e-7, c='r', linestyle='--')

plt.xlim(x_lower, x_upper)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# Define parameters
x_lower, x_upper = 3.6e-7, 4.5e-7
intensities = [1000, 1250, 1500, 1750, 2000, 2250, 2500]

# Extract and plot projections
peak_times = []
for intensity in intensities:
    idx = np.argmin(np.abs(bins[:-1] - intensity))
    projection = hists_norm[idx]
    
    # Get time indices for the region of interest
    time_mask = (x_edges >= x_lower) & (x_edges <= x_upper)
    times = x_edges[time_mask]
    proj_masked = projection[time_mask]
    
    plt.plot(times, proj_masked, label=f'Intensity {intensity}')
    
    peak_time = times[np.argmax(proj_masked)]
    peak_times.append(peak_time)

plt.xlabel('Time of flight (s)')
plt.ylabel('Relative normalized counts')
plt.title('1D Projections (3.2e-7 to 4.2e-7 s)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

peak_times = np.array(peak_times)
# Use only the higher intensity region where exponential behavior is clear
intensities = intensities[3:]
peak_times = peak_times[3:]

# Second plot: Exponential fitting
def exponential(x, a, b, c):
    return a * np.exp(b * (x - x[0])) + c  # Normalize x to improve fitting

# Provide initial guesses and bounds for better fitting
p0 = [1000, 1e7, 1000]  # Initial parameters
bounds = ([0, 0, 0], [10000, 1e8, 5000])  # Lower and upper bounds

popt, pcov = curve_fit(exponential, peak_times, intensities, p0=p0, bounds=bounds)

plt.figure(figsize=(10, 6))
plt.scatter(peak_times, intensities, color='blue', label='Peak positions')

# Plot fit line
x_fit = np.linspace(min(peak_times), max(peak_times), 100)
y_fit = exponential(x_fit, *popt)
plt.plot(x_fit, y_fit, 'r-', 
         label=f'Exponential fit\ny = {popt[0]:.2f}*exp({popt[1]:.2e}*(t-t0)) + {popt[2]:.2f}')

plt.xlabel('Time of flight (s)')
plt.ylabel('Intensity')
plt.title('Peak Position vs Intensity with Exponential Fit')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
hists = hists500
nbins_tof = NBINS_TOF
xlimits = (3e-7,7e-7)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins[:-1])

plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', vmax=.3)
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', cmap='viridis', norm=LogNorm())
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of events slice')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)
# plt.axvline(2.37e-7,c='r')
# plt.axvline(3.34e-7,c='r')
plt.axvline(4.6e-7,c='r')
plt.axvline(5.4e-7,c='r')
# plt.axhline(y=2000, c='red')
plt.show()

In [ ]:
hists = hists500
nbins_tof = NBINS_TOF
xlimits = (1e-7,5e-7)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins[:-1])

In [ ]:
hists = hists300
nbins_tof = NBINS_TOF
xlimits = (1e-7,5e-7)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins[:-1])

In [ ]:
hists = hists100
nbins_tof = NBINS_TOF
xlimits = (1e-7,5e-7)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins[:-1])

In [ ]:
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', vmax=.3)
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', cmap='viridis', norm=LogNorm())
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of events slice')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)
plt.axvline(1.9e-7,c='r')
plt.axvline(2.7e-7,c='r')
plt.axvline(3.7e-7,c='r')
plt.axvline(4.4e-7,c='r')
# plt.axhline(y=2000, c='red')
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', vmax=.3)
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', cmap='viridis', norm=LogNorm())
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of events slice')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)
plt.axvline(1.9e-7,c='r')
plt.axvline(2.7e-7,c='r')
plt.axvline(3.7e-7,c='r')
plt.axvline(4.4e-7,c='r')
# plt.axhline(y=2000, c='red')
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', vmax=.3)
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', cmap='viridis', norm=LogNorm())
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of events slice')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)
plt.axvline(1.9e-7,c='r')
plt.axvline(2.7e-7,c='r')
plt.axvline(3.7e-7,c='r')
plt.axvline(4.4e-7,c='r')
# plt.axhline(y=2000, c='red')
plt.show()

In [ ]:
nevents_heatmap_abs_tof(hists500,NBINS_TOF,bins,(1e-7,5e-7))

#### H ions rate of change

In [ ]:
nevents_bins = 30
start_time = 1.9e-7
end_time = 2.7e-7

ion_dfpulse500['bins'] = pd.cut(ion_dfpulse500['nevents_pulse'], bins=nevents_bins, include_lowest=True)
pulseIds_by_bin = ion_dfpulse500.groupby('bins')['pulseId'].apply(set).reset_index()

mask = (ion_dfevent500['tof'] >= start_time) & (ion_dfevent500['tof'] <= end_time)
filtered_dfevent = ion_dfevent500[mask]

counts = []
for pulse_ids in pulseIds_by_bin['pulseId']:
    count = filtered_dfevent['pulseId'].isin(pulse_ids).sum()
    counts.append(count)

rate_of_change_H = [counts[i] - counts[i-1] for i in range(1, len(counts))]

In [ ]:
nevents_bins = 30
start_time = 2.7e-7
end_time = 3.7e-7

ion_dfpulse500['bins'] = pd.cut(ion_dfpulse500['nevents_pulse'], bins=nevents_bins, include_lowest=True)
pulseIds_by_bin = ion_dfpulse500.groupby('bins')['pulseId'].apply(set).reset_index()

mask = (ion_dfevent500['tof'] >= start_time) & (ion_dfevent500['tof'] <= end_time)
filtered_dfevent = ion_dfevent500[mask]

counts = []
for pulse_ids in pulseIds_by_bin['pulseId']:
    count = filtered_dfevent['pulseId'].isin(pulse_ids).sum()
    counts.append(count)

rate_of_change_H2 = [counts[i] - counts[i-1] for i in range(1, len(counts))]

In [ ]:
nevents_bins = 30
start_time = 3.7e-7
end_time = 4.4e-7

ion_dfpulse500['bins'] = pd.cut(ion_dfpulse500['nevents_pulse'], bins=nevents_bins, include_lowest=True)
pulseIds_by_bin = ion_dfpulse500.groupby('bins')['pulseId'].apply(set).reset_index()

mask = (ion_dfevent500['tof'] >= start_time) & (ion_dfevent500['tof'] <= end_time)
filtered_dfevent = ion_dfevent500[mask]

counts = []
for pulse_ids in pulseIds_by_bin['pulseId']:
    count = filtered_dfevent['pulseId'].isin(pulse_ids).sum()
    counts.append(count)

rate_of_change_H3 = [counts[i] - counts[i-1] for i in range(1, len(counts))]

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(bins[1:-1], rate_of_change_H, label='H+')
plt.scatter(bins[1:-1], rate_of_change_H2, label='H2+')
plt.scatter(bins[1:-1], rate_of_change_H3, label='H3+')
plt.xlabel('Total number of hits')
plt.ylabel('Rate of change')
plt.title('Hydrogen ions rate of change in terms of total intensity')
plt.grid(True)
plt.legend()
plt.show()

#### H3+ intensity dependance

In [ ]:
nevents_bins = 30
start_time = 3.7e-7
end_time = 4.5e-7

ion_dfpulse500['bins'] = pd.cut(ion_dfpulse500['nevents_pulse'], bins=nevents_bins, include_lowest=True)
pulseIds_by_bin = ion_dfpulse500.groupby('bins')['pulseId'].apply(set).reset_index()

mask = (ion_dfevent500['tof'] >= start_time) & (ion_dfevent500['tof'] <= end_time)
filtered_dfevent = ion_dfevent500[mask]

counts = []
for pulse_ids in pulseIds_by_bin['pulseId']:
    count = filtered_dfevent['pulseId'].isin(pulse_ids).sum()
    counts.append(count)

lower_bounds = [int(interval.left) for interval in pulseIds_by_bin['bins']]

In [ ]:
nevents_bins = 30
start_time = 4.6e-7
end_time = 5.4e-7

ion_dfpulse500['bins'] = pd.cut(ion_dfpulse500['nevents_pulse'], bins=nevents_bins, include_lowest=True)
pulseIds_by_bin = ion_dfpulse500.groupby('bins')['pulseId'].apply(set).reset_index()

mask = (ion_dfevent500['tof'] >= start_time) & (ion_dfevent500['tof'] <= end_time)
filtered_dfevent = ion_dfevent500[mask]

bkgrd_counts = []
for pulse_ids in pulseIds_by_bin['pulseId']:
    count = filtered_dfevent['pulseId'].isin(pulse_ids).sum()
    bkgrd_counts.append(count)

bkgrd_lower_bounds = [int(interval.left) for interval in pulseIds_by_bin['bins']]

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(lower_bounds, counts, marker='o', label='H3+')
plt.scatter(bkgrd_lower_bounds, bkgrd_counts, marker='x', label='background')
plt.xlabel('Total number of hits')
plt.ylabel('Number of hits in the swoop')
plt.title('H3+ swoop intensity in terms of total intensity')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
nevents_bins = 30
start_time = 3.7e-7
end_time = 4.5e-7

ion_dfpulse300['bins'] = pd.cut(ion_dfpulse300['nevents_pulse'], bins=nevents_bins, include_lowest=True)
pulseIds_by_bin = ion_dfpulse300.groupby('bins')['pulseId'].apply(set).reset_index()

mask = (ion_dfevent300['tof'] >= start_time) & (ion_dfevent300['tof'] <= end_time)
filtered_dfevent = ion_dfevent300[mask]

counts = []
for pulse_ids in pulseIds_by_bin['pulseId']:
    count = filtered_dfevent['pulseId'].isin(pulse_ids).sum()
    counts.append(count)

lower_bounds = [int(interval.left) for interval in pulseIds_by_bin['bins']]

In [ ]:
nevents_bins = 30
start_time = 4.6e-7
end_time = 5.4e-7

ion_dfpulse300['bins'] = pd.cut(ion_dfpulse300['nevents_pulse'], bins=nevents_bins, include_lowest=True)
pulseIds_by_bin = ion_dfpulse300.groupby('bins')['pulseId'].apply(set).reset_index()

mask = (ion_dfevent300['tof'] >= start_time) & (ion_dfevent300['tof'] <= end_time)
filtered_dfevent = ion_dfevent300[mask]

bkgrd_counts = []
for pulse_ids in pulseIds_by_bin['pulseId']:
    count = filtered_dfevent['pulseId'].isin(pulse_ids).sum()
    bkgrd_counts.append(count)

bkgrd_lower_bounds = [int(interval.left) for interval in pulseIds_by_bin['bins']]

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(lower_bounds, counts, marker='o', label='H3+')
plt.scatter(bkgrd_lower_bounds, bkgrd_counts, marker='x', label='background')
plt.xlabel('Total number of hits')
plt.ylabel('Number of hits in the swoop')
plt.title('H3+ swoop intensity in terms of total intensity')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
nevents_bins = 30
start_time = 3.7e-7
end_time = 4.5e-7

ion_dfpulse100['bins'] = pd.cut(ion_dfpulse100['nevents_pulse'], bins=nevents_bins, include_lowest=True)
pulseIds_by_bin = ion_dfpulse100.groupby('bins')['pulseId'].apply(set).reset_index()

mask = (ion_dfevent300['tof'] >= start_time) & (ion_dfevent300['tof'] <= end_time)
filtered_dfevent = ion_dfevent300[mask]

counts = []
for pulse_ids in pulseIds_by_bin['pulseId']:
    count = filtered_dfevent['pulseId'].isin(pulse_ids).sum()
    counts.append(count)

lower_bounds = [int(interval.left) for interval in pulseIds_by_bin['bins']]

In [ ]:
nevents_bins = 30
start_time = 4.6e-7
end_time = 5.4e-7

ion_dfpulse100['bins'] = pd.cut(ion_dfpulse100['nevents_pulse'], bins=nevents_bins, include_lowest=True)
pulseIds_by_bin = ion_dfpulse100.groupby('bins')['pulseId'].apply(set).reset_index()

mask = (ion_dfevent300['tof'] >= start_time) & (ion_dfevent300['tof'] <= end_time)
filtered_dfevent = ion_dfevent300[mask]

bkgrd_counts = []
for pulse_ids in pulseIds_by_bin['pulseId']:
    count = filtered_dfevent['pulseId'].isin(pulse_ids).sum()
    bkgrd_counts.append(count)

bkgrd_lower_bounds = [int(interval.left) for interval in pulseIds_by_bin['bins']]

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(lower_bounds, counts, marker='o', label='H3+')
plt.scatter(bkgrd_lower_bounds, bkgrd_counts, marker='x', label='background')
plt.xlabel('Total number of hits')
plt.ylabel('Number of hits in the swoop')
plt.title('H3+ swoop intensity in terms of total intensity')
plt.grid(True)
plt.legend()
plt.show()

#### Silicon species intensity dependance

In [ ]:
hists = hists500
nbins_tof = NBINS_TOF
xlimits = (1e-7,5e-7)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins[:-1])

In [ ]:
plt.figure(figsize=(20, 8))

ax1 = plt.gca()
ax1.set_facecolor('white') 

c = ax1.pcolormesh(X, Y, hists_norm, shading='auto', cmap='plasma')
colorbar = plt.colorbar(c, label='Relative normalized counts (a.u.)', extend='max')
plt.xlabel('Time of flight ($10^{-7}$ s)', fontsize=24)
plt.ylabel('Hit intensity', fontsize=24)
# plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)

fig.set_facecolor('white')

# ax2 = ax1.twiny()

# top_tick_positions = [1.09e-6,1.13e-6,1.17e-6,1.19e-6,1.22e-6,1.25e-6,1.3e-6]
# top_tick_labels = ['750', '500', '300', '200', '100', '50', '0']
# ax2.set_xticks(top_tick_positions)
# ax2.set_xticklabels(top_tick_labels)

# ax2.set_xlim(ax1.get_xlim())
# ax2.set_xlabel('Simulated kinetic energy (eV)', fontsize=24)

# ax2.spines['top'].set_color('blue')
# ax2.xaxis.label.set_color('blue')
# ax2.tick_params(axis='x', colors='blue', labelsize=22)
# plt.axvline(2.37e-7,c='r',linewidth=3)
# plt.axvline(3.34e-7,c='r',linewidth=3)
# plt.axvline(4.1e-7,c='r',linewidth=3)
ax1.tick_params(axis='x', labelsize=22)
ax1.tick_params(axis='y', labelsize=22)
colorbar.ax.set_ylabel('Relative normalized counts', fontsize=24)
colorbar.ax.tick_params(labelsize=22)

plt.show()

In [ ]:
hists = hists500[:50]
nbins_tof = NBINS_TOF
xlimits = (1.1e-6,1.4e-6)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins[:50])

plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', vmax=.3)
# c = plt.pcolormesh(X, Y, hists_norm, shading='auto', cmap='viridis', norm=LogNorm())
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of events slice')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)
plt.axhline(y=1500, c='red')
plt.axhline(y=2000, c='red')
plt.axhline(y=2600, c='red')
plt.show()

In [ ]:
hists = hists500
nbins_tof = NBINS_TOF
xlimits = (1e-7,5e-7)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins[:-1])

plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of events slice')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)
plt.axvline(2.156e-7,c='r')
plt.axvline(2.222e-7,c='r')
plt.axvline(2.299e-7,c='r')
plt.axvline(2.376e-7,c='r')
plt.axvline(2.424e-7,c='r')
plt.axvline(2.485e-7,c='r')
plt.axvline(2.528e-7,c='r')
plt.axvline(2.63e-7,c='r')
plt.show()

In [ ]:
current_ion_dfpulse = ion_dfpulse500[ion_dfpulse500.nevents_pulse > 500][ion_dfpulse500.nevents_pulse < 4500]
current_ion_dfevent = ion_dfevent500[ion_dfevent500.pulseId.isin(current_ion_dfpulse.pulseId)]

In [ ]:
NBINS_TOF = 3000
NBINS_EVENTS = 5

filtered_dfevents, filtered_dfpulses, hists, bins = nions_binning_tof(current_ion_dfevent,current_ion_dfpulse,NBINS_EVENTS,NBINS_TOF)

In [ ]:
%matplotlib widget

xlimits = (1.6e-6,1.9e-6)

nbins_tof = NBINS_TOF

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins[:-1])

plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of events slice')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)
plt.show()

In [ ]:
#250 bin size, 9 bins
weak_Si_nevents_bins = [750,1000,1250,1500,1750,2000,2250,2500,2750] #3000
weak_Si_lower_limit = [1.302e-6,1.293e-6,1.285e-6,1.262e-6,1.212e-6,1.188e-6,1.156e-6,1.145e-6,1.140e-6]
weak_Si_upper_limit = [1.323e-6,1.320e-6,1.315e-6,1.312e-6,1.296e-6,1.279e-6,1.243e-6,1.191e-6,1.164e-6]

#500 bin size, 10 bins
mid_Si_nevents_bins = [3000,3500,4000,4500,5000,5500,6000,6500,7000,7500] #8000
mid_Si_lower_limit = [1.136e-6,1.132e-6,1.129e-6,1.127e-6,1.126e-6,1.125e-6,1.124e-6,1.123e-6,1.122e-6,1.121e-6]
mid_Si_upper_limit = [1.156e-6,1.150e-6,1.148e-6,1.144e-6,1.141e-6,1.138e-6,1.137e-6,1.136e-6,1.135e-6,1.135e-6]

#1000 bin size, 7 bins
strong_Si_nevents_bins = [8000,9000,10000,11000,12000,13000,14000] #15000
strong_Si_lower_limit = [1.122e-6,1.121e-6,1.121e-6,1.123e-6,1.122e-6,1.122e-6,1.121e-6]
strong_Si_upper_limit = [1.138e-6,1.148e-6,1.142e-6,1.139e-6,1.135e-6,1.134e-6,1.134e-6]

In [ ]:
#250 bin size, 6 bins
weak_SiO_nevents_bins = [500,750,1000,1250,1500,1750] #2000
weak_SiO_lower_limit = [1.492e-6,1.488e-6,1.485e-6,1.484e-6,1.472e-6,1.472e-6]
weak_SiO_upper_limit = [1.582e-6,1.566e-6,1.566e-6,1.562e-6,1.551e-6,1.542e-6]

#no clear data from 2000 to 3000 events

missing_SiO_nevents_bins = [2000]
missing_SiO_lower_limit = []
missing_SiO_upper_limit = []

#500 bin size, 6 bins
mid_SiO_nevents_bins = [3500,4000,4500,5000,5500] #6000
mid_SiO_lower_limit = [1.223e-6,1.218e-6,1.217e-6,1.217e-6,1.217e-6,]
mid_SiO_upper_limit = [1.263e-6,1.260e-6,1.252e-6,1.246e-6,1.244e-6]

#1000 bin size, 9 bins
strong_SiO_nevents_bins = [6000,7000,8000,9000,10000,11000,12000,13000,14000] #15000
strong_SiO_lower_limit = [1.219e-6,1.217e-6,1.215e-6,1.219e-6,1.220e-6,1.219e-6,1.219e-6,1.216e-6,1.215e-6]
strong_SiO_upper_limit = [1.243e-6,1.244e-6,1.243e-6,1.243e-6,1.243e-6,1.242e-6,1.245e-6,1.244e-6,1.246e-6]

In [ ]:
#250 bin size, 4 bins
weak_Si2_nevents_bins = [500,750,1000,1250] #1500
weak_Si2_lower_limit = [1.695e-6,1.684e-6,1.677e-6,1.669e-6]
weak_Si2_upper_limit = [1.763e-6,1.747e-6,1.726e-6,1.723e-6]

#100 bin size, 10 bins
mid_Si2_nevents_bins = [1500,1600,1700,1800,1900,2000,2100,2200,2300,2400] #2500
mid_Si2_lower_limit = [1.663e-6,1.651e-6,1.654e-6,1.647e-6,1.629e-6,1.603e-6,1.583e-6,1.534e-6,1.520e-6,1.514e-6]
mid_Si2_upper_limit = [1.698e-6,1.695e-6,1.691e-6,1.686e-6,1.676e-6,1.665e-6,1.647e-6,1.622e-6,1.587e-6,1.564e-6]

#500 bin size, 4 bins
strong_Si2_nevents_bins = [2500,3000,3500,4000] #4500
strong_Si2_lower_limit = [1.508e-6,1.503e-6,1.505e-6,1.502e-6]
strong_Si2_upper_limit = [1.552e-6,1.543e-6,1.538e-6,1.538e-6]

In [ ]:
Si_nevents_bins = sum([weak_Si_nevents_bins, mid_Si_nevents_bins, strong_Si_nevents_bins,[15000]], [])
Si_lower_limit = sum([weak_Si_lower_limit, mid_Si_lower_limit, strong_Si_lower_limit], [])
Si_upper_limit = sum([weak_Si_upper_limit, mid_Si_upper_limit, strong_Si_upper_limit], [])

SiO_nevents_bins = sum([weak_SiO_nevents_bins, missing_SiO_nevents_bins, mid_SiO_nevents_bins, strong_SiO_nevents_bins,[15000]], [])
SiO_lower_limit = sum([weak_SiO_lower_limit, missing_SiO_lower_limit, mid_SiO_lower_limit, strong_SiO_lower_limit], [])
SiO_upper_limit = sum([weak_SiO_upper_limit, missing_SiO_upper_limit, mid_SiO_upper_limit, strong_SiO_upper_limit], [])

Si2_nevents_bins = sum([weak_Si2_nevents_bins, mid_Si2_nevents_bins, strong_Si2_nevents_bins,[4500]], [])
Si2_lower_limit = sum([weak_Si2_lower_limit, mid_Si2_lower_limit, strong_Si2_lower_limit], [])
Si2_upper_limit = sum([weak_Si2_upper_limit, mid_Si2_upper_limit, strong_Si2_upper_limit], [])

In [ ]:
lower_limit = Si2_lower_limit
upper_limit = Si2_upper_limit
nevents_bins = Si2_nevents_bins

In [ ]:
nevents_bin_edges = nevents_bins
dfpulse = current_ion_dfpulse
dfevent = current_ion_dfevent
nbins_tof = 3000


filtered_dfevents = []
filtered_dfpulses = []
hists = []

for i in range(len(nevents_bin_edges) - 1):

    start_edge = nevents_bin_edges[i]
    end_edge = nevents_bin_edges[i + 1]

    # Filtering the pulses within the current bin range
    filtered_dfpulse = dfpulse[(dfpulse.nevents_pulse >= start_edge) & (dfpulse.nevents_pulse < end_edge)]
    filtered_dfevent = dfevent[dfevent.pulseId.isin(filtered_dfpulse.pulseId)]

    filtered_dfevents.append(filtered_dfevent)
    filtered_dfpulses.append(filtered_dfpulse)
    
    # Histogram of tof with predefined bins
    hist, bin_edges = np.histogram(
        filtered_dfevent.tof, 
        bins=np.linspace(0, TIME_BETWEEN_PULSES, nbins_tof + 1),
        range=(0, TIME_BETWEEN_PULSES)
    )
    
    # Normalize the histogram by the number of pulses in the filtered_dfpulse
    if len(filtered_dfpulse) > 0:
        hists.append(hist / len(filtered_dfpulse))
    else:
        hists.append(np.zeros_like(hist))  # Handle case with no pulses

In [ ]:
%matplotlib inline

xlimits = (1.4e-6, 1.8e-6)

precision = TIME_BETWEEN_PULSES / nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower / precision), int(x_upper / precision)
nbins_events = len(hists)
hists_norm = []

for i in range(nbins_events):
    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)
    
x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)

# Generate meshgrid
X, Y = np.meshgrid(x_edges, nevents_bin_edges[:-1])

plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Number of events slice')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)

# Adjust y-ticks to be centered on the y-bins
y_tick_positions = nevents_bin_edges[:-1] - 0.5 * np.diff(nevents_bin_edges)
plt.yticks(ticks=y_tick_positions, labels=nevents_bin_edges[:-1])

y_tick_edges = np.append(y_tick_positions,nevents_bin_edges[-1]- 0.5 * (nevents_bin_edges[-1]-nevents_bin_edges[-2]))

# Plotting the vertical lines at the specified x-coordinates using the centered y-tick positions
for i, x_coord in enumerate(lower_limit):
    y_lower = y_tick_edges[i] # Start of the y-bin
    y_upper = y_tick_edges[i+1] # End of the y-bin
    plt.plot([x_coord, x_coord], [y_lower, y_upper], color='red', linestyle='-', linewidth=2)

for i, x_coord in enumerate(upper_limit):
    y_lower = y_tick_edges[i] # Start of the y-bin
    y_upper = y_tick_edges[i+1] # End of the y-bin
    plt.plot([x_coord, x_coord], [y_lower, y_upper], color='red', linestyle='-', linewidth=2)

plt.show()

In [ ]:
lower_limit = Si2_lower_limit
upper_limit = Si2_upper_limit
nevents_bins = Si2_nevents_bins

In [ ]:
dfpulse['bins'] = pd.cut(dfpulse['nevents_pulse'], bins=nevents_bins, include_lowest=True)

pulseIds_by_bin = dfpulse.groupby('bins')['pulseId'].apply(list).reset_index()

filtered_dfevent = dfevent[dfevent.pulseId.isin(pulseIds_by_bin)]

counts = []

# Iterate over each bin range and corresponding pulseIds
for i, row in pulseIds_by_bin.iterrows():
    # Filter dfevent based on pulseIds for the current bin
    filtered_events = dfevent[dfevent['pulseId'].isin(row['pulseId'])]
    
    # Apply the counting logic
    count = ((filtered_events.tof >= lower_limit[i]) & (filtered_events.tof <= upper_limit[i])).sum()
    counts.append(count)
    
lower_bounds = [int(interval.left) for interval in pulseIds_by_bin['bins']]

plt.figure()
plt.scatter(lower_bounds, counts, marker='o')
plt.xlabel('Swoop intensity')
plt.ylabel('Number of hits')
plt.title('SiO swoop intensity in terms of total intensity')
plt.grid(True)
plt.show()

In [ ]:
def compute_swoop_intensity(dfevent,dfpulse,nevents_bins,lower_limit,upper_limit):
    
    dfpulse['bins'] = pd.cut(dfpulse['nevents_pulse'], bins=nevents_bins, include_lowest=True)

    pulseIds_by_bin = dfpulse.groupby('bins')['pulseId'].apply(list).reset_index()

    filtered_dfevent = dfevent[dfevent.pulseId.isin(pulseIds_by_bin)]

    counts = []

    # Iterate over each bin range and corresponding pulseIds
    for i, row in pulseIds_by_bin.iterrows():
        # Filter dfevent based on pulseIds for the current bin
        filtered_events = dfevent[dfevent['pulseId'].isin(row['pulseId'])]

        # Apply the counting logic
        count = ((filtered_events.tof >= lower_limit[i]) & (filtered_events.tof <= upper_limit[i])).sum()
        counts.append(count)

    lower_bounds = [int(interval.left) for interval in pulseIds_by_bin['bins']]
    
    return lower_bounds, counts

In [ ]:
%matplotlib inline

Si_lower_bounds, Si_counts = compute_swoop_intensity(dfevent,dfpulse,Si_nevents_bins,Si_lower_limit,Si_upper_limit)
SiO_lower_bounds, SiO_counts = compute_swoop_intensity(dfevent,dfpulse,SiO_nevents_bins,SiO_lower_limit,SiO_upper_limit)
Si2_lower_bounds, Si2_counts = compute_swoop_intensity(dfevent,dfpulse,Si2_nevents_bins,Si2_lower_limit,Si2_upper_limit)

plt.figure()
plt.scatter(Si_lower_bounds, Si_counts, label='Si')
plt.scatter(SiO_lower_bounds, SiO_counts, label='SiO')
plt.scatter(Si2_lower_bounds, Si2_counts, label='2Si')
plt.xlabel('Swoop intensity')
plt.ylabel('Number of hits')
plt.title('Swoop intensities in terms of total intensity')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
#hists = hists500
nbins_tof = NBINS_TOF
xlimits = (1.08e-6,1.2e-6)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []
hists_short = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist #/ max(shortened_hist)
    hists_norm.append(hist_norm)
    hists_short.append(shortened_hist)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins[:-1])
hists_norm = np.array(hists_norm)
hists_short = np.array(hists_short)

In [ ]:
from scipy.ndimage import label

# Plot the heatmap
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Hit intensity')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(X.min(), X.max())
plt.ylim(Y.min(), Y.max())

# Define a threshold for high-intensity regions
threshold = 20  # Adjust this value as necessary
high_intensity_mask = hists_norm > threshold

# Identify connected components in the high-intensity mask
labeled_array, num_features = label(high_intensity_mask)

# Find the largest connected component
sizes = np.bincount(labeled_array.ravel())
sizes[0] = 0  # Ignore the background
largest_label = sizes.argmax()

# Create a mask for the largest component (the swoop)
swoop_mask = labeled_array == largest_label

# Integrate the area under the high-intensity swoop
area_high_intensity_swoop = np.sum(swoop_mask * hists_short)

# Highlight the high-intensity swoop region
plt.contour(X, Y, swoop_mask, levels=[0.5], colors='red', linewidths=2)

plt.show()

print(f"Integrated area under the high-intensity swoop region: {area_high_intensity_swoop:.4f}")

In [ ]:
#hists = hists500
nbins_tof = NBINS_TOF
xlimits = (1.21e-6,1.4e-6)#(1.4e-6,1.8e-6)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events = len(hists)
hists_norm = []
hists_short = []

for i in range(nbins_events):

    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist #/ max(shortened_hist)
    hists_norm.append(hist_norm)
    hists_short.append(shortened_hist)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins[:-1])
hists_norm = np.array(hists_norm)
hists_short = np.array(hists_short)

In [ ]:
from scipy.ndimage import label

# Plot the heatmap
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Hit intensity')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(X.min(), X.max())
plt.ylim(Y.min(), Y.max())

# Define a threshold for high-intensity regions
threshold = 3  # Adjust this value as necessary
high_intensity_mask = hists_norm > threshold

# Identify connected components in the high-intensity mask
labeled_array, num_features = label(high_intensity_mask)

# Find the largest connected component
sizes = np.bincount(labeled_array.ravel())
sizes[0] = 0  # Ignore the background
largest_label = sizes.argmax()

# Create a mask for the largest component (the swoop)
swoop_mask = labeled_array == largest_label

# Integrate the area under the high-intensity swoop
area_high_intensity_swoop = np.sum(swoop_mask * hists_short)

# Highlight the high-intensity swoop region
plt.contour(X, Y, swoop_mask, levels=[0.5], colors='red', linewidths=2)

plt.show()

print(f"Integrated area under the high-intensity swoop region: {area_high_intensity_swoop:.4f}")

In [ ]:
#hists = hists500
nbins_tof = NBINS_TOF1
xlimits = (1.08e-6,1.4e-6)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events1 = len(hists2)
hists_norm = []
hists_short = []

for i in range(nbins_events1):

    shortened_hist = hists2[i][hist_lower:hist_upper]
    hist_norm = shortened_hist #/ max(shortened_hist)
    hists_norm.append(hist_norm)
    hists_short.append(shortened_hist)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins2[:-1])
hists_norm = np.array(hists_norm)
hists_short = np.array(hists_short)

In [ ]:
from scipy.ndimage import label

# Plot the heatmap
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Hit intensity')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(X.min(), X.max())
plt.ylim(Y.min(), Y.max())

# Define a threshold for high-intensity regions
threshold = 30  # Adjust this value as necessary
high_intensity_mask = hists_norm > threshold

# Identify connected components in the high-intensity mask
labeled_array, num_features = label(high_intensity_mask)

# Find the largest connected component
sizes = np.bincount(labeled_array.ravel())
sizes[0] = 0  # Ignore the background
largest_label = sizes.argmax()

# Create a mask for the largest component (the swoop)
swoop_mask = labeled_array == largest_label

# Integrate the area under the high-intensity swoop
area_high_intensity_swoop = np.sum(swoop_mask * hists_short)

# Highlight the high-intensity swoop region
plt.contour(X, Y, swoop_mask, levels=[0.5], colors='red', linewidths=2)

plt.show()

print(f"Integrated area under the high-intensity swoop region: {area_high_intensity_swoop:.4f}")

In [ ]:
#hists = hists500
nbins_tof = NBINS_TOF1
xlimits = (1.21e-6,1.4e-6)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events1 = len(hists2)
hists_norm = []
hists_short = []

for i in range(nbins_events1):

    shortened_hist = hists2[i][hist_lower:hist_upper]
    hist_norm = shortened_hist #/ max(shortened_hist)
    hists_norm.append(hist_norm)
    hists_short.append(shortened_hist)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins2[:-1])
hists_norm = np.array(hists_norm)
hists_short = np.array(hists_short)

In [ ]:
from scipy.ndimage import label

# Plot the heatmap
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Hit intensity')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(X.min(), X.max())
plt.ylim(Y.min(), Y.max())

# Define a threshold for high-intensity regions
threshold = 20  # Adjust this value as necessary
high_intensity_mask = hists_norm > threshold

# Identify connected components in the high-intensity mask
labeled_array, num_features = label(high_intensity_mask)

# Find the largest connected component
sizes = np.bincount(labeled_array.ravel())
sizes[0] = 0  # Ignore the background
largest_label = sizes.argmax()

# Create a mask for the largest component (the swoop)
swoop_mask = labeled_array == largest_label

# Integrate the area under the high-intensity swoop
area_high_intensity_swoop = np.sum(swoop_mask * hists_short)

# Highlight the high-intensity swoop region
plt.contour(X, Y, swoop_mask, levels=[0.5], colors='red', linewidths=2)

plt.show()

print(f"Integrated area under the high-intensity swoop region: {area_high_intensity_swoop:.4f}")

In [ ]:
NBINS_TOF1 = 4000
NBINS_EVENTS1 = 80

filtered_dfevents1, filtered_dfpulses1, hists1, bins1 = nions_binning_tof(ion_dfevent500,ion_dfpulse500,NBINS_EVENTS1,NBINS_TOF1)

In [ ]:
hists2 = hists1[15:65,:]
bins2 = bins1[15:66]

In [ ]:
#hists = hists500
nbins_tof = NBINS_TOF1
xlimits = (1.08e-6,1.4e-6)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events1 = len(hists2)
hists_norm = []
hists_short = []

for i in range(nbins_events1):

    shortened_hist = hists2[i][hist_lower:hist_upper]
    hist_norm = shortened_hist #/ max(shortened_hist)
    hists_norm.append(hist_norm)
    hists_short.append(shortened_hist)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins2[:-1])
hists_norm = np.array(hists_norm)
hists_short = np.array(hists_short)

In [ ]:
from scipy.ndimage import label

# Plot the heatmap
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Hit intensity')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(X.min(), X.max())
plt.ylim(Y.min(), Y.max())

# Define a threshold for high-intensity regions
threshold = 30  # Adjust this value as necessary
high_intensity_mask = hists_norm > threshold

# Identify connected components in the high-intensity mask
labeled_array, num_features = label(high_intensity_mask)

# Find the largest connected component
sizes = np.bincount(labeled_array.ravel())
sizes[0] = 0  # Ignore the background
largest_label = sizes.argmax()

# Create a mask for the largest component (the swoop)
swoop_mask = labeled_array == largest_label

# Integrate the area under the high-intensity swoop
area_high_intensity_swoop = np.sum(swoop_mask * hists_short)

# Highlight the high-intensity swoop region
plt.contour(X, Y, swoop_mask, levels=[0.5], colors='red', linewidths=2)

plt.show()

print(f"Integrated area under the high-intensity swoop region: {area_high_intensity_swoop:.4f}")

In [ ]:
#hists = hists500
nbins_tof = NBINS_TOF1
xlimits = (1.21e-6,1.4e-6)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events1 = len(hists2)
hists_norm = []
hists_short = []

for i in range(nbins_events1):

    shortened_hist = hists2[i][hist_lower:hist_upper]
    hist_norm = shortened_hist #/ max(shortened_hist)
    hists_norm.append(hist_norm)
    hists_short.append(shortened_hist)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins2[:-1])
hists_norm = np.array(hists_norm)
hists_short = np.array(hists_short)

In [ ]:
from scipy.ndimage import label

# Plot the heatmap
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (s)')
plt.ylabel('Hit intensity')
plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(X.min(), X.max())
plt.ylim(Y.min(), Y.max())

# Define a threshold for high-intensity regions
threshold = 20  # Adjust this value as necessary
high_intensity_mask = hists_norm > threshold

# Identify connected components in the high-intensity mask
labeled_array, num_features = label(high_intensity_mask)

# Find the largest connected component
sizes = np.bincount(labeled_array.ravel())
sizes[0] = 0  # Ignore the background
largest_label = sizes.argmax()

# Create a mask for the largest component (the swoop)
swoop_mask = labeled_array == largest_label

# Integrate the area under the high-intensity swoop
area_high_intensity_swoop = np.sum(swoop_mask * hists_short)

# Highlight the high-intensity swoop region
plt.contour(X, Y, swoop_mask, levels=[0.5], colors='red', linewidths=2)

plt.show()

print(f"Integrated area under the high-intensity swoop region: {area_high_intensity_swoop:.4f}")

In [ ]:
NBINS_TOF1 = 4000
NBINS_EVENTS1 = 60

filtered_dfevents1, filtered_dfpulses1, hists1, bins1 = nions_binning_tof(ion_dfevent500,ion_dfpulse500,NBINS_EVENTS1,NBINS_TOF1)

In [ ]:
NBINS_TOF1 = 4000
NBINS_EVENTS1 = 60

filtered_dfevents1, filtered_dfpulses1, hists1, bins1 = nions_binning_tof(dfevent_single_shot,dfpulse_single_shot,NBINS_EVENTS1,NBINS_TOF1)

In [ ]:
hists2 = hists1[15:65,:]
bins2 = bins1[15:66]

In [ ]:
hists2 = hists1[15:65,:]
bins2 = bins1[15:66]

In [ ]:
#hists = hists500
nbins_tof = NBINS_TOF1
xlimits = (1.08e-6,1.4e-6)

precision = TIME_BETWEEN_PULSES/nbins_tof
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins_events1 = len(hists1)
hists_norm = []
hists_short = []

for i in range(nbins_events1):

    shortened_hist = hists1[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)
    hists_short.append(shortened_hist)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
X, Y = np.meshgrid(x_edges, bins1[:-1])
hists_norm = np.array(hists_norm)
hists_short = np.array(hists_short)

In [ ]:
plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto', cmap='plasma')
colorbar = plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('Time of flight (μs)', fontsize=24)
plt.ylabel('Hit intensity', fontsize=24)
# plt.title('Relative heatmap for number of events slices with respect to tof')
plt.xlim(x_lower, x_upper)


ax1 = plt.gca()
# ax2 = ax1.twiny()

# top_tick_positions = [1.09e-6,1.13e-6,1.17e-6,1.19e-6,1.22e-6,1.25e-6,1.3e-6]
# top_tick_labels = ['750', '500', '300', '200', '100', '50', '0']
# ax2.set_xticks(top_tick_positions)
# ax2.set_xticklabels(top_tick_labels)

# ax2.set_xlim(ax1.get_xlim())
# ax2.set_xlabel('Simulated kinetic energy (eV)', fontsize=24)

# ax2.spines['top'].set_color('blue')
# ax2.xaxis.label.set_color('blue')
# ax2.tick_params(axis='x', colors='blue', labelsize=22)
ax1.tick_params(axis='x', labelsize=22)
ax1.tick_params(axis='y', labelsize=22)
colorbar.ax.set_ylabel('Relative normalized counts', fontsize=24)
colorbar.ax.tick_params(labelsize=22)

plt.show()

In [ ]:
NBINS_TOF = 1500
NBINS_EVENTS = 20

filtered_dfeventslow, filtered_dfpulseslow, histslow, binslow = nions_binning_tof(ion_dfevent_lower_max,ion_dfpulse_lower_max,NBINS_EVENTS,NBINS_TOF)

In [ ]:
nevents_heatmap_rel_tof(histslow,NBINS_TOF,binslow,(1e-7,5e-7))

In [ ]:
nevents_heatmap_rel_tof(histslow,NBINS_TOF,binslow,(1.1e-6,1.4e-6))

## Presentation Plots

In [ ]:
from matplotlib import gridspec

# Compute main plot data
histselected, bin_edgesselected = np.histogram(ion_dfevent500.mq, bins=np.linspace(0,120,1000),range=(0,120))

# Compute heatmap subplot data
Y_ZONE = (145, 0, 10, 256, 7)
fish_dfevent = fish_plot_ion_selection(calibrated_ion_dfevent,Y_ZONE)
fish_dfevent_yfiltered = fish_dfevent[fish_dfevent.y > 32][fish_dfevent.y < 224]

mq_bins = 241
mq_bin_edges = np.linspace(0,120,mq_bins)
fish_dfevent_yfiltered['mq_bin'] = pd.cut(fish_dfevent_yfiltered['mq'], bins=mq_bin_edges, labels=False)

heatmap_data = fish_dfevent_yfiltered.pivot_table(index='y', columns='mq_bin', aggfunc='size', fill_value=0)

max_value = heatmap_data.values.max()
normalized_data = heatmap_data.values / max_value

# Create a figure with gridspec
fig = plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1], hspace=0.1)

# Create main plot
ax_main = plt.subplot(gs[0])
x = bin_edgesselected[:-1]
ax_main.plot(x, histselected/max(histselected), linewidth = 1, c='b')
ax_main.set_ylabel('Normalized number of events')

# Create heatmap subplot
ax_heatmap = plt.subplot(gs[1], sharex=ax_main)
cax = ax_heatmap.imshow(normalized_data, cmap='viridis', aspect='auto', norm=LogNorm(), extent=[x.min(), x.max(), 0, 1])
ax_heatmap.set_xlabel('m/q')
ax_heatmap.set_yticklabels(np.linspace(224,34,6,dtype=int))
ax_heatmap.set_ylabel('y')

# Add colorbar
cbar = plt.colorbar(cax, ax=ax_heatmap, label='Normalized number of events', norm=LogNorm(), orientation='horizontal', pad=0.2)

plt.show()

In [ ]:
from matplotlib import gridspec

calibrated_ion_dfevent = ion_dfevent500

# Compute main plot data
histselected, bin_edgesselected = np.histogram(calibrated_ion_dfevent.mq, bins=np.linspace(0,120,1000),range=(0,120))

# Compute heatmap subplot data
Y_ZONE = (145, 0, 10, 256, 7)
fish_dfevent = fish_plot_ion_selection(calibrated_ion_dfevent,Y_ZONE)
fish_dfevent_yfiltered = fish_dfevent[fish_dfevent.y > 32][fish_dfevent.y < 224]

mq_bins = 241
mq_bin_edges = np.linspace(0,120,mq_bins)
fish_dfevent_yfiltered['mq_bin'] = pd.cut(fish_dfevent_yfiltered['mq'], bins=mq_bin_edges, labels=False)

heatmap_data = fish_dfevent_yfiltered.pivot_table(index='y', columns='mq_bin', aggfunc='size', fill_value=0)

max_value = heatmap_data.values.max()
normalized_data = heatmap_data.values / max_value

# Create a figure with gridspec
fig = plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1], hspace=0.1)

# Create main plot
ax_main = plt.subplot(gs[0])
x = bin_edgesselected[:-1]
ax_main.plot(x, histselected/max(histselected), linewidth = 1, c='b')
ax_main.set_ylabel('Normalized number of events')

# Create heatmap subplot
ax_heatmap = plt.subplot(gs[1], sharex=ax_main)
cax = ax_heatmap.imshow(normalized_data, cmap='magma', aspect='auto', norm=LogNorm(), extent=[x.min(), x.max(), 0, 1])
ax_heatmap.set_xlabel('m/q')
ax_heatmap.set_yticklabels(np.linspace(224,34,6,dtype=int))
ax_heatmap.set_ylabel('y')

# Add colorbar
cbar = plt.colorbar(cax, ax=ax_heatmap, label='Normalized number of events', norm=LogNorm(), orientation='horizontal', pad=0.2)

plt.show()

In [ ]:
heatmap(calibrated_selected_dfevent)

LOWER_MQ = .5
UPPER_MQ = 1.5
mqselected_dfevent,mqselected_dfpulse,mqselected_etof = mq_selection(calibrated_selected_dfevent,selected_dfpulse,selected_etof,LOWER_MQ,UPPER_MQ)
heatmap(mqselected_dfevent)

In [ ]:
short_dfevent = ion_dfevent[ion_dfevent.tof < 40e-9][ion_dfevent.tof > 10e-9]
heatmap(short_dfevent)

In [ ]:
NBINS_MQ = 5000
NBINS_EVENTS = 300
xlimits = (0,5)

filtered_dfevents, filtered_dfpulses, filtered_etofs, hists, bins = nevents_binning(calibrated_selected_dfevent,selected_dfpulse,selected_etof,NBINS_EVENTS,NBINS_MQ)

precision = 200/NBINS_MQ
x_lower, x_upper = xlimits
hist_lower, hist_upper = int(x_lower/precision), int(x_upper/precision)
nbins = len(hists)-150
hists_norm = []
    
for i in range(nbins):
    
    shortened_hist = hists[i][hist_lower:hist_upper]
    hist_norm = shortened_hist / max(shortened_hist)
    hists_norm.append(hist_norm)

x_edges = np.linspace(x_lower, x_upper, hist_upper-hist_lower)
y_edges = bins[:-1-150]
X, Y = np.meshgrid(x_edges, y_edges)

plt.figure(figsize=(20, 8))
c = plt.pcolormesh(X, Y, hists_norm, shading='auto')
plt.colorbar(c, label='Relative normalized counts', extend='max')
plt.xlabel('m/q')
plt.ylabel('Number of events slice')
plt.title('Relative heatmap for number of events slices with respect to m/q')
plt.xlim(x_lower, x_upper)
plt.show()

In [ ]:
bounded_calibrated_selected_dfevent = calibrated_selected_dfevent[calibrated_selected_dfevent.tof < TIME_BETWEEN_PULSES]
hist, bin_edges = np.histogram(bounded_calibrated_selected_dfevent.tof, bins=1000)

channel_time = TIME_BETWEEN_PULSES/CHANNELS_PER_PULSE
    
xaxis = np.arange(CHANNELS_PER_PULSE)*channel_time
avg_selected_etof = -np.mean(selected_etof, axis=0)
    
plt.figure()
plt.plot(bin_edges[:-1], hist/max(hist))
plt.plot(xaxis,avg_selected_etof/max(avg_selected_etof))
plt.xlabel('Time of flight (s)')
plt.ylabel('Normalized signal')
plt.xlim(0,20e-7)
plt.show()

In [ ]:
bounded_selected_dfevent = selected_dfevent[selected_dfevent.tof < TIME_BETWEEN_PULSES]
hist, bin_edges = np.histogram(bounded_selected_dfevent.tof, bins=1000)
    
plt.figure()
plt.plot(bin_edges[:-1], hist/max(hist))
plt.xlabel('Time of flight (s)')
plt.ylabel('Normalized signal')
plt.yscale('log')
plt.xlim(0,1e-7)
plt.show()

In [ ]:
X_LIMITS = (5e-9,16e-8)

channel_time = TIME_BETWEEN_PULSES/CHANNELS_PER_PULSE

lower_xlimit, upper_xlimit = X_LIMITS
lower_int_xlimit, upper_int_xlimit = int(lower_xlimit/channel_time), int(upper_xlimit/channel_time)
shortened_etof = selected_etof[:,lower_int_xlimit:upper_int_xlimit]
shortened_backgrd_etof = backgrd_etof[:,lower_int_xlimit:upper_int_xlimit]
    
xaxis = np.arange(lower_int_xlimit,upper_int_xlimit)*channel_time
avg_selected_etof = -np.mean(shortened_etof, axis=0)
avg_backgrd_etof = -np.mean(shortened_backgrd_etof, axis=0)
max_selected_etof = max(avg_selected_etof)
max_backgrd_etof = max(avg_backgrd_etof)

plt.figure()
plt.plot(xaxis,avg_selected_etof/max_selected_etof,c='r')
plt.plot(xaxis,avg_backgrd_etof/max_backgrd_etof,c='g')
plt.xlabel('Time of flight (s)')
plt.ylabel('Normalized signal')
plt.title('Electrons time of flight')
plt.xlim(lower_xlimit, upper_xlimit)
plt.show()

## Covariances

In [ ]:
MQ_BINS = 400

mq_np_covariance(calibrated_ion_dfevent,MQ_BINS)

In [ ]:
LOG = False
VMIN = -1
VMAX = 10
    
mq_np_covariance(calibrated_selected_dfevent,MQ_BINS,LOG,VMIN,VMAX)

In [ ]:
MQ_BINS = 400
MQ_BIN_RANGE = (0,3)

mq_covariance_1d(calibrated_selected_dfevent,MQ_BIN_RANGE,MQ_BINS)

In [ ]:
def mq_covariance_1d(dfevent,mq_bin_range,mq_bins=200):
    'Produces 1d covariance of an m/q bin range vs m/q employing the numpy cov function'
    'Uses dfevent as input, can select number of mq bins'
    
    mq_bin_edges = np.linspace(0,200,mq_bins+1)

    dfevent['mq_bin'] = pd.cut(dfevent['mq'], bins=mq_bin_edges)

    result_matrix = pd.crosstab(dfevent['pulseId'], dfevent['mq_bin'])
    result_numpy_matrix = result_matrix.values

    array_hyd = result_matrix.values[:,mq_bin_range[0]:mq_bin_range[1]].sum(axis=1)
    shape = result_numpy_matrix.shape[1]

    # Calculate the covariance between each row of result_numpy_matrix and array_hyd
    covariances = np.array([np.cov(np.column_stack((result_numpy_matrix[:, j], array_hyd)), rowvar=False)[0, 1] for j in range(shape)])

    xaxis = np.linspace(0,200,shape)
    fig, ax = plt.subplots()
    ax.plot(xaxis,covariances)
    plt.xlabel('m/q')
    plt.ylabel('Covariance with m/q')
    plt.title(f'Covariance Plot between m/q bin range {mq_bin_range} and m/q')
    ax.set_yscale('symlog', linthresh=10)
    plt.show()

In [ ]:
def tof_covariance_1d(dfevent, tof_range, nbins_ion_tof=200, max_ion_limit=TIME_BETWEEN_PULSES):
    '''Produces 1d covariance of TOF range vs TOF using numpy cov function'''
    
    # Create TOF bins
    tof_bin_edges = np.linspace(0, max_ion_limit, nbins_ion_tof+1)
    dfevent['tof_bin'] = pd.cut(dfevent['tof'], bins=tof_bin_edges)

    # Convert time values to bin indices
    start_bin = np.digitize(tof_range[0], tof_bin_edges) - 1
    end_bin = np.digitize(tof_range[1], tof_bin_edges) - 1
    
    # Clamp to valid range
    start_bin = max(0, start_bin)
    end_bin = min(nbins_ion_tof, end_bin)

    # Create pulseId vs TOF bin matrix
    result_matrix = pd.crosstab(dfevent['pulseId'], dfevent['tof_bin'])
    result_numpy_matrix = result_matrix.values

    # Sum over specified TOF bin range
    array_hyd = result_matrix.values[:, start_bin:end_bin].sum(axis=1)
    shape = result_numpy_matrix.shape[1]

    # Calculate covariance with each TOF bin
    covariances = np.array([np.cov(np.column_stack((result_numpy_matrix[:, j], array_hyd)), rowvar=False)[0, 1] for j in range(shape)])

    # Plot configuration
    xaxis = np.linspace(0, max_ion_limit, shape)
    fig, ax = plt.subplots()
    ax.plot(xaxis, covariances)
    plt.xlabel('TOF')
    plt.ylabel('Covariance with TOF')
    plt.title(f'Covariance Plot between TOF range {tof_range} and TOF')
    ax.set_yscale('symlog', linthresh=10)
    plt.show()


In [ ]:
TOF_BINS = 400
TOF_RANGE = (3.8e-7, 4.2e-7)

tof_covariance_1d(ion_dfevent500,TOF_RANGE,TOF_BINS)

In [ ]:
ion_tof(ion_dfevent500)

In [ ]:
# Unclear when this is needed (for runs [389,390] it is; for runs [375 to 404] it is not)
new_calibrated_selected_dfevent = fix_missing_row(calibrated_selected_dfevent,selected_dfpulse,MQ_BINS)

In [ ]:
mq_covariance(new_calibrated_selected_dfevent,selected_dfpulse,MQ_BINS)

In [ ]:
LOG = False
VMIN = -1
VMAX = 1
    
mq_covariance(new_calibrated_selected_dfevent,selected_dfpulse,MQ_BINS,LOG,VMIN,VMAX)

In [ ]:
for ALPHA in np.arange(1,1.51,.1):
    print('alpha = ', ALPHA)
    mq_partial_covariance(new_calibrated_selected_dfevent,selected_dfpulse,MQ_BINS,ALPHA,LOG,VMIN,VMAX)

In [ ]:
NBINS_ION_TOF = 300
MAX_ION_LIMIT = 1e-6
ALPHA = 1
LOG = False
VMIN = -1e5
VMAX = 1e5

ion_ion_covariance(ion_dfevent500,ion_dfpulse500,NBINS_ION_TOF,MAX_ION_LIMIT,ALPHA,LOG,VMIN,VMAX)

In [ ]:
NBINS_ION_TOF = 500
MAX_ION_LIMIT = 5e-7
ALPHA = 1
LOG = True
VMIN = 0
VMAX = 5

ion_ion_covariance(ion_dfevent500,ion_dfpulse500,NBINS_ION_TOF,MAX_ION_LIMIT,ALPHA,LOG,VMIN,VMAX)

In [ ]:
NBINS_ION_TOF = 500
MAX_ION_LIMIT = 3e-6
ALPHA = 1
LOG = False
VMIN = -10
VMAX = 10

ion_ion_covariance(ion_dfevent500,ion_dfpulse500,NBINS_ION_TOF,MAX_ION_LIMIT,ALPHA,LOG,VMIN,VMAX)

In [ ]:
NBINS_ION_TOF = 500
MAX_ION_LIMIT = 2e-6
ALPHA = 1
LOG = False
VMIN = -10
VMAX = 10

ion_ion_covariance(ion_dfevent,ion_dfpulse,NBINS_ION_TOF,MAX_ION_LIMIT,ALPHA,LOG,VMIN,VMAX)

In [ ]:
NBINS_ION_TOF = 150
NBINS_E_TOF = 100
MAX_ION_LIMIT = 3e-6
MAX_E_LIMIT = 3e-7
ALPHA = 1
LOG = True
VMIN = -1e5
VMAX = 1e5

etof_ion_covariance(dfevent1,dfpulse1,etof1,NBINS_ION_TOF,NBINS_E_TOF,MAX_ION_LIMIT,MAX_E_LIMIT,ALPHA,LOG)#,VMIN,VMAX)

In [ ]:
NBINS_ION_TOF = 150
NBINS_E_TOF = 100
MAX_ION_LIMIT = 3e-6
MAX_E_LIMIT = 3e-7
ALPHA = 1
LOG = True
VMIN = -1e5
VMAX = 1e5

etof_ion_covariance(dfevent2,dfpulse2,etof2,NBINS_ION_TOF,NBINS_E_TOF,MAX_ION_LIMIT,MAX_E_LIMIT,ALPHA,LOG)#,VMIN,VMAX)

In [ ]:
NBINS_ION_TOF = 150
NBINS_E_TOF = 100
MAX_ION_LIMIT = 3e-6
MAX_E_LIMIT = 3e-7
ALPHA = 1
LOG = True
VMIN = -1e5
VMAX = 1e5

etof_ion_covariance(dfevent3,dfpulse3,etof3,NBINS_ION_TOF,NBINS_E_TOF,MAX_ION_LIMIT,MAX_E_LIMIT,ALPHA,LOG)#,VMIN,VMAX)

In [ ]:
NBINS_ION_TOF = 150
NBINS_E_TOF = 100
MAX_ION_LIMIT = 3e-6
MAX_E_LIMIT = 3e-7
ALPHA = 1
LOG = True
VMIN = -1e2
VMAX = 1e2

etof_ion_covariance(selected_dfevent,selected_dfpulse,selected_etof,NBINS_ION_TOF,NBINS_E_TOF,MAX_ION_LIMIT,MAX_E_LIMIT,ALPHA,LOG)#,VMIN,VMAX)

In [ ]:
def etof_ion_covariance_1d(dfevent, etof, etof_range, nbins_ion_tof=200, max_ion_limit=TIME_BETWEEN_PULSES):
    'Produces 1D covariance between selected etof range and ion TOF'
    
    # Create ion TOF bins
    ion_tof_bin_edges = np.linspace(0, max_ion_limit, nbins_ion_tof+1)
    dfevent['tof_bin'] = pd.cut(dfevent['tof'], bins=ion_tof_bin_edges)
    ion_matrix = pd.crosstab(dfevent['pulseId'], dfevent['tof_bin'])
    
    # Process etof data
    coords_etof = etof.assign_coords(data=np.arange(0, TIME_BETWEEN_PULSES, channel_time))
    e_tof_bin_edges = np.linspace(0, max_ion_limit, nbins_ion_tof+1)
    binned_etof = coords_etof.groupby_bins("data", e_tof_bin_edges).sum()
    e_matrix = binned_etof.to_pandas()
    
    # Align matrices
    ion_list = ion_matrix.index.to_list()
    # ion_list.append(0)
    e_list = e_matrix.index.to_list()
    truefalse = np.equal(ion_list, e_list)
    first_instance = np.argmax(~truefalse)
    # e_matrix = e_matrix.drop(e_list[first_instance])
    
    # Extract etof range of interest
    etof_range_idx = (e_matrix.columns >= etof_range[0]) & (e_matrix.columns <= etof_range[1])
    etof_sum = e_matrix.iloc[:, etof_range_idx].sum(axis=1)
    
    # Calculate covariances
    covariances = np.array([
        np.cov(np.column_stack((ion_matrix.values[:, j], etof_sum)), rowvar=False)[0, 1] 
        for j in range(ion_matrix.shape[1])
    ])
    
    # Plot results
    xaxis = np.linspace(0, max_ion_limit, len(covariances))
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(xaxis, covariances)
    plt.xlabel('Ion Time of Flight (s)')
    plt.ylabel('Covariance with eTOF')
    plt.title(f'Covariance between eTOF range {etof_range[0]:.2e}-{etof_range[1]:.2e} s and Ion TOF')
    
    # Add vertical lines at key positions
    plt.axvline(3.7e-7, c='r', linestyle='--')
    plt.axvline(4.1e-7, c='r', linestyle='--')
    plt.axvline(4.5e-7, c='r', linestyle='--')
    
    ax.set_yscale('symlog', linthresh=10)
    plt.grid(True, alpha=0.3)
    plt.show()


In [ ]:
def etof_ion_covariance_1d(dfevent, etof, etof_range, nbins_ion_tof=200, max_ion_limit=TIME_BETWEEN_PULSES):
    'Produces 1D covariance between selected etof range and ion TOF'
    
    # Create ion TOF bins
    ion_tof_bin_edges = np.linspace(0, max_ion_limit, nbins_ion_tof+1)
    dfevent['tof_bin'] = pd.cut(dfevent['tof'], bins=ion_tof_bin_edges)
    ion_matrix = pd.crosstab(dfevent['pulseId'], dfevent['tof_bin'])
    
    # Process etof data
    coords_etof = etof.assign_coords(data=np.arange(0, TIME_BETWEEN_PULSES, channel_time))
    e_tof_bin_edges = np.linspace(0, max_ion_limit, nbins_ion_tof+1)
    binned_etof = coords_etof.groupby_bins("data", e_tof_bin_edges).sum()
    e_matrix = binned_etof.to_pandas()
    
    # Align matrices
    ion_list = ion_matrix.index.to_list()
    # ion_list.append(0)
    e_list = e_matrix.index.to_list()
    truefalse = np.equal(ion_list, e_list)
    first_instance = np.argmax(~truefalse)
    # e_matrix = e_matrix.drop(e_list[first_instance])
    
    # Extract etof range of interest using the bin midpoints
    bin_midpoints = np.array([interval.mid for interval in e_matrix.columns])
    etof_range_idx = (bin_midpoints >= etof_range[0]) & (bin_midpoints <= etof_range[1])
    etof_sum = e_matrix.iloc[:, etof_range_idx].sum(axis=1)
    
    # Calculate covariances
    covariances = np.array([
        np.cov(np.column_stack((ion_matrix.values[:, j], etof_sum)), rowvar=False)[0, 1] 
        for j in range(ion_matrix.shape[1])
    ])
    
    # Plot results
    xaxis = np.linspace(0, max_ion_limit, len(covariances))
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(xaxis, covariances)
    plt.xlabel('Ion Time of Flight (s)')
    plt.ylabel('Covariance with eTOF')
    plt.title(f'Covariance between eTOF range {etof_range[0]:.2e}-{etof_range[1]:.2e} s and Ion TOF')
    
    plt.axvline(3.7e-7, c='r', linestyle='--')
    plt.axvline(4.1e-7, c='r', linestyle='--')
    plt.axvline(4.5e-7, c='r', linestyle='--')
    
    ax.set_yscale('symlog', linthresh=10)
    plt.grid(True, alpha=0.3)
    plt.show()


In [ ]:
etof_range = (1.2e-7, 2e-7)
etof_ion_covariance_1d(selected_dfevent, selected_etof, etof_range, nbins_ion_tof=200)


## CNN

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, roc_curve, auc
import cv2

# Function to load and preprocess data from xarray
def load_and_preprocess_data(pnccd_array, label_array, test_size=0.2, random_state=42):
    """Load and preprocess the xarray data arrays."""
    # Convert xarray to numpy
    images = pnccd_array.values
    
    # Split data into training and testing sets with stratification
    X_train, X_test, y_train, y_test = train_test_split(
        images, labels, test_size=test_size, random_state=random_state, stratify=labels
    )
    
    # Normalize the images to [0, 1] range
    X_train = X_train.astype('float32') / np.max(X_train)
    X_test = X_test.astype('float32') / np.max(X_test)
    
    # Reshape to add channel dimension
    if X_train.ndim == 3:
        X_train = X_train[..., np.newaxis]
        X_test = X_test[..., np.newaxis]
    
    return X_train, X_test, y_train, y_test

# Convert images to polar coordinates for better feature extraction
def to_polar(image, center=None):
    """Transform image to polar coordinates."""
    h, w = image.shape[:2]
    if center is None:
        center = (h // 2, w // 2)
        
    # Calculate maximum radius
    max_radius = np.sqrt((h - center[0])**2 + (w - center[1])**2)
    
    # Define output polar image dimensions (radius, angle)
    polar_h = int(max_radius)
    polar_w = 360
    
    # Create mapping matrix
    mapping = np.zeros((polar_h, polar_w, 2), dtype=np.float32)
    
    # Fill mapping matrix
    for r in range(polar_h):
        for theta in range(polar_w):
            # Convert to radians
            theta_rad = np.deg2rad(theta)
            
            # Calculate corresponding cartesian coordinates
            x = center[0] + r * np.cos(theta_rad)
            y = center[1] + r * np.sin(theta_rad)
            
            mapping[r, theta] = [y, x]
    
    # Perform the remapping
    polar = cv2.remap(image, mapping, None, cv2.INTER_LINEAR)
    
    return polar

# Data generator with balanced sampling and augmentation
class BalancedDataGenerator(tf.keras.utils.Sequence):
    """Data generator that provides balanced batches and applies augmentation."""
    def __init__(self, X, y, batch_size=32, augment_minority=True, use_polar=False):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.augment_minority = augment_minority
        self.use_polar = use_polar
        
        # Find indices for each class
        self.majority_indices = np.where(y == 0)[0]
        self.minority_indices = np.where(y == 1)[0]
        
        # Calculate samples per batch
        self.n_majority = self.batch_size // 2
        self.n_minority = self.batch_size - self.n_majority
        
        # Data augmentation
        self.datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=360,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest'
        )
        
        self.on_epoch_end()
    
    def __len__(self):
        """Return the number of batches per epoch."""
        return int(np.ceil(len(self.minority_indices) * 10 / self.n_minority))
    
    def __getitem__(self, index):
        """Generate one batch of data."""
        # Generate batch indices with balanced classes
        majority_batch_indices = np.random.choice(
            self.majority_indices, self.n_majority, replace=False
        )
        minority_batch_indices = np.random.choice(
            self.minority_indices, self.n_minority, replace=True
        )
        batch_indices = np.concatenate([majority_batch_indices, minority_batch_indices])
        np.random.shuffle(batch_indices)
        
        # Generate data
        X_batch = np.array([self.X[i].copy() for i in batch_indices])
        y_batch = np.array([self.y[i] for i in batch_indices])
        
        # Apply transformations
        for i, (x, y) in enumerate(zip(X_batch, y_batch)):
            # Apply polar transformation if requested
            if self.use_polar and np.random.random() > 0.5:
                if x.ndim == 3:
                    polar_image = to_polar(x[:,:,0])
                    X_batch[i] = polar_image[..., np.newaxis]
                else:
                    X_batch[i] = to_polar(x)
            
            # Apply augmentation to minority class
            if self.augment_minority and y == 1:
                X_batch[i] = self.datagen.random_transform(x)
        
        return X_batch, y_batch
    
    def on_epoch_end(self):
        """Shuffle indices after each epoch."""
        np.random.shuffle(self.majority_indices)
        np.random.shuffle(self.minority_indices)

# Build a CNN model specifically for diffraction pattern classification
def build_model(input_shape, filters_start=32):
    """Build a CNN model designed for diffraction pattern classification."""
    model = models.Sequential()
    
    # First convolutional block
    model.add(layers.Conv2D(filters_start, (3, 3), padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(filters_start, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Second convolutional block
    model.add(layers.Conv2D(filters_start*2, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(filters_start*2, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Third convolutional block
    model.add(layers.Conv2D(filters_start*4, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Global pooling and dense layers
    model.add(layers.GlobalAveragePooling2D())  # Good for rotational invariance
    model.add(layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification
    
    # Compile model with weighted binary crossentropy
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    
    return model

# Function to train model with class balancing
def train_model(X_train, y_train, X_val, y_val, model, batch_size=32, epochs=50, use_generator=True, use_polar=True):
    """Train the model with balanced sampling and appropriate callbacks."""
    # Calculate class weights to handle imbalance
    n_samples = len(y_train)
    n_monomers = np.sum(y_train == 0)
    n_dimers = np.sum(y_train == 1)
    class_weight = {
        0: n_samples / (2 * n_monomers),
        1: n_samples / (2 * n_dimers)
    }
    
    # Callbacks for training
    callbacks_list = [
        callbacks.EarlyStopping(
            monitor='val_precision',
            patience=10,
            mode='max',
            restore_best_weights=True
        ),
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6
        ),
        callbacks.ModelCheckpoint(
            filepath='best_diffraction_model.h5',
            monitor='val_precision',
            save_best_only=True,
            mode='max'
        )
    ]
    
    # Train with generator for balanced batches
    if use_generator:
        train_gen = BalancedDataGenerator(
            X_train, y_train, batch_size=batch_size, 
            augment_minority=True, use_polar=use_polar
        )
        
        history = model.fit(
            train_gen,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks_list,
            class_weight=class_weight
        )
    else:
        # Standard training with class weights
        history = model.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks_list,
            class_weight=class_weight
        )
    
    return model, history

# Function to evaluate model and find optimal threshold
def evaluate_model(model, X_test, y_test):
    """Evaluate model and determine optimal classification threshold."""
    # Get predictions
    y_pred_prob = model.predict(X_test).flatten()
    
    # Find optimal threshold using precision-recall curve
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_prob)
    f1_scores = 2 * recall * precision / (recall + precision + 1e-10)
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx]
    
    # Apply threshold
    y_pred = (y_pred_prob >= optimal_threshold).astype(int)
    
    # Calculate metrics
    report = classification_report(y_test, y_pred, output_dict=True)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    
    # Collect metrics
    metrics = {
        'report': report,
        'confusion_matrix': conf_matrix,
        'optimal_threshold': optimal_threshold,
        'roc_auc': roc_auc,
        'precision': precision,
        'recall': recall,
        'fpr': fpr,
        'tpr': tpr
    }
    
    return metrics, y_pred

# Main function to run the entire pipeline
def main(pnccd_array, label_array):
    """Complete pipeline for training and evaluating the model."""
    # Load and preprocess data
    X_train, X_test, y_train, y_test = load_and_preprocess_data(
        pnccd_array, label_array
    )
    
    # Further split training data for validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, stratify=y_train
    )
    
    # Print dataset information
    print(f"Dataset shapes:")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")
    print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")
    print(f"Class distribution:")
    print(f"Training: Monomers = {np.sum(y_train == 0)}, Dimers = {np.sum(y_train == 1)}")
    
    # Build the model
    model = build_model(X_train.shape[1:])
    model.summary()
    
    # Train the model
    model, history = train_model(
        X_train, y_train, X_val, y_val, model,
        batch_size=32, epochs=50, use_generator=True, use_polar=False
    )
    
    # Evaluate model
    metrics, y_pred = evaluate_model(model, X_test, y_test)
    
    return model, history, metrics

In [ ]:
# Example testing code
def test_model(model, pnccd_array, label_array):
    # Select test samples by train ID
    test_ids = np.random.choice(pnccd_array.trainId.values, 100, replace=False)
    
    # Extract test data
    test_data = pnccd_array.sel(trainId=test_ids).values
    test_labels = label_array.sel(trainId=test_ids).values  # Convert to 0-based
    
    # Preprocess
    test_data = test_data.astype('float32') / np.max(test_data)
    test_data = test_data[..., np.newaxis]  # Add channel dimension
    
    # Predict
    predictions = model.predict(test_data)
    
    # Get classification using optimal threshold (from previous evaluation)
    y_pred = (predictions >= 0.5).astype(int).flatten()

    cm = confusion_matrix(test_labels, y_pred)
    sns.heatmap(cm, annot=True)
    plt.show()

    # Calculate metrics
    print(classification_report(test_labels, y_pred))
    
    return test_data, test_labels, y_pred

In [ ]:
cleaned_array = actual_result.dropna(dim='trainId')

trainIds = np.array(cleaned_array.trainId)
label_array = np.array(cleaned_array, dtype=int)

pnccd_array = high_fluence_pnccd[high_fluence_pnccd.trainId.isin(trainIds)]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform
import cv2

def physically_meaningful_augmentation(image, n_augmentations=4):
    """
    Apply physically meaningful augmentations to diffraction patterns.
    
    Parameters:
    -----------
    image : numpy array
        Original diffraction pattern image
    n_augmentations : int
        Number of augmented versions to generate
        
    Returns:
    --------
    list of numpy arrays
        Augmented images
    """
    # Ensure image is 2D
    if image.ndim == 3:
        image = image.squeeze()
    
    # Extract image properties
    height, width = image.shape
    center_y, center_x = height // 2, width // 2
    
    augmented_images = []
    
    for i in range(n_augmentations):
        # Start with a copy of the original
        aug_img = image.copy()
        
        # 1. Rotation - physically meaningful for diffraction patterns
        # Rotate around the center of the diffraction pattern
        rotation_angle = np.random.uniform(0, 360)
        aug_img = transform.rotate(
            aug_img, 
            rotation_angle, 
            center=(center_x, center_y),
            mode='reflect', 
            preserve_range=True
        )
        
        # 2. Minor zoom - simulates slight changes in detector distance
        # Use very conservative zoom to maintain pattern integrity
        if np.random.random() > 0.5:
            zoom_factor = np.random.uniform(0.95, 1.05)
            aug_img = transform.rescale(
                aug_img, 
                zoom_factor, 
                anti_aliasing=True,
                preserve_range=True
            )
            
            # Crop or pad to maintain original size
            if zoom_factor > 1:
                # Crop
                new_h, new_w = aug_img.shape
                start_h = (new_h - height) // 2
                start_w = (new_w - width) // 2
                aug_img = aug_img[start_h:start_h+height, start_w:start_w+width]
            else:
                # Pad
                new_h, new_w = aug_img.shape
                pad_h = (height - new_h) // 2
                pad_w = (width - new_w) // 2
                aug_img = np.pad(
                    aug_img, 
                    ((pad_h, height - new_h - pad_h), 
                     (pad_w, width - new_w - pad_w)), 
                    mode='constant'
                )
        
        # 3. Minor intensity variations - simulates exposure differences
        if np.random.random() > 0.5:
            intensity_factor = np.random.uniform(0.9, 1.1)
            aug_img = aug_img * intensity_factor
            
            # Ensure we don't exceed original range
            aug_img = np.clip(aug_img, 0, np.max(image))
        
        # 4. Minor noise addition - simulates detector noise
        if np.random.random() > 0.5:
            noise_level = np.random.uniform(0, 0.03 * np.max(image))
            noise = np.random.normal(0, noise_level, aug_img.shape)
            aug_img = aug_img + noise
            
            # Ensure we stay in valid range
            aug_img = np.clip(aug_img, 0, np.max(image))
            
        augmented_images.append(aug_img)
    
    return augmented_images

def to_polar(image):
    """
    Transform image to polar coordinates centered at the diffraction pattern center.
    
    Parameters:
    -----------
    image : numpy array
        Original diffraction pattern image
        
    Returns:
    --------
    numpy array
        Polar coordinate representation
    """
    # Ensure image is 2D
    if image.ndim == 3:
        image = image.squeeze()
        
    height, width = image.shape
    
    # For diffraction patterns, center is typically in the middle
    center = (height // 2, width // 2)
    
    # Calculate maximum radius
    max_radius = min(center[0], center[1], height - center[0], width - center[1])
    
    # Define output polar image dimensions (radius, angle)
    polar_height = max_radius
    polar_width = 360  # Full circle in degrees
    
    # Create coordinate mapping
    y, x = np.indices((height, width))
    
    # Calculate r and theta for each pixel
    r = np.sqrt((y - center[0])**2 + (x - center[1])**2)
    theta = np.arctan2(y - center[0], x - center[1]) * 180 / np.pi
    
    # Normalize radius to range [0, 1]
    r_normalized = r / max_radius
    
    # Shift theta to range [0, 360]
    theta = (theta + 360) % 360
    
    # Map to output coordinates
    r_idx = np.clip(r, 0, max_radius-1).astype(int)
    theta_idx = (theta * polar_width / 360).astype(int) % polar_width
    
    # Create output polar image
    polar = np.zeros((polar_height, polar_width))
    
    # For each pixel in polar space, find the corresponding pixel in the image
    for i in range(polar_height):
        for j in range(polar_width):
            # Convert polar to Cartesian
            theta_rad = j * 2 * np.pi / polar_width
            r_val = i
            
            y_val = int(center[0] + r_val * np.sin(theta_rad))
            x_val = int(center[1] + r_val * np.cos(theta_rad))
            
            # Check if within image bounds
            if 0 <= y_val < height and 0 <= x_val < width:
                polar[i, j] = image[y_val, x_val]
    
    return polar

# Function to visualize original and augmented images
def visualize_augmentations(original_image, n_augmentations=4):
    """
    Generate and visualize physically meaningful augmentations for diffraction patterns.
    
    Parameters:
    -----------
    original_image : numpy array
        Original diffraction pattern image
    n_augmentations : int
        Number of augmented versions to generate
    """
    # Generate augmentations
    augmented_images = physically_meaningful_augmentation(original_image, n_augmentations)
    
    # Create figure for visualization
    plt.figure(figsize=(15, 5))
    
    # Display original image
    plt.subplot(1, n_augmentations + 1, 1)
    plt.imshow(original_image, cmap='viridis')
    plt.title("Original")
    plt.axis('off')
    
    # Display augmented images
    for i, aug_img in enumerate(augmented_images):
        plt.subplot(1, n_augmentations + 1, i + 2)
        plt.imshow(aug_img, cmap='viridis')
        plt.title(f"Aug #{i+1}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Visualize polar transforms
    plt.figure(figsize=(15, 5))
    
    # Display original polar transform
    original_polar = to_polar(original_image)
    plt.subplot(1, n_augmentations + 1, 1)
    plt.imshow(original_polar, cmap='viridis')
    plt.title("Original (Polar)")
    plt.axis('off')
    
    # Display augmented polar transforms
    for i, aug_img in enumerate(augmented_images):
        aug_polar = to_polar(aug_img)
        plt.subplot(1, n_augmentations + 1, i + 2)
        plt.imshow(aug_polar, cmap='viridis')
        plt.title(f"Aug #{i+1} (Polar)")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()


In [ ]:
pnccd_np_array = np.array(pnccd_array)
visualize_augmentations(pnccd_np_array[0], n_augmentations=4)

In [ ]:
model, history, metrics = main(pnccd_array, label_array)

In [ ]:

# Call this function with your trained model
test_data, test_labels, predictions = test_model(model, pnccd_array, cleaned_array)


In [ ]:
def test_data_loading(pnccd_array, label_array):
    """Test that data loads correctly and matches expected shapes and label values."""
    assert pnccd_array.shape[0] == label_array.shape[0], "Mismatch in number of images and labels"
    assert set(np.unique(label_array.values)).issubset({0, 1}), "Labels should be 1 (monomer) or 2 (dimer)"
    print("Data loading test passed.")

def test_class_distribution(label_array):
    """Print and check class distribution."""
    unique, counts = np.unique(label_array.values, return_counts=True)
    print("Class distribution:")
    for u, c in zip(unique, counts):
        print(f"Label {u}: {c} samples")
    assert min(counts) > 0, "One of the classes has zero samples."
    print("Class distribution test passed.")

def test_model_architecture(model, input_shape):
    """Check if the model can process a batch and output correct shape."""
    dummy_input = np.random.rand(2, *input_shape).astype(np.float32)
    output = model.predict(dummy_input)
    assert output.shape == (2, 1), "Model output shape mismatch."
    print("Model architecture test passed.")

def test_hyperparameter_grid(X_train, y_train, X_val, y_val, build_model_fn, param_grid):
    """Test different hyperparameters and print validation accuracy."""
    results = []
    for filters in param_grid['filters_start']:
        model = build_model_fn(X_train.shape[1:], filters_start=filters)
        model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_val, y_val), verbose=0)
        val_acc = model.evaluate(X_val, y_val, verbose=0)[1]
        results.append((filters, val_acc))
        print(f"Filters: {filters}, Val Accuracy: {val_acc:.3f}")
    return results


In [ ]:
def test_performance_metrics(model, X_test, y_test, threshold=0.5):
    """Print classification report and confusion matrix."""
    from sklearn.metrics import classification_report, confusion_matrix
    y_pred_prob = model.predict(X_test).flatten()
    y_pred = (y_pred_prob >= threshold).astype(int)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))


In [ ]:
n = len(test_data)
plt.figure(figsize=(20,int(3*n/5)))
for i in range(n):
    plt.subplot(int(n/5),5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_data[i])
    plt.xlabel(f'actual = {test_labels[i]}')
    plt.xlabel(f'predicted = {predictions[i]}')
plt.show()

## Classifier

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from scipy.fftpack import fft2, fftshift
import xarray as xr
from skimage.transform import warp_polar

da = high_fluence_pnccd

n = 20  # Number of random images to plot
random_indices = np.random.choice(da.shape[0], n, replace=False)

fig, axes = plt.subplots(n, 5, figsize=(20, 3 * n))

for i, idx in enumerate(random_indices):
    image = da[idx].values
    
    # Original image
    axes[i, 0].imshow(image, cmap='viridis')
    axes[i, 0].set_title(f"Original Image {idx}")

    # Greyscale version (already greyscale)
    axes[i, 1].imshow(image, cmap='gray')
    axes[i, 1].set_title("Greyscale Image")

    # Convolution version
    kernel = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
    convolved_image = convolve2d(image, kernel, mode='same', boundary='wrap')
    axes[i, 2].imshow(convolved_image, cmap='gray')
    axes[i, 2].set_title("Convolution Image")

    # Fourier transform
    f_transform = np.log(np.abs(fftshift(fft2(image))) + 1)
    axes[i, 3].imshow(f_transform, cmap='gray')
    axes[i, 3].set_title("Fourier Transform")

    # Cartesian to polar coordinate conversion
    polar_image = polar_to_rectilinear(image)
    
    axes[i, 4].imshow(polar_image, cmap='viridis', aspect='auto')
    axes[i, 4].set_title("Polar Coordinates")
    axes[i, 4].set_xlabel("Angle (θ)")
    axes[i, 4].set_ylabel("Radius (r)")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
import cv2
from pathlib import Path
import glob

def extract_azimuthal_intensity(image, center=None, radius_range=None, 
                               bins=360, mask_beamstop=True):
    """
    Extract azimuthal intensity variation from a diffraction pattern.
    
    Parameters:
    -----------
    image : 2D numpy array
        The diffraction pattern image
    center : tuple, optional
        (y, x) coordinates of diffraction center. If None, uses center of mass
    radius_range : tuple, optional
        (min_radius, max_radius) to consider for integration. If None, uses full image
    bins : int, optional
        Number of azimuthal bins (default: 360 for 1-degree resolution)
    mask_beamstop : bool, optional
        Whether to exclude the beamstop region from the analysis
        
    Returns:
    --------
    tuple : (angles, intensities)
        angles: array of azimuthal angles in degrees
        intensities: normalized intensity at each angle
    """
    # Convert to grayscale if needed
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Find the center if not provided
    if center is None:
        # Use a thresholded image for better center detection
        threshold = np.percentile(image, 95)  # Adjust percentile as needed
        binary_image = image > threshold
        center = center_of_mass(binary_image)

    print(center)
    
    # Create coordinate grid
    rows, cols = image.shape
    y, x = np.indices((rows, cols))
    
    # Calculate radial distance from center
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    
    # Calculate azimuthal angle (in degrees)
    print(y,x)
    phi = np.arctan2(y - center[1], x - center[0]) * 180 / np.pi
    # Normalize to [0, 360)
    phi = (phi + 360) % 360
    
    # Create mask for beamstop if needed
    if mask_beamstop:
        # Adjust threshold and size as needed for your images
        threshold_value = np.percentile(image, 10)
        beamstop_mask = image <= threshold_value
        image = np.ma.array(image, mask=beamstop_mask)
    
    # Set radius range if not provided
    if radius_range is None:
        min_radius = 5  # Avoid center pixels
        max_radius = min(center[0], center[1], 
                         rows - center[0], cols - center[1])
    else:
        min_radius, max_radius = radius_range

    plt.imshow(image)
    plt.plot(center[0],center[1],'or')
    
    # Create mask for specified radius range
    radius_mask = (r >= min_radius) & (r <= max_radius)
    
    # Bin the azimuthal angles
    bin_edges = np.linspace(0, 360, bins + 1)
    azimuthal_intensity = np.zeros(bins)
    
    for i in range(bins):
        angle_mask = (phi >= bin_edges[i]) & (phi < bin_edges[i + 1])
        valid_mask = angle_mask & radius_mask
        if np.any(valid_mask):
            azimuthal_intensity[i] = np.mean(image[valid_mask])
    
    # Normalize the intensity
    if np.max(azimuthal_intensity) > 0:
        azimuthal_intensity = azimuthal_intensity / np.max(azimuthal_intensity)
    
    return bin_edges[:-1], azimuthal_intensity

def plot_azimuthal_profile(image, angles, intensities, title="Azimuthal Intensity Profile"):
    """Plot the original image alongside its azimuthal intensity profile"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot the original image
    ax1.imshow(image, cmap='viridis')
    ax1.set_title("Original Diffraction Pattern")
    ax1.axis('off')
    
    # Plot the azimuthal profile
    ax2.plot(angles, intensities, 'b-')
    ax2.set_xlabel("Azimuthal Angle (degrees)")
    ax2.set_ylabel("Normalized Intensity")
    ax2.set_title(title)
    ax2.set_xlim(0, 360)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

def process_multiple_images(image_folder, pattern="*.jpg", save_folder=None):
    """Process multiple diffraction pattern images"""
    if save_folder and not Path(save_folder).exists():
        Path(save_folder).mkdir(parents=True)
    
    image_paths = glob.glob(str(Path(image_folder) / pattern))
    results = []
    
    for img_path in image_paths:
        img_name = Path(img_path).stem
        print(f"Processing {img_name}...")
        
        # Load image
        image = cv2.imread(img_path)
        if image is None:
            print(f"Failed to load {img_path}")
            continue
            
        # Extract azimuthal profile
        angles, intensities = extract_azimuthal_intensity(image)
        
        # Plot and save if needed
        fig = plot_azimuthal_profile(image, angles, intensities, 
                                    title=f"Azimuthal Profile: {img_name}")
        
        if save_folder:
            fig.savefig(str(Path(save_folder) / f"{img_name}_azimuthal.png"))
            plt.close(fig)
            
            # Save the data as CSV
            data = np.column_stack((angles, intensities))
            np.savetxt(str(Path(save_folder) / f"{img_name}_azimuthal.csv"), 
                      data, delimiter=',', header='angle,intensity')
        
        results.append({
            'name': img_name,
            'angles': angles,
            'intensities': intensities
        })
        
    return results

In [ ]:
angles, intensities = extract_azimuthal_intensity(monomer, (104,100))
plot_azimuthal_profile(monomer, angles, intensities)
plt.show()

In [ ]:
import cv2

def polar_to_rectilinear(data_array):
    """
    Converts a polar diffraction image (as an xarray DataArray) to a rectilinear image.

    Args:
        data_array (xr.DataArray): The input polar diffraction image as an xarray DataArray.

    Returns:
        rectilinear_image (numpy.ndarray): The rectilinear transformed image.
    """
    # Get the image data as a NumPy array
    # image = data_array.values

    # Get image dimensions
    height, width = image.shape

    # Define the center of the diffraction pattern (assumed bottom-center)
    center_x, center_y = width // 2 + 3, height

    # Define the maximum radius to transform
    max_radius = min(center_x, height)

    # Define the output rectilinear image size
    rect_height = max_radius  # Vertical axis corresponds to radius
    rect_width = 180  # Only half of the original 360 degrees

    # Create a mesh grid for the polar transformation (half-circle)
    theta = np.linspace(0, np.pi, rect_width)  # 0 to 180 degrees
    r = np.linspace(0, max_radius, rect_height)  # Radial distances

    # Create coordinate matrices
    Theta, R = np.meshgrid(theta, r)

    # Convert polar coordinates to Cartesian
    X = center_x + R * np.cos(Theta)
    Y = center_y - R * np.sin(Theta)

    # Map coordinates to integer values
    X = X.astype(np.float32)
    Y = Y.astype(np.float32)

    # Perform the remapping
    rectilinear_image = cv2.remap(image, X, Y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

    # Flip vertically to keep intensity strongest at the bottom
    rectilinear_image = cv2.flip(rectilinear_image, 0)

    return rectilinear_image

In [ ]:
monomer_rect = polar_to_rectilinear(monomer)
dimer_rect = polar_to_rectilinear(dimer)

In [ ]:
plt.imshow(monomer_rect)

In [ ]:
plt.imshow(dimer_rect)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import map_coordinates
import xarray as xr

def polar_to_cartesian(polar_data):
    """
    Convert a polar image (r, θ) to Cartesian coordinates (x, y)
    
    Parameters:
    -----------
    polar_data : xarray.DataArray
        The input data in polar coordinates where:
        - First dimension (dim_0) is radius
        - Second dimension (dim_1) is angle in degrees
    
    Returns:
    --------
    cartesian_image : np.ndarray
        The transformed image in Cartesian coordinates
    """
    # Extract values from the DataArray
    polar_image = polar_data.values
    
    # Get dimensions
    r_dim, theta_dim = polar_image.shape
    
    # Create coordinate arrays for radius and angle
    # Assuming angles go from 0 to 360 degrees
    angles = np.linspace(0, 2*np.pi, theta_dim, endpoint=False)
    
    # Determine the output image size (should be large enough to contain the full circle)
    max_radius = r_dim - 1
    output_shape = (2*max_radius, 2*max_radius)
    
    # Initialize the output Cartesian image
    cartesian_image = np.zeros(output_shape)
    
    # Create coordinate meshgrid for the output image
    y_coords, x_coords = np.indices(output_shape)
    
    # Calculate the center of the output image
    y_center, x_center = max_radius, max_radius
    
    # Convert Cartesian coordinates to polar coordinates
    x_from_center = x_coords - x_center
    y_from_center = y_coords - y_center
    r = np.sqrt(x_from_center**2 + y_from_center**2)
    theta = np.arctan2(y_from_center, x_from_center)
    
    # Convert negative angles to positive
    theta = np.mod(theta, 2*np.pi)
    
    # Scale theta to indices in the polar image
    theta_idx = theta * theta_dim / (2*np.pi)
    
    # Create coordinate arrays for mapping
    # Only map points within the valid radius
    mask = r <= max_radius
    coords = np.vstack((r[mask], theta_idx[mask]))
    
    # Sample from the polar image using interpolation
    cartesian_image[mask] = map_coordinates(polar_image, coords, order=1)
    
    return cartesian_image

def plot_polar_and_cartesian(polar_data, cartesian_image, title='Diffraction Pattern'):
    """
    Plot both the polar and Cartesian versions of the image
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Plot polar image
    im1 = ax1.imshow(polar_data.values, aspect='auto', cmap='viridis')
    ax1.set_title('Polar Coordinates')
    ax1.set_xlabel('Angle (θ)')
    ax1.set_ylabel('Radius (r)')
    
    # Plot Cartesian image
    im2 = ax2.imshow(cartesian_image, cmap='viridis')
    ax2.set_title('Cartesian Coordinates')
    ax2.set_xlabel('X-axis')
    ax2.set_ylabel('Y-axis')
    
    # Add colorbars
    plt.colorbar(im1, ax=ax1, label='Intensity')
    plt.colorbar(im2, ax=ax2, label='Intensity')
    
    plt.suptitle(title)
    plt.tight_layout()
    return fig

# Main function to process the DataArray
def process_diffraction_pattern(data_array, n_samples=1):
    """
    Process n_samples random images from the DataArray
    
    Parameters:
    -----------
    data_array : xarray.DataArray
        The input data array with dimensions (trainId, dim_0, dim_1)
    n_samples : int
        Number of random samples to process
    """
    # Select random samples if needed
    if n_samples == 1:
        # Just take the first image
        sample_idx = 0
        polar_data = data_array.isel(trainId=sample_idx)
    else:
        # Select random samples
        sample_indices = np.random.choice(data_array.dims['trainId'], n_samples, replace=False)
        for i, idx in enumerate(sample_indices):
            polar_data = data_array.isel(trainId=idx)
            cartesian_image = polar_to_cartesian(polar_data)
            plot_polar_and_cartesian(polar_data, cartesian_image, 
                                     title=f'Diffraction Pattern (Sample {i+1}, trainId={idx})')
            plt.show()
        return
    
    # For single sample case
    cartesian_image = polar_to_cartesian(polar_data)
    fig = plot_polar_and_cartesian(polar_data, cartesian_image)
    
    return cartesian_image

cartesian_image = process_diffraction_pattern(da)

# Example usage (assuming da is your xarray DataArray):
"""
cartesian_image = process_diffraction_pattern(da)

# Or process multiple patterns
process_diffraction_pattern(da, n_samples=3)
"""


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import cv2
from tqdm.notebook import tqdm  # For showing progress in a notebook

# Function to preprocess a diffraction image
def preprocess_diffraction_image(image):
    """Preprocess a diffraction image."""
    # Convert image to float and normalize
    image = image.astype(float)
    
    # Apply Gaussian blur to reduce noise
    image = cv2.GaussianBlur(image, (5, 5), 0)
    
    return image

# Function to extract features from a diffraction image
def extract_diffraction_features(image):
    """Extract features that distinguish between monomers and dimers."""
    # Get image dimensions and center
    h, w = image.shape
    center_y, center_x = h // 2, w // 2
    
    # Create coordinate grids
    y, x = np.ogrid[:h, :w]
    
    # Calculate radius and angle for each pixel
    r = np.sqrt((x - center_x)**2 + (y - center_y)**2).astype(int)
    theta = np.arctan2(y - center_y, x - center_x)
    theta = (theta + 2*np.pi) % (2*np.pi)
    
    # 1. Extract radial profile (intensity vs. radius)
    max_r = int(np.max(r))
    radial_profile = np.zeros(max_r + 1)
    for i in range(max_r + 1):
        mask = r == i
        if mask.any():
            radial_profile[i] = np.mean(image[mask])
    
    # Downsample radial profile
    n_radial_samples = 40
    radial_profile_downsampled = np.interp(
        np.linspace(0, max_r, n_radial_samples), 
        np.arange(max_r + 1), 
        radial_profile
    )
    
    # 2. Extract azimuthal profiles at different radii
    n_angular_bins = 36  # 10 degrees per bin
    azimuthal_features = []
    
    # Sample at 3 different radii
    sample_radii = [max_r // 3, max_r // 2, 2 * max_r // 3]
    
    for r_sample in sample_radii:
        azimuthal_profile = np.zeros(n_angular_bins)
        
        for i in range(n_angular_bins):
            theta_min = i * (2*np.pi/n_angular_bins)
            theta_max = (i+1) * (2*np.pi/n_angular_bins)
            
            # Select pixels in this angular bin and radius range
            angular_mask = (theta >= theta_min) & (theta < theta_max)
            radius_mask = (r >= r_sample-2) & (r <= r_sample+2)
            mask = angular_mask & radius_mask
            
            if mask.any():
                azimuthal_profile[i] = np.mean(image[mask])
        
        # Calculate variance (higher for interference patterns)
        azimuthal_var = np.var(azimuthal_profile)
        # Calculate max/min ratio (higher for interference patterns)
        azimuthal_max_min_ratio = np.max(azimuthal_profile) / (np.min(azimuthal_profile) + 1e-6)
        
        azimuthal_features.extend([azimuthal_var, azimuthal_max_min_ratio])
    
    # Combine features
    features = np.concatenate([radial_profile_downsampled, azimuthal_features])
    
    return features

# Main function to classify diffraction patterns from xarray
def classify_diffraction_patterns(data_array, sample_size=None):
    """
    Classify diffraction images from an xarray DataArray into monomers and dimers.
    
    Parameters:
    -----------
    data_array : xarray.DataArray
        The xarray containing the diffraction patterns
    sample_size : int, optional
        Number of images to sample for clustering (useful for large datasets)
        If None, all images are used
    """
    # Get number of images
    n_images = data_array.shape[0]
    
    # Sample images if sample_size is provided
    if sample_size is not None and sample_size < n_images:
        indices = np.random.choice(n_images, sample_size, replace=False)
    else:
        indices = np.arange(n_images)
    
    # Extract features from each image
    features_list = []
    
    print(f"Extracting features from {len(indices)} images...")
    for i in tqdm(indices):
        # Get image from the xarray
        img = data_array[i].values
        
        # Preprocess image
        processed_img = preprocess_diffraction_image(img)
        
        # Extract features
        features = extract_diffraction_features(processed_img)
        features_list.append(features)
    
    # Convert to numpy array
    features_array = np.array(features_list)
    
    # Standardize features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features_array)
    
    # Apply K-means clustering
    print("Applying K-means clustering...")
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    sample_labels = kmeans.fit_predict(features_scaled)
    
    # If we sampled, predict for all images
    if sample_size is not None and sample_size < n_images:
        print("Predicting labels for all images...")
        labels = np.zeros(n_images, dtype=int)
        labels[indices] = sample_labels
        
        # Process remaining images
        remaining_indices = np.setdiff1d(np.arange(n_images), indices)
        for i in tqdm(remaining_indices):
            img = data_array[i].values
            processed_img = preprocess_diffraction_image(img)
            features = extract_diffraction_features(processed_img)
            features_scaled = scaler.transform(features.reshape(1, -1))
            labels[i] = kmeans.predict(features_scaled)[0]
    else:
        labels = sample_labels
    
    return labels, kmeans, scaler, indices

# Function to determine which cluster corresponds to monomers and which to dimers
def determine_cluster_identity(data_array, indices, labels):
    """Determine which cluster is monomer and which is dimer."""
    # Get indices for each cluster
    cluster_0_indices = indices[labels[indices] == 0]
    cluster_1_indices = indices[labels[indices] == 1]
    
    # If either cluster is empty, return a default mapping
    if len(cluster_0_indices) == 0 or len(cluster_1_indices) == 0:
        return {0: "Cluster 0", 1: "Cluster 1"}
    
    # Calculate average azimuthal variance for each cluster
    azimuthal_vars_0 = []
    azimuthal_vars_1 = []
    
    print("Analyzing cluster characteristics...")
    # Sample a few images from each cluster for efficiency
    sample_size = min(20, len(cluster_0_indices), len(cluster_1_indices))
    
    for i in np.random.choice(cluster_0_indices, sample_size, replace=False):
        img = data_array[i].values
        processed_img = preprocess_diffraction_image(img)
        features = extract_diffraction_features(processed_img)
        # The azimuthal variance features start after the radial profile
        azimuthal_vars = features[40:][::2]  # Every other feature is a variance
        azimuthal_vars_0.append(np.mean(azimuthal_vars))
    
    for i in np.random.choice(cluster_1_indices, sample_size, replace=False):
        img = data_array[i].values
        processed_img = preprocess_diffraction_image(img)
        features = extract_diffraction_features(processed_img)
        azimuthal_vars = features[40:][::2]
        azimuthal_vars_1.append(np.mean(azimuthal_vars))
    
    avg_var_0 = np.mean(azimuthal_vars_0)
    avg_var_1 = np.mean(azimuthal_vars_1)
    
    # The cluster with lower azimuthal variance corresponds to monomers (full rings)
    # The cluster with higher azimuthal variance corresponds to dimers (interference patterns)
    if avg_var_0 < avg_var_1:
        return {0: "Monomer (Full Rings)", 1: "Dimer (Interference Pattern)"}
    else:
        return {1: "Monomer (Full Rings)", 0: "Dimer (Interference Pattern)"}

# Function to visualize classification results
def visualize_classification(data_array, labels, label_map, n_per_class=3):
    """Visualize classification results showing examples from each class."""
    fig, axes = plt.subplots(2, n_per_class, figsize=(15, 8))
    fig.suptitle("Diffraction Pattern Classification Results", fontsize=16)
    
    # For each cluster
    for cluster_idx in range(2):
        # Get indices of images in this cluster
        cluster_indices = np.where(labels == cluster_idx)[0]
        
        if len(cluster_indices) == 0:
            continue
            
        # Sample n_per_class images
        sample_indices = np.random.choice(cluster_indices, 
                                         min(n_per_class, len(cluster_indices)), 
                                         replace=False)
        
        # Display images
        for i, img_idx in enumerate(sample_indices):
            if i < n_per_class:
                ax = axes[cluster_idx, i]
                img = data_array[img_idx].values
                ax.imshow(img, cmap='viridis')
                ax.set_title(f"{label_map[cluster_idx]}\nImage {img_idx}")
                ax.axis('off')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()

# Function to analyze the distribution of classes
def analyze_distribution(labels, label_map):
    """Analyze and visualize the distribution of classes."""
    unique_labels, counts = np.unique(labels, return_counts=True)
    
    # Create a bar chart
    plt.figure(figsize=(8, 5))
    bars = plt.bar([label_map[label] for label in unique_labels], counts)
    
    # Add count labels on top of bars
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, 
                 bar.get_height() + 5, 
                 str(count), 
                 ha='center', va='bottom')
    
    plt.title('Distribution of Diffraction Pattern Classes')
    plt.ylabel('Number of Images')
    plt.tight_layout()
    plt.show()
    
    # Print percentages
    total = len(labels)
    print("Class Distribution:")
    for label in unique_labels:
        percent = 100 * np.sum(labels == label) / total
        print(f"{label_map[label]}: {np.sum(labels == label)} images ({percent:.2f}%)")


In [ ]:
pnccd = high_fluence_pnccd

# First, inspect the loaded data
print(f"Data shape: {pnccd.shape}")
print(f"Number of images: {pnccd.dims[0]}")
print(f"Image dimensions: {pnccd.shape[1]} x {pnccd.shape[2]}")

# Display a few example images
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, ax in enumerate(axes):
    idx = np.random.randint(0, pnccd.shape[0])
    ax.imshow(pnccd[idx].values, cmap='viridis')
    ax.set_title(f"Image {idx}")
    ax.axis('off')
plt.tight_layout()
plt.show()

# Set sample size for efficient processing
# Use a smaller sample for initial model training, then classify all images
sample_size = 200  # Adjust based on computational resources

# Run the classification
labels, kmeans, scaler, sampled_indices = classify_diffraction_patterns(
    pnccd, sample_size=sample_size
)

# Determine which cluster is monomer/dimer
label_map = determine_cluster_identity(pnccd, sampled_indices, labels)
print(f"Cluster mapping: {label_map}")

# Visualize classification results
visualize_classification(pnccd, labels, label_map, n_per_class=3)

# Analyze distribution of classes
analyze_distribution(labels, label_map)

# Create a new xarray DataArray with the labels
label_da = xr.DataArray(
    labels,
    dims=['trainId'],
    coords={'trainId': pnccd.coords['trainId']}
)

# Find indices of each class
monomer_indices = np.where(labels == [k for k, v in label_map.items() if 'Monomer' in v][0])[0]
dimer_indices = np.where(labels == [k for k, v in label_map.items() if 'Dimer' in v][0])[0]

print(f"Found {len(monomer_indices)} monomers and {len(dimer_indices)} dimers")

# Optional: Save the classification results
# label_da.to_netcdf('diffraction_labels.nc')


In [ ]:
# Add these imports to your existing code
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

def visualize_clusters_2d(data_array, features_scaled, labels, kmeans, label_map, method='pca'):
    """
    Visualize the clustering results in 2D space using dimensionality reduction.
    
    Parameters:
    -----------
    data_array : xarray.DataArray
        The original image data array
    features_scaled : array
        The scaled feature array used for clustering
    labels : array
        Cluster labels assigned to each data point
    kmeans : KMeans
        The fitted KMeans model
    label_map : dict
        Mapping from cluster indices to meaningful labels
    method : str
        Dimensionality reduction method ('pca' or 'tsne')
    """
    # Apply dimensionality reduction
    if method.lower() == 'pca':
        print("Applying PCA for dimensionality reduction...")
        reducer = PCA(n_components=2)
        reduced_features = reducer.fit_transform(features_scaled)
        title_prefix = "PCA"
        
        # Calculate explained variance
        explained_variance = reducer.explained_variance_ratio_.sum() * 100
        title_suffix = f"(Explained variance: {explained_variance:.2f}%)"
    else:
        print("Applying t-SNE for dimensionality reduction...")
        reducer = TSNE(n_components=2, random_state=42, perplexity=30)
        reduced_features = reducer.fit_transform(features_scaled)
        title_prefix = "t-SNE"
        title_suffix = ""
    
    # Create a figure
    plt.figure(figsize=(12, 10))
    
    # Create custom colormap
    colors = ['#1f77b4', '#ff7f0e']  # Blue and orange
    cmap = ListedColormap(colors)
    
    # Plot data points
    scatter = plt.scatter(reduced_features[:, 0], reduced_features[:, 1], 
                 c=labels, cmap=cmap, alpha=0.7, s=50)
    
    # If using PCA, also show cluster centers
    if method.lower() == 'pca':
        # Transform cluster centers
        centers_reduced = reducer.transform(kmeans.cluster_centers_)
        
        # Plot cluster centers
        plt.scatter(centers_reduced[:, 0], centers_reduced[:, 1], 
                   marker='X', s=200, c='red', label='Cluster Centers')
        
        # Draw decision boundary (for visual clarity, only draw a limited sample)
        # This is a simplified approximation of the decision boundary
        x_min, x_max = reduced_features[:, 0].min() - 1, reduced_features[:, 0].max() + 1
        y_min, y_max = reduced_features[:, 1].min() - 1, reduced_features[:, 1].max() + 1
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
        
        # For visualization purposes, we use the 2D PCA space to approximate decision boundary
        # This isn't the true boundary in the original feature space
        xy = np.vstack([xx.ravel(), yy.ravel()]).T
        Z = kmeans.predict(reducer.inverse_transform(xy))
        Z = Z.reshape(xx.shape)
        
        # Plot decision boundary with transparency
        plt.contour(xx, yy, Z, colors='k', linestyles='-', alpha=0.3)
    
    # Create legend with cluster labels
    legend_elements = []
    for i in range(len(label_map)):
        legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                               markerfacecolor=colors[i], markersize=10, 
                               label=label_map[i]))
    
    # Add cluster centers to legend
    legend_elements.append(plt.Line2D([0], [0], marker='X', color='w', 
                          markerfacecolor='red', markersize=10, 
                          label='Cluster Centers'))
    
    plt.legend(handles=legend_elements, loc='best', fontsize=12)
    
    # Set plot title and labels
    plt.title(f"{title_prefix} Visualization of Diffraction Pattern Clusters {title_suffix}", fontsize=16)
    plt.xlabel(f"{title_prefix} Component 1", fontsize=14)
    plt.ylabel(f"{title_prefix} Component 2", fontsize=14)
    
    # Add grid
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Improve layout
    plt.tight_layout()
    plt.show()
    
    # Display example images from each cluster
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f"Example images from each cluster", fontsize=16)
    
    for cluster_idx in range(2):
        cluster_indices = np.where(labels == cluster_idx)[0]
        if len(cluster_indices) == 0:
            continue
            
        # Sample 3 images (or fewer if not enough)
        sample_indices = np.random.choice(cluster_indices, min(3, len(cluster_indices)), replace=False)
        
        for j, img_idx in enumerate(sample_indices):
            if j < 3:
                ax = axes[cluster_idx, j]
                img = data_array[img_idx].values
                im = ax.imshow(img, cmap='viridis')
                ax.set_title(f"{label_map[cluster_idx]}\nImage {img_idx}")
                ax.axis('off')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()

    return reduced_features

def visualize_feature_importance(kmeans, scaler, label_map, n_features=10):
    """
    Visualize the most important features that distinguish the clusters.
    """
    # Calculate the absolute difference between cluster centers
    center_diff = np.abs(kmeans.cluster_centers_[0] - kmeans.cluster_centers_[1])
    
    # Get the indices of the top differentiating features
    top_indices = np.argsort(center_diff)[-n_features:]
    
    # Create feature names
    feature_names = []
    for i in range(len(center_diff)):
        if i < 40:  # First 40 are radial features
            feature_names.append(f"Radial_{i}")
        else:  # Remaining are azimuthal features
            idx = (i - 40) // 2
            if (i - 40) % 2 == 0:
                feature_names.append(f"Azimuthal_var_{idx}")
            else:
                feature_names.append(f"Azimuthal_ratio_{idx}")
    
    # Calculate the original values (not scaled)
    centers_original = scaler.inverse_transform(kmeans.cluster_centers_)
    
    # Plot
    plt.figure(figsize=(14, 8))
    
    # Plot a bar for each top feature
    y_pos = np.arange(n_features)
    
    # Plot values for cluster 0
    plt.barh(y_pos - 0.2, centers_original[0, top_indices], height=0.4, 
            color='#1f77b4', alpha=0.7, label=label_map[0])
    
    # Plot values for cluster 1
    plt.barh(y_pos + 0.2, centers_original[1, top_indices], height=0.4, 
            color='#ff7f0e', alpha=0.7, label=label_map[1])
    
    # Set labels and title
    plt.yticks(y_pos, [feature_names[i] for i in top_indices])
    plt.xlabel('Feature Value', fontsize=12)
    plt.title('Top Differentiating Features Between Clusters', fontsize=16)
    plt.legend()
    
    # Improve layout
    plt.tight_layout()
    plt.show()


In [ ]:
# Run this line first to enable interactive mode in your Jupyter notebook 
%matplotlib notebook

def interactive_tsne_plot(pnccd, reduced_features, labels, label_map):
    """
    Create an interactive t-SNE plot where clicking on a point displays the corresponding image.
    
    Parameters:
    -----------
    pnccd : xarray.DataArray
        The original diffraction pattern images
    reduced_features : numpy.ndarray
        The t-SNE embedding coordinates from visualize_clusters_2d
    labels : numpy.ndarray
        Cluster labels for each image
    label_map : dict
        Mapping from cluster indices to meaningful labels
    """
    from sklearn.neighbors import NearestNeighbors
    import matplotlib.pyplot as plt
    
    # Create a nearest neighbors model for finding closest point to click
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(reduced_features)
    
    # Set up the figure with two subplots side by side
    fig, (ax_tsne, ax_img) = plt.subplots(1, 2, figsize=(18, 8))
    
    # Create scatter plot on the first axis
    colors = ['#1f77b4', '#ff7f0e']  # Blue for monomers, orange for dimers
    
    for i, (cluster_idx, cluster_name) in enumerate(label_map.items()):
        mask = labels == cluster_idx
        ax_tsne.scatter(
            reduced_features[mask, 0], 
            reduced_features[mask, 1],
            c=colors[i], 
            alpha=0.7, 
            s=50, 
            label=cluster_name
        )
    
    # Add cluster centers if available
    ax_tsne.scatter(
        [reduced_features[np.where(labels == k)[0][0], 0] for k in label_map.keys()],
        [reduced_features[np.where(labels == k)[0][0], 1] for k in label_map.keys()],
        marker='X',
        s=200,
        c='red',
        label='Cluster Centers'
    )
    
    # Add labels and legend to t-SNE plot
    ax_tsne.set_title("t-SNE Visualization (Click on a point to view image)", fontsize=16)
    ax_tsne.set_xlabel("t-SNE Component 1", fontsize=14)
    ax_tsne.set_ylabel("t-SNE Component 2", fontsize=14)
    ax_tsne.legend(fontsize=12)
    ax_tsne.grid(True, linestyle='--', alpha=0.7)
    
    # Initial display in image subplot
    img_placeholder = np.zeros((pnccd.shape[1], pnccd.shape[2]))
    img_display = ax_img.imshow(img_placeholder, cmap='viridis')
    ax_img.set_title("Click on a point to display its image", fontsize=14)
    ax_img.axis('off')
    
    # For highlighting the selected point
    highlight, = ax_tsne.plot([], [], 'o', markersize=12, color='red', fillstyle='none', mew=2)
    
    # Function to handle click events
    def onclick(event):
        # Check if the click is within the scatter plot axes
        if event.inaxes != ax_tsne:
            return
        
        # Get the coordinates of the click
        click_x, click_y = event.xdata, event.ydata
        click_point = np.array([[click_x, click_y]])
        
        # Find the nearest point in the t-SNE space
        distances, indices = nbrs.kneighbors(click_point)
        nearest_idx = indices[0][0]
        
        # Update highlight position
        highlight.set_data([reduced_features[nearest_idx, 0]], [reduced_features[nearest_idx, 1]])
        
        # Get the image and its label
        img = pnccd[nearest_idx].values
        cluster_label = labels[nearest_idx]
        label_text = label_map[cluster_label]
        
        # Update the image display
        img_display.set_data(img)
        img_display.set_clim(vmin=np.min(img), vmax=np.max(img))
        ax_img.set_title(f"{label_text} - Image #{nearest_idx}", fontsize=14)
        
        # Redraw the figure to update the plot
        fig.canvas.draw_idle()
    
    # Connect the click event to the handler function
    fig.canvas.mpl_connect('button_press_event', onclick)
    
    # Make sure layout looks good
    plt.tight_layout()
    plt.show()


In [ ]:
interactive_tsne_plot(pnccd=high_fluence_pnccd, 
                     reduced_features=reduced_features, 
                     labels=labels, 
                     label_map=label_map)


In [ ]:
# Assuming you have the following variables:
# - pnccd: xarray.DataArray containing diffraction pattern images
# - features_scaled: the scaled features used for t-SNE
# - labels: cluster assignments for each image
# - label_map: dict mapping from cluster index to label name
# - reduced_features: the t-SNE embedding coordinates

# Create the interactive plot
interactive_tsne_plot(pnccd, all_features_scaled, labels, label_map)


## the working CNN